In [ ]:
import os
import pandas as pd
import numpy as np
import re
from datetime import date, datetime, timedelta
import warnings
import glob 
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

# Desactiva las advertencias
warnings.filterwarnings('ignore')

# Columnas por importancia SIC

In [3]:
columnas_codigos = ['SOSP_DIAG_COD_2', 'SOSP_DIAG_COD_1', 'SOSP_DIAG_COD_3', 'SOSP_DIAG_COD_4', 'SOSP_DIAG_COD_5']
columnas_diagnosticos = ['SOSP_DIAG_DESC_2', 'SOSP_DIAG_DESC_1', 'SOSP_DIAG_DESC_3', 'SOSP_DIAG_DESC_4', 'SOSP_DIAG_DESC_5']


# Funciones

In [4]:
def cargar_dataframe_ambulatorio(ruta):
    encoding = 'ISO-8859-1'
    delimitador = ';'
    return pd.read_csv(ruta, delimiter=delimitador, encoding=encoding, on_bad_lines='skip')


In [5]:
def gestion_fecha(dataframe):
    fecha_hoy = datetime.today()
    fecha_inicio = fecha_hoy - timedelta(days=10)
    fecha_inicio = pd.to_datetime(fecha_inicio, format='%d/%m/%Y')
    #fecha_inicio = pd.to_datetime('01/01/2023', format='%d/%m/%Y')
    return dataframe[dataframe['FECHA_SOLIC'] >= fecha_inicio]

In [6]:
import pandas as pd
import re



def buscar_diagnostico(fila, patrones):
        if fila['PROB_SALUD'] != '':
            return fila['PROB_SALUD']
        for columna in columnas_diagnosticos:

            if pd.notna(fila[columna]) and fila[columna] != '' and fila['PROB_SALUD'] == '':
                
                #if columna == 'SOSP_DIAG_DESC_2'  and fila[columna] == np.nan:
                #    continue
                for patron, diagnostico, condicion in patrones:
                    if re.search(patron, fila[columna]):
                        if condicion is None or condicion(fila):
                            return diagnostico
                break
        return ''

def buscar_diagnostico_cod(fila, patrones):
    if (fila['SOSP_DIAG_DESC_2'] != '' and fila['SOSP_DIAG_DESC_2'] != 'nan') and fila['SOSP_DIAG_COD_2'] == np.nan:
            return ''
    for columna in columnas_codigos:
        if pd.notna(fila[columna]) and fila[columna] != '' and fila['PROB_SALUD'] == '':
            #if columna == 'SOSP_DIAG_COD_2'  and fila[columna] == np.nan:
            #        print('no hay segundo diagnostico')
            #        continue
            for patron, diagnostico, condicion in patrones:
                #print(f"patron {patron}")
                #print(f"re.search(patron, fila[columna]) {re.search(patron, fila[columna])}")
                if re.search(patron, fila[columna]):
                    
                    if condicion is None or condicion(fila):
                        return diagnostico
            break
            
    return ''

def filtro_dataframe_ca_colorectal(dataframe, columnas_codigos, columnas_diagnosticos):
    

    # Se crea un patrón regex para identificar los códigos CIE-10 asociados al cáncer colorrectal
    patron_cancer_colorrectal = r'^C18.*|C19.*|C20.*'
    
    # Filtrar el DataFrame para encontrar las filas que contienen el código de cáncer colorrectal
    filtro_codigos = dataframe[columnas_codigos].apply(lambda x: x.str.contains(patron_cancer_colorrectal, na=False)).any(axis=1)

    #dataframe["SOSPECHA_DIAG"] = dataframe["SOSPECHA_DIAG"].astype(str)
    #dataframe["COMENTARIO_SOLICITUD"] = dataframe["COMENTARIO_SOLICITUD"].astype(str)


   # Patrones para identificar diagnósticos de cáncer colorrectal en texto libre
    patron_diagnostico_colorrectal = re.compile(r'\bC18\.|\bC19\.|\bC20\.|\b(?:c[aáx[^mn]]*.*col[oó]n.*|\bc[aáx[^mn]]*.*rect[oa])\b', re.IGNORECASE)
    patron_adenocarcinoma_colorrectal = re.compile(r'\badenocarcinoma.*rect[a-zá-ú]*', re.IGNORECASE)
    patron_carcinoma_escamoso_colorrectal = re.compile(r'\bcarcinoma.*escamoso.*rect[a-zá-ú]*', re.IGNORECASE)
    patron_tumor_maligno_colon = re.compile(r'\btumor.*maligno.*colon', re.IGNORECASE)
    patron_tumor_maligno_recto = re.compile(r'\btumor.*maligno.*recto', re.IGNORECASE)
    patron_neoplasia_maligna_colon = re.compile(r'\bneoplasia.*maligna.*colon', re.IGNORECASE)
    patron_neoplasia_maligna_recto = re.compile(r'\bneoplasia.*maligna.*recto', re.IGNORECASE)
    patron_cancer_colorrectal_metastasico = re.compile(r'\bc[a-zá-ú]*[ \.]*rect[a-zá-ú]*.*metast[a-zá-ú]*', re.IGNORECASE)
    patron_cancer_colon_avanzado = re.compile(r'\bc[a-zá-ú]*[ \.]*colon.*avanzado', re.IGNORECASE)
    patron_cancer_recto_avanzado = re.compile(r'\bc[a-zá-ú]*[ \.]*rect[a-zá-ú]*.*avanzado', re.IGNORECASE)


    # Términos acompañados por otros
    patron_neoplasia_ubicaciones = re.compile(r'\bneoplasia.*(?:colon\b|recto\b|rectal\b|transverso\b|sigmoides\b|apéndice\b|apendicular\b|neuroendocrino\b canal anal\b|conducto anal\b)', re.IGNORECASE)
    patron_adenoca_ubicaciones = re.compile(r'\badenoca.*(?:colon\b|recto\b|rectal\b|transverso\b|sigmoides\b|apéndice\b|apendicular\b|neuroendocrino\b|canal anal\b|conducto anal\b)', re.IGNORECASE)
    patron_gist_ubicaciones = re.compile(r'\bgist.*(?:colon\b|recto\b|rectal|transverso\b|sigmoides\b|apéndice\b|apendicular\b|neuroendocrino\b|canal anal\b|conducto anal\b)', re.IGNORECASE)
    patron_cancer_ubicaciones = re.compile(r'\bca[aáx[^mn](?:ncer)[ \.]*?.*(?:colon\b|recto\b|rectal\b|transverso\b|sigmoides\b|apéndice\b|apendicular\b|neuroendocrino\b|canal anal\b|conducto anal\b)', re.IGNORECASE)
    patron_tumor_ubicaciones = re.compile(r'\btu(?:mor)?.*(?:colon\b|recto\b|rectal\b|transverso\b|sigmoides\b|apéndice\b|apendicular\b|neuroendocrino\b|canal anal\b|conducto anal\b)', re.IGNORECASE)

    
    # Asegurarse de que las columnas de diagnóstico sean de tipo string
    dataframe[columnas_diagnosticos] = dataframe[columnas_diagnosticos].astype(str)
    dataframe[columnas_codigos] = dataframe[columnas_codigos].astype(str)
    dataframe[columnas_diagnosticos] = dataframe[columnas_diagnosticos].replace('nan', np.nan)
    dataframe[columnas_codigos] = dataframe[columnas_codigos].replace('nan', np.nan)
    # Inicializar la columna 'PROB_SALUD' con valores vacíos
    dataframe['PROB_SALUD'] = ''

    # Definir una función para buscar en las columnas en orden
    
    
    # Aplicar la función de búsqueda de diagnósticos
    patrones = [(patron_cancer_colorrectal, 'Cáncer Colorectal', None)]
    dataframe['PROB_SALUD'] = dataframe.apply(buscar_diagnostico_cod, axis=1, patrones=patrones)
    
    
    # Aplicar la función de búsqueda de diagnósticos
    patrones = [(patron_diagnostico_colorrectal, 'Cáncer Colorectal', None), 
                (patron_adenocarcinoma_colorrectal, 'Cáncer Colorectal', None),
                (patron_carcinoma_escamoso_colorrectal, 'Cáncer Colorectal', None),
                (patron_tumor_maligno_colon, 'Cáncer Colorectal', None),
                (patron_tumor_maligno_recto, 'Cáncer Colorectal', None),
                (patron_neoplasia_maligna_colon, 'Cáncer Colorectal', None),
                (patron_neoplasia_maligna_recto, 'Cáncer Colorectal', None),
                (patron_cancer_colorrectal_metastasico, 'Cáncer Colorectal', None),
                (patron_cancer_colon_avanzado, 'Cáncer Colorectal', None),
                (patron_cancer_recto_avanzado, 'Cáncer Colorectal', None),
                (patron_neoplasia_ubicaciones, 'Cáncer Colorectal', None),
                (patron_adenoca_ubicaciones, 'Cáncer Colorectal', None),
                (patron_gist_ubicaciones, 'Cáncer Colorectal', None),
                (patron_cancer_ubicaciones, 'Cáncer Colorectal', None),
                (patron_tumor_ubicaciones, 'Cáncer Colorectal', None)]
    dataframe['PROB_SALUD'] = dataframe.apply(buscar_diagnostico, axis=1, patrones=patrones)

   

    

    
    
    # Filtrar el DataFrame para encontrar las filas que contienen el diagnóstico de cáncer colorrectal en texto libre
    filtro_diagnosticos = dataframe[columnas_diagnosticos].apply(lambda x:
        x.str.contains(patron_diagnostico_colorrectal, na=False) |
        x.str.contains(patron_adenocarcinoma_colorrectal, na=False) |
        x.str.contains(patron_carcinoma_escamoso_colorrectal, na=False) |
        x.str.contains(patron_tumor_maligno_colon, na=False) |
        x.str.contains(patron_tumor_maligno_recto, na=False) |
        x.str.contains(patron_neoplasia_maligna_colon, na=False) |
        x.str.contains(patron_neoplasia_maligna_recto, na=False) |
        x.str.contains(patron_cancer_colorrectal_metastasico, na=False) |
        x.str.contains(patron_cancer_colon_avanzado, na=False) |
        x.str.contains(patron_cancer_recto_avanzado, na=False) |
        x.str.contains(patron_neoplasia_ubicaciones, na=False) |
        x.str.contains(patron_adenoca_ubicaciones, na=False) |
        x.str.contains(patron_gist_ubicaciones, na=False) |
        x.str.contains(patron_cancer_ubicaciones, na=False) |
        x.str.contains(patron_tumor_ubicaciones, na=False)
        ).any(axis=1)

    # Combinar los filtros para obtener las filas que cumplen con al menos uno de los criterios
    return dataframe[(filtro_diagnosticos|filtro_codigos)].copy()

def filtro_dataframe_ca_gastrico_inter(dataframe, columnas_codigos, columnas_diagnosticos):
    # Definir las columnas de códigos y de diagnósticos en texto libre
    #columnas_codigos = ['SOSP_DIAG_COD_1', 'SOSP_DIAG_COD_2', 'SOSP_DIAG_COD_3', 'SOSP_DIAG_COD_4', 'SOSP_DIAG_COD_5']
    #columnas_diagnosticos = ['SOSP_DIAG_DESC_1', 'SOSP_DIAG_DESC_2', 'SOSP_DIAG_DESC_3', 'SOSP_DIAG_DESC_4', 'SOSP_DIAG_DESC_5']

    # Patrón para identificar los códigos CIE-10 asociados al cáncer gástrico
    patron_cancer_gastrico = r'^C16.*'

    # Filtrar el DataFrame para encontrar las filas que contienen el código de cáncer gástrico
    filtro_codigos = dataframe[columnas_codigos].apply(lambda x: x.str.contains(patron_cancer_gastrico, na=False)).any(axis=1)

    # Patrones para identificar diagnósticos de cáncer gástrico en texto libre
    patron_adenocarcinoma_gastrico = re.compile(r'adenocarcinoma.*g[aá]strico', re.IGNORECASE)
    patron_carcinoma_gastrico = re.compile(r'carcinoma.*g[aá]strico', re.IGNORECASE)
    patron_linfoma_gastrico = re.compile(r'linfoma.*g[aá]strico', re.IGNORECASE)
    patron_sarcoma_gastrico = re.compile(r'sarcoma.*g[aá]strico', re.IGNORECASE)
    patron_neoplasia_gastrica = re.compile(r'neoplasia.*g[aá]strica', re.IGNORECASE)
    patron_tumor_estomago = re.compile(r'tumor.*est[oó]mago.*', re.IGNORECASE)
    patron_cancer_estomago = re.compile(r'c[aá]ncer.*est[oó]mago', re.IGNORECASE)
    patron_adenocarcinoma_intestinal = re.compile(r'adenocarcinoma.*intestinal', re.IGNORECASE)
    patron_adenocarcinoma_difuso = re.compile(r'adenocarcinoma.*difuso', re.IGNORECASE)
    patron_carcinoma_anillo_sello = re.compile(r'carcinoma.*anillo.*sello', re.IGNORECASE)
    patron_cancer_gastrico_metastasico = re.compile(r'c[aá]ncer.*g[aá]strico.*metast[aá]sico', re.IGNORECASE)
    patron_tumor_gist_gastrico = re.compile(r'tumor.*g[aá]strico', re.IGNORECASE)

    dataframe[columnas_diagnosticos] = dataframe[columnas_diagnosticos].astype(str)
    dataframe[columnas_codigos] = dataframe[columnas_codigos].astype(str)
    dataframe[columnas_diagnosticos] = dataframe[columnas_diagnosticos].replace('nan', np.nan)
    dataframe[columnas_codigos] = dataframe[columnas_codigos].replace('nan', np.nan)
    # Inicializar la columna 'PROB_SALUD' con valores vacíos
    dataframe['PROB_SALUD'] = ''

    # Aplicar la función de búsqueda de diagnósticos
    patrones = [(patron_cancer_gastrico, 'Cáncer Gástrico', None)]
    dataframe['PROB_SALUD'] = dataframe.apply(buscar_diagnostico_cod, axis=1, patrones=patrones)
    
    
    # Aplicar la función de búsqueda de diagnósticos
    patrones = [(patron_adenocarcinoma_gastrico, 'Cáncer Gástrico', None), 
                (patron_carcinoma_gastrico, 'Cáncer Gástrico', None),
                (patron_linfoma_gastrico, 'Cáncer Gástrico', None),
                (patron_sarcoma_gastrico, 'Cáncer Gástrico', None),
                (patron_neoplasia_gastrica, 'Cáncer Gástrico', None),
                (patron_tumor_estomago, 'Cáncer Gástrico', None),
                (patron_cancer_estomago, 'Cáncer Gástrico', None),
                (patron_adenocarcinoma_intestinal, 'Cáncer Gástrico', None),
                (patron_adenocarcinoma_difuso, 'Cáncer Gástrico', None),
                (patron_carcinoma_anillo_sello, 'Cáncer Gástrico', None),
                (patron_cancer_gastrico_metastasico, 'Cáncer Gástrico', None),
                (patron_tumor_gist_gastrico, 'Cáncer Gástrico', None)]
    dataframe['PROB_SALUD'] = dataframe.apply(buscar_diagnostico, axis=1, patrones=patrones)


    filtro_diagnosticos = dataframe[columnas_diagnosticos].apply(lambda x:
        x.str.contains(patron_adenocarcinoma_gastrico, na=False) |
        x.str.contains(patron_carcinoma_gastrico, na=False) |
        x.str.contains(patron_linfoma_gastrico, na=False) |
        x.str.contains(patron_sarcoma_gastrico, na=False) |
        x.str.contains(patron_neoplasia_gastrica, na=False) |
        x.str.contains(patron_tumor_estomago, na=False) |
        x.str.contains(patron_cancer_estomago, na=False) |
        x.str.contains(patron_adenocarcinoma_intestinal, na=False) |
        x.str.contains(patron_adenocarcinoma_difuso, na=False) |
        x.str.contains(patron_carcinoma_anillo_sello, na=False) |
        x.str.contains(patron_cancer_gastrico_metastasico, na=False) |
        x.str.contains(patron_tumor_gist_gastrico, na=False)).any(axis=1)
    
    # Filtrar por la comuna La Florida
    filtro_comuna = dataframe['COMUNA'] == 'La Florida'

    # Combinar los filtros para obtener las filas que cumplen con al menos uno de los criterios
    return dataframe[(filtro_codigos | filtro_diagnosticos )].copy()

def filtro_dataframe_ca_tiroides_inter(dataframe, columnas_codigos, columnas_diagnosticos):
    # Definir las columnas de códigos y de diagnósticos en texto libre
    columnas_codigos = ['SOSP_DIAG_COD_1', 'SOSP_DIAG_COD_2', 'SOSP_DIAG_COD_3', 'SOSP_DIAG_COD_4', 'SOSP_DIAG_COD_5']
    columnas_diagnosticos = ['SOSP_DIAG_DESC_1', 'SOSP_DIAG_DESC_2', 'SOSP_DIAG_DESC_3', 'SOSP_DIAG_DESC_4', 'SOSP_DIAG_DESC_5']

    # Se crea un patrón regex para identificar los códigos CIE-! asociados al cáncer de tiroides
    patron_cancer_tiroides = r'^C73.*'

    # Filtrar el DataFrame para encontrar las filas que contienen el código de cáncer de tiroides
    filtro_codigos = dataframe[columnas_codigos].apply(lambda x: x.str.contains(patron_cancer_tiroides, na=False)).any(axis=1)

    # Patrones para identificar diagnósticos de cáncer de tiroides en texto libre
    #patron_diagnostico_tiroides = re.compile(r'\bc[a-zá-ú]*[ \.]*tiroide[a-zá-ú]*\b', re.IGNORECASE)
    patron_diagnostico_tiroides = re.compile(r'\bc(?!oe).*tiroide(?!o)[a-zá-ú]*\b', re.IGNORECASE)

    patron_carci_in_situ = re.compile(r'carci.*tiroid[a-zá-ú]*', re.IGNORECASE)
    patron_tumor_maligno = re.compile(r'tumor.*maligno.*glandula tiroides|tumor.*maligno.*tiroid[a-zá-ú]*', re.IGNORECASE)
    patron_carci_in_situ_tiroides = re.compile(r'carci.*in\s*situ.*tiroid[a-zá-ú]*', re.IGNORECASE)
    patron_nodulos_tirad_4_5 = re.compile(r'n[oó]dulos.*tirad\s*[5]', re.IGNORECASE)


    # Asegurarse de que las columnas de diagnóstico sean de tipo string
    dataframe[columnas_diagnosticos] = dataframe[columnas_diagnosticos].astype(str)
    dataframe[columnas_codigos] = dataframe[columnas_codigos].astype(str)
    dataframe[columnas_diagnosticos] = dataframe[columnas_diagnosticos].replace('nan', np.nan)
    dataframe[columnas_codigos] = dataframe[columnas_codigos].replace('nan', np.nan)
    # Inicializar la columna 'PROB_SALUD' con valores vacíos
    dataframe['PROB_SALUD'] = ''

    # Definir una función para buscar en las columnas en orden
    
    
    # Aplicar la función de búsqueda de diagnósticos
    patrones = [(patron_cancer_tiroides, 'Cáncer Tiroides', None)]
    dataframe['PROB_SALUD'] = dataframe.apply(buscar_diagnostico_cod, axis=1, patrones=patrones)
    
    
    # Aplicar la función de búsqueda de diagnósticos
    patrones = [(patron_carci_in_situ, 'Cáncer Tiroides', None), 
                (patron_diagnostico_tiroides, 'Cáncer Tiroides', None),
                (patron_tumor_maligno, 'Cáncer Tiroides', None),
                (patron_carci_in_situ_tiroides, 'Cáncer Tiroides', None),
                (patron_nodulos_tirad_4_5, 'Cáncer Tiroides', None)]
    dataframe['PROB_SALUD'] = dataframe.apply(buscar_diagnostico, axis=1, patrones=patrones)

   



    # Filtrar el DataFrame para encontrar las filas que contienen el diagnóstico de cáncer de tiroides en texto libre
    filtro_diagnosticos = dataframe[columnas_diagnosticos].apply(lambda x:
        x.str.contains(patron_diagnostico_tiroides, na=False) |
        x.str.contains(patron_carci_in_situ, na=False) |
        x.str.contains(patron_tumor_maligno, na=False) |
        x.str.contains(patron_carci_in_situ_tiroides, na=False) |
        x.str.contains(patron_nodulos_tirad_4_5, na=False)).any(axis=1)

    # Filtrar por la comuna La Florida
    filtro_comuna = dataframe['COMUNA'] == 'La Florida'

    # Combinar los filtros para obtener las filas que cumplen con al menos uno de los criterios
    resultado_filtrado = dataframe[(filtro_codigos | filtro_diagnosticos)].copy()
    return resultado_filtrado

def filtro_dataframe_ca_pulmon_inter(dataframe, columnas_codigos, columnas_diagnosticos):
    
    # Se crea un patrón regex para identificar los códigos CIE-10 asociados al cáncer de pulmón
    patron_cancer_pulmon = r'^C34.*'
    # Filtrar el DataFrame para encontrar las filas que contienen el código de cáncer de pulmón
    filtro_codigos = dataframe[columnas_codigos].apply(lambda x: x.str.contains(patron_cancer_pulmon, na=False)).any(axis=1)

    # Patrones para identificar diagnósticos de cáncer de pulmón en texto libre
    patron_diagnostico_pulmon = re.compile(r'(\bc[aá]|\bc)([ .]+)?(\bpulm[oó]n|\bpulmonar)', re.IGNORECASE)
    patron_tumor_pulmon = re.compile(r'(\bt|\btumor)([ .]+)?(\bpulm[oó]n|\bpulmonar)', re.IGNORECASE)
    patron_carcinoma_pulmon = re.compile(r'(\bc|\bcarcinoma)([ .]+)?(\bpulm[oó]n|\bpulmonar)', re.IGNORECASE)

    # Asegurarse de que las columnas de diagnóstico sean de tipo string
    dataframe[columnas_diagnosticos] = dataframe[columnas_diagnosticos].astype(str)
    dataframe[columnas_codigos] = dataframe[columnas_codigos].astype(str)
    dataframe[columnas_diagnosticos] = dataframe[columnas_diagnosticos].replace('nan', np.nan)
    dataframe[columnas_codigos] = dataframe[columnas_codigos].replace('nan', np.nan)
    # Inicializar la columna 'PROB_SALUD' con valores vacíos
    dataframe['PROB_SALUD'] = ''

    # Definir una función para buscar en las columnas en orden
    
    
    # Aplicar la función de búsqueda de diagnósticos
    patrones = [(patron_cancer_pulmon, 'Cáncer Pulmón', None)]
    dataframe['PROB_SALUD'] = dataframe.apply(buscar_diagnostico_cod, axis=1, patrones=patrones)
    
    
    # Aplicar la función de búsqueda de diagnósticos
    patrones = [(patron_diagnostico_pulmon, 'Cáncer Pulmón', None), 
                (patron_tumor_pulmon, 'Cáncer Pulmón', None),
                (patron_carcinoma_pulmon, 'Cáncer Pulmón', None)]
    dataframe['PROB_SALUD'] = dataframe.apply(buscar_diagnostico, axis=1, patrones=patrones)

   


    # Filtrar el DataFrame para encontrar las filas que contienen el diagnóstico de cáncer de pulmón en texto libre
    filtro_diagnosticos = dataframe[columnas_diagnosticos].apply(lambda x:
        x.str.contains(patron_diagnostico_pulmon, na=False) |
        x.str.contains(patron_tumor_pulmon, na=False) |
        x.str.contains(patron_carcinoma_pulmon, na=False)).any(axis=1)


    # Filtrar por la comuna La Florida
    #filtro_comuna = dataframe['COMUNA'] == 'La Florida'

    # Combinar los filtros para obtener las filas que cumplen con al menos uno de los criterios
    resultado_filtrado = dataframe[filtro_codigos | filtro_diagnosticos].copy()
    return resultado_filtrado

def filtro_dataframe_ca_prostata_inter(dataframe, columnas_codigos, columnas_diagnosticos):
    
    # Se crea un patrón regex para identificar los códigos CIE-! asociados al cáncer de próstata
    patron_cancer_prostata = r'^C61.*'
    # Filtrar el DataFrame para encontrar las filas que contienen el código de cáncer de próstata
    filtro_codigos = dataframe[columnas_codigos].apply(lambda x: x.str.contains(patron_cancer_prostata, na=False)).any(axis=1)

    # Patrones para identificar diagnósticos de cáncer de próstata en texto libre
    patron_diagnostico_prostata_1 = re.compile(r'\bc(?!o)[a-zá-ú]*[ \.]*pr[oó]stat[aá][a-zá-ú]*\b', re.IGNORECASE)
    patron_diagnostico_prostata_2 = re.compile(r'\bc[aá][ncer]?.*pr[oó]stat[aá][a-zá-ú]*', re.IGNORECASE)
    patron_diagnostico_prostata_3 = re.compile(r'\btumor.*maligno.*pr[oó]stat[aá][a-zá-ú]*', re.IGNORECASE)
    patron_diagnostico_prostata_4 = re.compile(r'\bcarci.*noma.*pr[oó]stat[aá][a-zá-ú]*', re.IGNORECASE)
    patron_diagnostico_prostata_5 = re.compile(r'\bneoplasia.*pr[oó]stat[aá][a-zá-ú]*', re.IGNORECASE)
    patron_diagnostico_prostata_6 = re.compile(r'\badenocarcinoma.*pr[oó]stat[aá][a-zá-ú]*', re.IGNORECASE)
    patron_diagnostico_prostata_7 = re.compile(r'\btumor.*prostatic[aá][a-zá-ú]*', re.IGNORECASE)
    patron_diagnostico_prostata_8 = re.compile(r'\bcarci.*noma.*glandula.*prostatic[aá][a-zá-ú]*', re.IGNORECASE)


    # Asegurarse de que las columnas de diagnóstico sean de tipo string
    dataframe[columnas_diagnosticos] = dataframe[columnas_diagnosticos].astype(str)
    dataframe[columnas_codigos] = dataframe[columnas_codigos].astype(str)
    dataframe[columnas_diagnosticos] = dataframe[columnas_diagnosticos].replace('nan', np.nan)
    dataframe[columnas_codigos] = dataframe[columnas_codigos].replace('nan', np.nan)
    # Inicializar la columna 'PROB_SALUD' con valores vacíos
    dataframe['PROB_SALUD'] = ''

    # Definir una función para buscar en las columnas en orden
    
    
    # Aplicar la función de búsqueda de diagnósticos
    patrones = [(patron_cancer_prostata, 'Cáncer Próstata', None)]
    dataframe['PROB_SALUD'] = dataframe.apply(buscar_diagnostico_cod, axis=1, patrones=patrones)
    
    
    # Aplicar la función de búsqueda de diagnósticos
    patrones = [(patron_diagnostico_prostata_1, 'Cáncer Próstata', None), 
                (patron_diagnostico_prostata_2, 'Cáncer Próstata', None),
                (patron_diagnostico_prostata_3, 'Cáncer Próstata', None),
                (patron_diagnostico_prostata_4, 'Cáncer Próstata', None),
                (patron_diagnostico_prostata_5, 'Cáncer Próstata', None),
                (patron_diagnostico_prostata_6, 'Cáncer Próstata', None),
                (patron_diagnostico_prostata_7, 'Cáncer Próstata', None),
                (patron_diagnostico_prostata_8, 'Cáncer Próstata', None)]
    dataframe['PROB_SALUD'] = dataframe.apply(buscar_diagnostico, axis=1, patrones=patrones)

    # Filtrar el DataFrame para encontrar las filas que contienen el diagnóstico de cáncer de próstata en texto libre
    filtro_diagnosticos = dataframe[columnas_diagnosticos].apply(lambda x:
        x.str.contains(patron_diagnostico_prostata_1, na=False) |
        x.str.contains(patron_diagnostico_prostata_2, na=False) |
        x.str.contains(patron_diagnostico_prostata_3, na=False) |
        x.str.contains(patron_diagnostico_prostata_4, na=False) |
        x.str.contains(patron_diagnostico_prostata_5, na=False) |
        x.str.contains(patron_diagnostico_prostata_6, na=False) |
        x.str.contains(patron_diagnostico_prostata_7, na=False) |
        x.str.contains(patron_diagnostico_prostata_8, na=False)).any(axis=1)

    # Filtrar por la comuna La Florida
    #filtro_comuna = dataframe['COMUNA'] == 'La Florida'

    # Combinar los filtros para obtener las filas que cumplen con al menos uno de los criterios
    resultado_filtrado = dataframe[filtro_codigos | filtro_diagnosticos].copy()
    return resultado_filtrado

def filtro_dataframe_ca_renal_inter(dataframe, columnas_codigos, columnas_diagnosticos):
    # Definir las columnas de códigos y de diagnósticos en texto libre
    columnas_codigos = ['SOSP_DIAG_COD_1', 'SOSP_DIAG_COD_2', 'SOSP_DIAG_COD_3', 'SOSP_DIAG_COD_4', 'SOSP_DIAG_COD_5']
    columnas_diagnosticos = ['SOSP_DIAG_DESC_1', 'SOSP_DIAG_DESC_2', 'SOSP_DIAG_DESC_3', 'SOSP_DIAG_DESC_4', 'SOSP_DIAG_DESC_5']

    # Se crea un patrón regex para identificar los códigos CIE-10 asociados al cáncer renal
    patron_cancer_renal = r'^C64.*'

    # Filtrar el DataFrame para encontrar las filas que contienen el código de cáncer renal
    filtro_codigos = dataframe[columnas_codigos].apply(lambda x: x.str.contains(patron_cancer_renal, na=False)).any(axis=1)

    # Patrones para identificar diagnósticos de cáncer renal en texto libre
    patron_diagnostico_renal = re.compile(r'\bc[aá]*[ \.]*renal.*\b', re.IGNORECASE)
    patron_tumor_renal = re.compile(r'\btu[mor]?.*renal', re.IGNORECASE)
    patron_carcinoma_renal = re.compile(r'\bcarcinoma.*renal', re.IGNORECASE)
    patron_adenocarcinoma_renal = re.compile(r'\badenocarcinoma.*renal', re.IGNORECASE)
    patron_celulas_claras_renal = re.compile(r'\bcélulas claras.*renal', re.IGNORECASE)
    patron_celulas_renales_renal = re.compile(r'\bcélulas renales.*renal', re.IGNORECASE)
    patron_carcinoma_papilar_renal = re.compile(r'\bcarcinoma papilar.*renal', re.IGNORECASE)
    patron_transicionales_renal = re.compile(r'\bcélulas transicionales.*renal', re.IGNORECASE)
    patron_cromofobas_renal = re.compile(r'\bcélulas cromófobas.*renal', re.IGNORECASE)
    patron_medular_renal = re.compile(r'\bcarcinoma medular.*renal', re.IGNORECASE)
    patron_sarcomatoide_renal = re.compile(r'\bcarcinoma sarcomatoide.*renal', re.IGNORECASE)
    patron_granulares_renal = re.compile(r'\btumor.*renal.*células granulares', re.IGNORECASE)
    patron_wilms_renal = re.compile(r'\btumor de Wilms', re.IGNORECASE)
    patron_hipernefroma = re.compile(r'\bhipernefroma', re.IGNORECASE)

    # Asegurarse de que las columnas de diagnóstico sean de tipo string
    dataframe[columnas_diagnosticos] = dataframe[columnas_diagnosticos].astype(str)
    dataframe[columnas_codigos] = dataframe[columnas_codigos].astype(str)
    dataframe[columnas_diagnosticos] = dataframe[columnas_diagnosticos].replace('nan', np.nan)
    dataframe[columnas_codigos] = dataframe[columnas_codigos].replace('nan', np.nan)
    # Inicializar la columna 'PROB_SALUD' con valores vacíos
    dataframe['PROB_SALUD'] = ''

    # Definir una función para buscar en las columnas en orden
    
    
    # Aplicar la función de búsqueda de diagnósticos
    patrones = [(patron_cancer_renal, 'Cáncer Renal', None)]
    dataframe['PROB_SALUD'] = dataframe.apply(buscar_diagnostico_cod, axis=1, patrones=patrones)
    
    
    # Aplicar la función de búsqueda de diagnósticos
    patrones = [(patron_diagnostico_renal, 'Cáncer Renal', None), 
                (patron_tumor_renal, 'Cáncer Renal', None),
                (patron_carcinoma_renal, 'Cáncer Renal', None),
                (patron_adenocarcinoma_renal, 'Cáncer Renal', None),
                (patron_celulas_claras_renal, 'Cáncer Renal', None),
                (patron_celulas_renales_renal, 'Cáncer Renal', None),
                (patron_carcinoma_papilar_renal, 'Cáncer Renal', None),
                (patron_transicionales_renal, 'Cáncer Renal', None),
                (patron_cromofobas_renal, 'Cáncer Renal', None),
                (patron_medular_renal, 'Cáncer Renal', None),
                (patron_sarcomatoide_renal, 'Cáncer Renal', None),
                (patron_granulares_renal, 'Cáncer Renal', None),
                (patron_wilms_renal, 'Cáncer Renal', None),
                (patron_hipernefroma, 'Cáncer Renal', None)]
    dataframe['PROB_SALUD'] = dataframe.apply(buscar_diagnostico, axis=1, patrones=patrones)


    # Filtrar el DataFrame para encontrar las filas que contienen el diagnóstico de cáncer renal en texto libre
    filtro_diagnosticos = dataframe[columnas_diagnosticos].apply(lambda x:
        x.str.contains(patron_diagnostico_renal, na=False) |
        x.str.contains(patron_tumor_renal, na=False) |
        x.str.contains(patron_carcinoma_renal, na=False) |
        x.str.contains(patron_adenocarcinoma_renal, na=False) |
        x.str.contains(patron_celulas_claras_renal, na=False) |
        x.str.contains(patron_celulas_renales_renal, na=False) |
        x.str.contains(patron_carcinoma_papilar_renal, na=False) |
        x.str.contains(patron_transicionales_renal, na=False) |
        x.str.contains(patron_cromofobas_renal, na=False) |
        x.str.contains(patron_medular_renal, na=False) |
        x.str.contains(patron_sarcomatoide_renal, na=False) |
        x.str.contains(patron_granulares_renal, na=False) |
        x.str.contains(patron_hipernefroma, na=False) |
        x.str.contains(patron_wilms_renal, na=False)).any(axis=1)

    # Filtrar por la comuna La Florida
    #filtro_comuna = dataframe['COMUNA'] == 'La Florida'

    # Combinar los filtros para obtener las filas que cumplen con al menos uno de los criterios
    resultado_filtrado = dataframe[filtro_codigos | filtro_diagnosticos].copy()
    return resultado_filtrado

def filtro_dataframe_ca_testiculo_inter(dataframe, columnas_codigos, columnas_diagnosticos):
    # Definir las columnas de códigos y de diagnósticos en texto libre
    columnas_codigos = ['SOSP_DIAG_COD_1', 'SOSP_DIAG_COD_2', 'SOSP_DIAG_COD_3', 'SOSP_DIAG_COD_4', 'SOSP_DIAG_COD_5']
    columnas_diagnosticos = ['SOSP_DIAG_DESC_1', 'SOSP_DIAG_DESC_2', 'SOSP_DIAG_DESC_3', 'SOSP_DIAG_DESC_4', 'SOSP_DIAG_DESC_5']

    # Se crea un patrón regex para identificar los códigos CIE-! asociados al cáncer de testículo
    patron_cancer_testiculo = r'^C62.*'

    # Filtrar el DataFrame para encontrar las filas que contienen el código de cáncer de testículo
    filtro_codigos = dataframe[columnas_codigos].apply(lambda x: x.str.contains(patron_cancer_testiculo, na=False)).any(axis=1)

    # Patrones para identificar diagnósticos de cáncer de testículo en texto libre
    patron_diagnostico_testiculo = re.compile(r'\bc[ aá][ncer]?.*(test[ií]culo|escrot[oa]|orqui[td]ectom[ií]a)\b', re.IGNORECASE)
    patron_tumor_testicular = re.compile(r'\btumor.*test[ií]culo\b', re.IGNORECASE)
    patron_neoplasia_testicular = re.compile(r'\bneoplasia.*test[ií]culo\b', re.IGNORECASE)
    patron_carcinoma_embrionario = re.compile(r'\bcarcinoma.*embrionario\b', re.IGNORECASE)
    patron_seminoma = re.compile(r'\bseminoma\b', re.IGNORECASE)
    patron_teratoma_testicular = re.compile(r'\bteratoma.*test[ií]culo\b', re.IGNORECASE)
    patron_metastasis_testicular = re.compile(r'\bmet[aá]stasis.*test[ií]culo\b', re.IGNORECASE)

    # Asegurarse de que las columnas de diagnóstico sean de tipo string
    dataframe[columnas_diagnosticos] = dataframe[columnas_diagnosticos].astype(str)
    dataframe[columnas_codigos] = dataframe[columnas_codigos].astype(str)
    dataframe[columnas_diagnosticos] = dataframe[columnas_diagnosticos].replace('nan', np.nan)
    dataframe[columnas_codigos] = dataframe[columnas_codigos].replace('nan', np.nan)
    # Inicializar la columna 'PROB_SALUD' con valores vacíos
    dataframe['PROB_SALUD'] = ''

    # Definir una función para buscar en las columnas en orden
    
    
    # Aplicar la función de búsqueda de diagnósticos
    patrones = [(patron_cancer_testiculo, 'Cáncer Testículo', None)]
    dataframe['PROB_SALUD'] = dataframe.apply(buscar_diagnostico_cod, axis=1, patrones=patrones)
    
    
    # Aplicar la función de búsqueda de diagnósticos
    patrones = [(patron_diagnostico_testiculo, 'Cáncer Testículo', None), 
                (patron_tumor_testicular, 'Cáncer Testículo', None),
                (patron_neoplasia_testicular, 'Cáncer Testículo', None),
                (patron_carcinoma_embrionario, 'Cáncer Testículo', None),
                (patron_carcinoma_embrionario, 'Cáncer Testículo', None),
                (patron_seminoma, 'Cáncer Testículo', None),
                (patron_teratoma_testicular, 'Cáncer Testículo', None),
                (patron_metastasis_testicular, 'Cáncer Testículo', None)]
    dataframe['PROB_SALUD'] = dataframe.apply(buscar_diagnostico, axis=1, patrones=patrones)


    # Filtrar el DataFrame para encontrar las filas que contienen los diagnósticos de cáncer de testículo en texto libre
    filtro_diagnosticos = dataframe[columnas_diagnosticos].apply(lambda x:
        x.str.contains(patron_diagnostico_testiculo, na=False) |
        x.str.contains(patron_tumor_testicular, na=False) |
        x.str.contains(patron_neoplasia_testicular, na=False) |
        x.str.contains(patron_carcinoma_embrionario, na=False) |
        x.str.contains(patron_seminoma, na=False) |
        x.str.contains(patron_teratoma_testicular, na=False) |
        x.str.contains(patron_metastasis_testicular, na=False)).any(axis=1)


    # Filtrar por la comuna La Florida
    #filtro_comuna = dataframe['COMUNA'] == 'La Florida'

    # Combinar los filtros para obtener las filas que cumplen con al menos uno de los criterios
    resultado_filtrado = dataframe[filtro_codigos | filtro_diagnosticos].copy()
    return resultado_filtrado

def filtro_dataframe_ca_leucemia_inter(dataframe, columnas_codigos, columnas_diagnosticos):
    # Definir las columnas de códigos y de diagnósticos en texto libre
    columnas_codigos = ['SOSP_DIAG_COD_1', 'SOSP_DIAG_COD_2', 'SOSP_DIAG_COD_3', 'SOSP_DIAG_COD_4', 'SOSP_DIAG_COD_5']
    columnas_diagnosticos = ['SOSP_DIAG_DESC_1', 'SOSP_DIAG_DESC_2', 'SOSP_DIAG_DESC_3', 'SOSP_DIAG_DESC_4', 'SOSP_DIAG_DESC_5']

    # Códigos CIE-10 para cánceres hematológicos
    patron_leucemia = r'^C91.*|C92.*|C93.*|C94.*|C95.*'  # Leucemia

    # Filtrar el DataFrame para encontrar las filas que contienen los códigos de cáncer hematológico
    filtro_codigos = dataframe[columnas_codigos].apply(lambda x: x.str.contains(patron_leucemia, na=False)).any(axis=1)

    # Patrones para identificar diagnósticos de cáncer hematológico en texto libre
    patron_leucemia_diagnostico = re.compile(r'\bleucemia', re.IGNORECASE)
    #patron_cancer_hematologico_general = re.compile(r'(\bcancer|\bhema?toncologico)', re.IGNORECASE)
    patron_cancer_sangre = re.compile(r'\bcancer.*sangre', re.IGNORECASE)
    patron_cancer_sang = re.compile(r'\bcanc.*sang', re.IGNORECASE)
    patron_cancer_med = re.compile(r'\bcanc.*med', re.IGNORECASE)
    # Patrones para identificar neoplasias hematológicas
    patron_neoplasia_sangre = re.compile(r'\bneoplasia.*sangre', re.IGNORECASE)
    patron_neoplasia_medula = re.compile(r'\bneoplasia.*medula', re.IGNORECASE)


    # Asegurarse de que las columnas de diagnóstico sean de tipo string
    dataframe[columnas_diagnosticos] = dataframe[columnas_diagnosticos].astype(str)
    dataframe[columnas_codigos] = dataframe[columnas_codigos].astype(str)
    dataframe[columnas_diagnosticos] = dataframe[columnas_diagnosticos].replace('nan', np.nan)
    dataframe[columnas_codigos] = dataframe[columnas_codigos].replace('nan', np.nan)
    # Inicializar la columna 'PROB_SALUD' con valores vacíos
    dataframe['PROB_SALUD'] = ''

    # Definir una función para buscar en las columnas en orden
    
    
    # Aplicar la función de búsqueda de diagnósticos
    patrones = [(patron_leucemia, 'Leucemia', None)]
    dataframe['PROB_SALUD'] = dataframe.apply(buscar_diagnostico_cod, axis=1, patrones=patrones)
    
    
    # Aplicar la función de búsqueda de diagnósticos
    patrones = [(patron_leucemia_diagnostico, 'Leucemia', None), 
                (patron_cancer_sangre, 'Leucemia', None),
                (patron_cancer_sang, 'Leucemia', None),
                (patron_cancer_med, 'Leucemia', None),
                (patron_neoplasia_sangre, 'Leucemia', None),
                (patron_neoplasia_medula, 'Leucemia', None)]
    dataframe['PROB_SALUD'] = dataframe.apply(buscar_diagnostico, axis=1, patrones=patrones)



    # Filtrar el DataFrame para encontrar las filas que contienen el diagnóstico de cáncer hematológico en texto libre
    filtro_diagnosticos = dataframe[columnas_diagnosticos].apply(lambda x:
    x.str.contains(patron_leucemia_diagnostico, na=False) |
    #x.str.contains(patron_cancer_hematologico_general, na=False) |
    x.str.contains(patron_cancer_sangre, na=False) |
    x.str.contains(patron_neoplasia_sangre, na=False) |
    x.str.contains(patron_neoplasia_medula, na=False) |
    x.str.contains(patron_cancer_sang, na=False) |
    x.str.contains(patron_cancer_med, na=False)).any(axis=1)

    # Filtrar por la comuna La Florida
    filtro_comuna = dataframe['COMUNA'] == 'La Florida'

    # Combinar los filtros para obtener las filas que cumplen con al menos uno de los criterios
    resultado_filtrado = dataframe[(filtro_codigos | filtro_diagnosticos)].copy()
    return resultado_filtrado

def filtro_dataframe_ca_linfoma_inter(dataframe, columnas_codigos, columnas_diagnosticos):
    # Definir las columnas de códigos y de diagnósticos en texto libre
    columnas_codigos = ['SOSP_DIAG_COD_1', 'SOSP_DIAG_COD_2', 'SOSP_DIAG_COD_3', 'SOSP_DIAG_COD_4', 'SOSP_DIAG_COD_5']
    columnas_diagnosticos = ['SOSP_DIAG_DESC_1', 'SOSP_DIAG_DESC_2', 'SOSP_DIAG_DESC_3', 'SOSP_DIAG_DESC_4', 'SOSP_DIAG_DESC_5']

    # Códigos CIE-10 para cánceres hematológicos
    patron_linfoma = r'^C81.*|C82.*|C83.*|C84.*|C85.*'     # Linfoma


    # Filtrar el DataFrame para encontrar las filas que contienen los códigos de cáncer hematológico
    filtro_codigos = dataframe[columnas_codigos].apply(lambda x: x.str.contains(patron_linfoma, na=False)).any(axis=1)

    # Patrones para identificar diagnósticos de cáncer hematológico en texto libre
    patron_linfoma_diagnostico = re.compile(r'\blinfoma', re.IGNORECASE)
    #patron_cancer_hematologico_general = re.compile(r'(\bcancer|\bhema?toncologico)', re.IGNORECASE)
    patron_cancer_linfatico = re.compile(r'\bcancer.*linfatico', re.IGNORECASE)
    patron_cancer_linf = re.compile(r'\bcanc.*linf', re.IGNORECASE)
    patron_neoplasia_linfatico = re.compile(r'\bneoplasia.*linfatico', re.IGNORECASE)

     # Asegurarse de que las columnas de diagnóstico sean de tipo string
    dataframe[columnas_diagnosticos] = dataframe[columnas_diagnosticos].astype(str)
    dataframe[columnas_codigos] = dataframe[columnas_codigos].astype(str)
    dataframe[columnas_diagnosticos] = dataframe[columnas_diagnosticos].replace('nan', np.nan)
    dataframe[columnas_codigos] = dataframe[columnas_codigos].replace('nan', np.nan)
    # Inicializar la columna 'PROB_SALUD' con valores vacíos
    dataframe['PROB_SALUD'] = ''

    # Definir una función para buscar en las columnas en orden
    
    
    # Aplicar la función de búsqueda de diagnósticos
    patrones = [(patron_linfoma, 'Linfoma', None)]
    dataframe['PROB_SALUD'] = dataframe.apply(buscar_diagnostico_cod, axis=1, patrones=patrones)
    
    
    # Aplicar la función de búsqueda de diagnósticos
    patrones = [(patron_linfoma_diagnostico, 'Linfoma', None), 
                (patron_cancer_linfatico, 'Linfoma', None),
                (patron_cancer_linf, 'Linfoma', None),
                (patron_neoplasia_linfatico, 'Linfoma', None)]
    dataframe['PROB_SALUD'] = dataframe.apply(buscar_diagnostico, axis=1, patrones=patrones)


    # Filtrar el DataFrame para encontrar las filas que contienen el diagnóstico de cáncer hematológico en texto libre
    filtro_diagnosticos = dataframe[columnas_diagnosticos].apply(lambda x:
    x.str.contains(patron_linfoma_diagnostico, na=False) |
    x.str.contains(patron_cancer_linfatico, na=False) |
    x.str.contains(patron_neoplasia_linfatico, na=False) |

    x.str.contains(patron_cancer_linf, na=False)).any(axis=1)

    # Filtrar por la comuna La Florida
    filtro_comuna = dataframe['COMUNA'] == 'La Florida'

    # Combinar los filtros para obtener las filas que cumplen con al menos uno de los criterios
    resultado_filtrado = dataframe[(filtro_codigos | filtro_diagnosticos)].copy()
    return resultado_filtrado

def filtro_dataframe_ca_mieloma_inter(dataframe, columnas_codigos, columnas_diagnosticos):
    # Definir las columnas de códigos y de diagnósticos en texto libre
    columnas_codigos = ['SOSP_DIAG_COD_1', 'SOSP_DIAG_COD_2', 'SOSP_DIAG_COD_3', 'SOSP_DIAG_COD_4', 'SOSP_DIAG_COD_5']
    columnas_diagnosticos = ['SOSP_DIAG_DESC_1', 'SOSP_DIAG_DESC_2', 'SOSP_DIAG_DESC_3', 'SOSP_DIAG_DESC_4', 'SOSP_DIAG_DESC_5']

    # Códigos CIE-10 para cánceres hematológicos
    patron_mieloma = r'^C90.*'                             # Mieloma múltiple

    # Filtrar el DataFrame para encontrar las filas que contienen los códigos de cáncer hematológico
    filtro_codigos = dataframe[columnas_codigos].apply(lambda x: x.str.contains(patron_mieloma, na=False)).any(axis=1)

    # Patrones para identificar diagnósticos de cáncer hematológico en texto libre
    patron_mieloma_diagnostico = re.compile(r'\bmieloma', re.IGNORECASE)
    patron_cancer_plasma = re.compile(r'\bcancer.*plasma', re.IGNORECASE)
    patron_cancer_plasm = re.compile(r'\bcanc.*plasm', re.IGNORECASE)
    # Patrones para identificar neoplasias hematológicas
    patron_neoplasia_plasma = re.compile(r'\bneoplasia.*plasma', re.IGNORECASE)


    # Asegurarse de que las columnas de diagnóstico sean de tipo string
    dataframe[columnas_diagnosticos] = dataframe[columnas_diagnosticos].astype(str)
    dataframe[columnas_codigos] = dataframe[columnas_codigos].astype(str)
    dataframe[columnas_diagnosticos] = dataframe[columnas_diagnosticos].replace('nan', np.nan)
    dataframe[columnas_codigos] = dataframe[columnas_codigos].replace('nan', np.nan)
    # Inicializar la columna 'PROB_SALUD' con valores vacíos
    dataframe['PROB_SALUD'] = ''

    # Definir una función para buscar en las columnas en orden
    
    
    # Aplicar la función de búsqueda de diagnósticos
    patrones = [(patron_mieloma, 'Mieloma', None)]
    dataframe['PROB_SALUD'] = dataframe.apply(buscar_diagnostico_cod, axis=1, patrones=patrones)
    
    
    # Aplicar la función de búsqueda de diagnósticos
    patrones = [(patron_mieloma_diagnostico, 'Mieloma', None), 
                (patron_cancer_plasma, 'Mieloma', None),
                (patron_cancer_plasm, 'Mieloma', None),
                (patron_neoplasia_plasma, 'Mieloma', None)]
    dataframe['PROB_SALUD'] = dataframe.apply(buscar_diagnostico, axis=1, patrones=patrones)



    # Filtrar el DataFrame para encontrar las filas que contienen el diagnóstico de cáncer hematológico en texto libre
    filtro_diagnosticos = dataframe[columnas_diagnosticos].apply(lambda x:
    x.str.contains(patron_mieloma_diagnostico, na=False) |
    #x.str.contains(patron_cancer_hematologico_general, na=False) |
    x.str.contains(patron_cancer_plasma, na=False) |
    x.str.contains(patron_neoplasia_plasma, na=False) |
    x.str.contains(patron_cancer_plasm, na=False)).any(axis=1)

    # Filtrar por la comuna La Florida
    filtro_comuna = dataframe['COMUNA'] == 'La Florida'

    # Combinar los filtros para obtener las filas que cumplen con al menos uno de los criterios
    resultado_filtrado = dataframe[(filtro_codigos | filtro_diagnosticos)].copy()
    return resultado_filtrado

def filtro_dataframe_ca_vesical_inter(dataframe, columnas_codigos, columnas_diagnosticos):
    # Definir las columnas de códigos y de diagnósticos en texto libre
    columnas_codigos = ['SOSP_DIAG_COD_1', 'SOSP_DIAG_COD_2', 'SOSP_DIAG_COD_3', 'SOSP_DIAG_COD_4', 'SOSP_DIAG_COD_5']
    columnas_diagnosticos = ['SOSP_DIAG_DESC_1', 'SOSP_DIAG_DESC_2', 'SOSP_DIAG_DESC_3', 'SOSP_DIAG_DESC_4', 'SOSP_DIAG_DESC_5']

    # Patrón regex para identificar los códigos CIE-10 asociados al cáncer vesical
    patron_cancer_vesical = r'^C67.*'  # Códigos CIE-10 para cáncer de vejiga

    # Filtrar el DataFrame para encontrar las filas que contienen el código de cáncer vesical
    filtro_codigos = dataframe[columnas_codigos].apply(lambda x: x.str.contains(patron_cancer_vesical, na=False)).any(axis=1)

    # Patrones para identificar diagnósticos de cáncer vesical en texto libre
    patron_diagnostico_vesical = re.compile(r'\bc[aá]*[ \.]*ves[ií]cal*\b', re.IGNORECASE)
    patron_carcinoma_vesical = re.compile(r'\bcarcinoma.*ves[ií]cal[a-zá-ú]*', re.IGNORECASE)
    patron_tumor_maligno_vesical = re.compile(r'\btumor.*maligno.*vejiga|\btumor.*maligno.*ves[ií]cal*', re.IGNORECASE)
    patron_carcinoma_transicional_vesical = re.compile(r'\bcarcinoma.*transicional.*vejiga|\bcarcinoma.*transicional.*ves[ií]cal[a-zá-ú]*', re.IGNORECASE)
    patron_carcinoma_urotelial_vesical = re.compile(r'\bcarcinoma.*urotelial.*vejiga|\bcarcinoma.*urotelial.*ves[ií]cal[a-zá-ú]*', re.IGNORECASE)
    patron_carcinoma_escamoso_vesical = re.compile(r'c\barcinoma.*escamoso.*vejiga|\bcarcinoma.*escamoso.*ves[ií]cal[a-zá-ú]*', re.IGNORECASE)
    patron_adenocarcinoma_vesical = re.compile(r'\badenocarcinoma.*vejiga|\badenocarcinoma.*ves[ií]cal*', re.IGNORECASE)

    # Asegurarse de que las columnas de diagnóstico sean de tipo string
    dataframe[columnas_diagnosticos] = dataframe[columnas_diagnosticos].astype(str)
    dataframe[columnas_codigos] = dataframe[columnas_codigos].astype(str)
    dataframe[columnas_diagnosticos] = dataframe[columnas_diagnosticos].replace('nan', np.nan)
    dataframe[columnas_codigos] = dataframe[columnas_codigos].replace('nan', np.nan)
    # Inicializar la columna 'PROB_SALUD' con valores vacíos
    dataframe['PROB_SALUD'] = ''

    # Definir una función para buscar en las columnas en orden
    
    
    # Aplicar la función de búsqueda de diagnósticos
    patrones = [(patron_cancer_vesical, 'Cáncer Vesical', None)]
    dataframe['PROB_SALUD'] = dataframe.apply(buscar_diagnostico_cod, axis=1, patrones=patrones)
    
    
    # Aplicar la función de búsqueda de diagnósticos
    patrones = [(patron_diagnostico_vesical, 'Cáncer Vesical', None), 
                (patron_carcinoma_vesical, 'Cáncer Vesical', None),
                (patron_tumor_maligno_vesical, 'Cáncer Vesical', None),
                (patron_carcinoma_transicional_vesical, 'Cáncer Vesical', None),
                (patron_carcinoma_urotelial_vesical, 'Cáncer Vesical', None),
                (patron_carcinoma_escamoso_vesical, 'Cáncer Vesical', None),
                (patron_adenocarcinoma_vesical, 'Cáncer Vesical', None)]
    dataframe['PROB_SALUD'] = dataframe.apply(buscar_diagnostico, axis=1, patrones=patrones)



    # Filtrar el DataFrame para encontrar las filas que contienen el diagnóstico de cáncer vesical en texto libre
    filtro_diagnosticos = dataframe[columnas_diagnosticos].apply(lambda x:
        x.str.contains(patron_diagnostico_vesical, na=False) |
        x.str.contains(patron_carcinoma_vesical, na=False) |
        x.str.contains(patron_tumor_maligno_vesical, na=False) |
        x.str.contains(patron_carcinoma_transicional_vesical, na=False) |
        x.str.contains(patron_carcinoma_urotelial_vesical, na=False) |
        x.str.contains(patron_carcinoma_escamoso_vesical, na=False) |
        x.str.contains(patron_adenocarcinoma_vesical, na=False)).any(axis=1)

    # Filtrar por la comuna La Florida

    filtro_comuna = dataframe['COMUNA'] == 'La Florida'

    # Combinar los filtros para obtener las filas que cumplen con al menos uno de los criterios
    resultado_filtrado = dataframe[(filtro_codigos | filtro_diagnosticos)].copy()
    return resultado_filtrado

def crear_dataframe_inter(resultado_filtrado):

    def concatenar_diagnosticos(row):
        diagnosticos = []
        for col in columnas_diagnosticos:
            valor = row[col]
            if isinstance(valor, str) and valor.strip():
                diagnosticos.append(valor)
        return ','.join(diagnosticos)
    nuevo_dataframe = pd.DataFrame({
        'Problema Oncologico': resultado_filtrado['PROB_SALUD'],
        'RUT': resultado_filtrado['RUT'],
        'DV': resultado_filtrado['DV'],
        'Nombre del Paciente':resultado_filtrado['NOM_PAC'] + ' ' + resultado_filtrado['APELLPAT_PAC']+' '+resultado_filtrado['APELLMAT_PAC'],
        'N° Telefono 1': resultado_filtrado['FONO_MOVIL'],
        'N° Telefono 2': resultado_filtrado['FONO_FIJO_1'],
        'N° Telefono 3': resultado_filtrado['FONO_FIJO_2'],
        'Fecha de SIC': resultado_filtrado['FECHA_SOLIC'],
        'Establecimiento de Origen': resultado_filtrado['ESTAB_ORIGEN_DESC'],
        'Especialidad Destino': resultado_filtrado['ESPECIALIDAD_DESTINO'],
        'Estado de SIC': resultado_filtrado['ESTADO_SOL'],
        'Fecha Proximo Hito': np.nan,
        #'Observaciones del Diagnostico': np.nan,
        #'Diagnostico CIE-10': np.nan,
        'DIAGNOSTICOS': resultado_filtrado.apply(concatenar_diagnosticos, axis=1),
        'Fecha Proximo Hito': np.nan,
        'Nombre agenda Medico especialista': np.nan,
    })

    return nuevo_dataframe
def crear_dataframe_ges(dataframe_tiroides):
    # Crear un nuevo DataFrame con las columnas deseadas
    nuevo_dataframe_2 = pd.DataFrame({
        'Problema Oncologico': dataframe_tiroides['Problema de salud'],
        'RUT': dataframe_tiroides['RUT'],
        'DV': dataframe_tiroides['DV'],
        'Nombre del Paciente':dataframe_tiroides['Nombre'],
        'N° Telefono 1': dataframe_tiroides['Telefonos.x'],
        'N° Telefono 2': dataframe_tiroides['Telefonos.y'],
        'Fecha de SIC': dataframe_tiroides['Fecha inicio'],
        'Establecimiento de Origen': np.nan,
        'Especialidad Destino': np.nan,
        'Estado de SIC': np.nan,
        'N° Telefono 3': np.nan,
        'Nombre garantia': dataframe_tiroides['Nombre garantia'],
        'Fecha Proximo Hito': np.nan,
        'DIAGNOSTICOS': np.nan,
        'CODIGOS_DIAGNOSTICO': np.nan,
        'Fecha Proximo Hito': np.nan,
        'Nombre agenda Medico especialista': np.nan,
    })
    return nuevo_dataframe_2



# Ejemplo de uso
# df = pd.read_csv('tu_archivo.csv')
# df_filtrado = filtro_dataframe_ca_colorectal(df, columnas_codigos, columnas_diagnosticos)
# print(df_filtrado)


# Ruta SIC Local

In [7]:
#dir_actual = os.getcwd()
# Construir la ruta relativa al directorio donde se encuentra el archivo CSV
#dir_archivo_csv = os.path.join(dir_actual, '..','..')
#dir_datos_hlf = os.path.join(dir_archivo_csv, 'Datos HLF')
#rutas_archivos_4 = os.path.join(dir_datos_hlf, '20240603_Informe_Solicitudes_DerivacionCE.csv')

# Ruta SIC HLF

In [8]:
import win32com.client

dir_actual = os.getcwd()
dir_archivo_csv = os.path.join(dir_actual,'..','Reporteria.lnk')

def get_target_path(lnk_file):
    shell = win32com.client.Dispatch("WScript.Shell")
    shortcut = shell.CreateShortCut(lnk_file)
    return shortcut.Targetpath

# Ruta del archivo de acceso directo (.lnk)
ruta_del_lnk = os.path.join(dir_actual,'..','Reporteria.lnk')
# Obtener la ruta de destino del archivo de acceso directo
ruta_reporteria = get_target_path(ruta_del_lnk)

# Ruta del archivo de acceso directo (.lnk)
ruta_del_lnk = os.path.join(dir_actual,'..','Unidad_Gestion_Del_Paciente.lnk')
# Obtener la ruta de destino del archivo de acceso directo
ruta_ges = get_target_path(ruta_del_lnk)

In [9]:
#Reportes SIC

patron_sic = os.path.join(ruta_reporteria,'Reportes_Solicitudes','Mensual', '*Informe_Solicitudes_DerivacionCE*')

archivos_sic  = glob.glob(patron_sic)

archivos_ordenados_sic  = sorted(archivos_sic , key=os.path.getmtime, reverse = True)

rutas_archivos_4 = archivos_ordenados_sic[:1]

rutas_archivos_4 = rutas_archivos_4[0]

# DEBUG


In [10]:
dataframe = cargar_dataframe_ambulatorio(rutas_archivos_4)

In [11]:
dataframe['ESPECIALIDAD_DESTINO'] = dataframe['ESPECIALIDAD_DESTINO'].astype(str)

In [12]:
list(dataframe['ESPECIALIDAD_DESTINO'].unique())

['Pediatría',
 'Cardiología Infantil',
 'Broncopulmonar Infantil',
 'Nefrología Infantil',
 'Cirugía Vascular Periférica',
 'Alto Riesgo Obstétrico',
 'Ginecología Adulto',
 'Hospitalización Domiciliaria MQ',
 'Cirugía Adulto',
 'Traumatología Adulto',
 'Neurología Adulto',
 'Cardiología Adulto',
 'Neurología Infantil',
 'Endocrinología Adulto',
 'Dermatología Adulto',
 'Psiquiatría Adulto',
 'Ortodoncia',
 'Cirugía Proctológica',
 'Otorrinolaringología',
 'Oftalmología',
 'Infectología Adulto',
 'Hospital de Día',
 'Hematología Adulto',
 'Inmunología Adulto',
 'Unidad de Patología Cervical',
 'TACO',
 'Medicina Interna',
 'Urología Adulto',
 'Broncopulmonar Adulto',
 'Med. Física y Rehab. Adultos',
 'Cirugía de mamas',
 'Traumatología Infantil',
 'Gastroenterología Adulto',
 'Psiquiatría Infantil',
 'Kinesiología MFR',
 'Med.Física y Rehabilitación Adulto',
 'Reumatología',
 'Cirugía Infantil',
 'Endodoncia',
 'Cirugía Abdominal Adulto',
 'Operatoria',
 'Endocrinología Infantil',
 'Re

# Filtro especialidad SIC

In [13]:
regex_esp = re.compile(r'(?i)\bbroncopulmonar|cirug[íi]a.*(abdominal|adulto|cabeza.*cuello|t[óo]rax|digestiva|general|proctol[óo]gica)|dermatolog[íi]a.*adulto|endocrinolog[íi]a.*adulto|gastroenterolog[íi]a.*adulto|hematolog[íi]a.*adulto|medicina.*interna|traumatolog[íi]a|urolog[íi]a.*adulto', re.IGNORECASE)


dataframe = cargar_dataframe_ambulatorio(rutas_archivos_4)
dataframe['NRO_IDENTIFICADOR'] = dataframe['NRO_IDENTIFICADOR'].astype(str)
dataframe['NRO_IDENTIFICADOR'] = dataframe['NRO_IDENTIFICADOR'].str.replace('.','', regex = False)
dataframe['NRO_IDENTIFICADOR'] = dataframe['NRO_IDENTIFICADOR'].str.replace('-','', regex = False)
dataframe = dataframe[dataframe['ESPECIALIDAD_DESTINO'].str.contains(regex_esp, na=False)]
dataframe = dataframe[dataframe['ESTADO_SOL'].isin(['Solicitada','Pendiente de Autorizar', 'Citada', 'Finalizada', 'Rechazada'])]
dataframe['FECHA_SOLIC'] = pd.to_datetime(dataframe['FECHA_SOLIC'], format='%d/%m/%Y')
    
dataframe = gestion_fecha(dataframe)

    # Definir la fecha de inicio para el filtro


In [14]:
dataframe['ESPECIALIDAD_DESTINO'].value_counts()

ESPECIALIDAD_DESTINO
Medicina Interna            26
Cirugía Adulto              25
Urología Adulto             21
Traumatología Adulto        18
Neurología Adulto           16
Broncopulmonar Adulto       13
Gastroenterología Adulto    12
Traumatología Infantil      10
Dermatología Adulto          9
Cirugía Abdominal Adulto     8
Endocrinología Adulto        8
Hematología Adulto           7
Cirugia Cabeza y Cuello      6
Broncopulmonar Infantil      4
Cirugía Tórax Adulto         3
Cirugía Proctológica         3
Broncopulmonar               1
Name: count, dtype: int64

In [15]:
resultado_filtrado = filtro_dataframe_ca_colorectal(dataframe, columnas_codigos, columnas_diagnosticos)

In [16]:
resultado_filtrado[['SOSP_DIAG_DESC_2', 'SOSP_DIAG_DESC_1', 'SOSP_DIAG_DESC_3','PROB_SALUD']]

,SOSP_DIAG_DESC_2,SOSP_DIAG_DESC_1,SOSP_DIAG_DESC_3,PROB_SALUD
566200,Tumor maligno del colon,CANCER DE COLON,NaN,Cáncer Colorectal
585759,SINDROME DE RESECCION ANTERIOR,Tumor maligno del recto,TU DE RECTO,Cáncer Colorectal
585771,Tumor maligno del recto,CANCER DE RECTO QUE TERMINO QT RT PREOP,NaN,Cáncer Colorectal
585773,SINDROME DE RESECCION ANTERIOR,Tumor maligno del recto,TU DE RECTO,Cáncer Colorectal


In [17]:
resultado_filtrado[['SOSP_DIAG_COD_2', 'SOSP_DIAG_COD_1', 'SOSP_DIAG_COD_3','PROB_SALUD']]

,SOSP_DIAG_COD_2,SOSP_DIAG_COD_1,SOSP_DIAG_COD_3,PROB_SALUD
566200,C18,NaN,NaN,Cáncer Colorectal
585759,NaN,C20,NaN,Cáncer Colorectal
585771,C20,NaN,NaN,Cáncer Colorectal
585773,NaN,C20,NaN,Cáncer Colorectal


In [18]:
resultado_filtrado_2 = filtro_dataframe_ca_gastrico_inter(dataframe, columnas_codigos, columnas_diagnosticos)

In [19]:
resultado_filtrado_2[['SOSP_DIAG_DESC_2', 'SOSP_DIAG_DESC_1', 'SOSP_DIAG_DESC_3','PROB_SALUD']]

,SOSP_DIAG_DESC_2,SOSP_DIAG_DESC_1,SOSP_DIAG_DESC_3,PROB_SALUD


In [20]:
resultado_filtrado_3 = filtro_dataframe_ca_tiroides_inter(dataframe, columnas_codigos, columnas_diagnosticos)

In [21]:
resultado_filtrado_3[['SOSP_DIAG_DESC_2', 'SOSP_DIAG_DESC_1', 'SOSP_DIAG_DESC_3','PROB_SALUD']]

,SOSP_DIAG_DESC_2,SOSP_DIAG_DESC_1,SOSP_DIAG_DESC_3,PROB_SALUD


In [22]:
resultado_filtrado_4 = filtro_dataframe_ca_pulmon_inter(dataframe, columnas_codigos, columnas_diagnosticos)

In [23]:
resultado_filtrado_4[['SOSP_DIAG_DESC_2', 'SOSP_DIAG_DESC_1', 'SOSP_DIAG_DESC_3','PROB_SALUD']]

,SOSP_DIAG_DESC_2,SOSP_DIAG_DESC_1,SOSP_DIAG_DESC_3,PROB_SALUD
608162,NaN,Tumor maligno de los bronquios y del pulmon,NaN,Cáncer Pulmón


In [24]:
resultado_filtrado_4[['SOSP_DIAG_COD_2', 'SOSP_DIAG_COD_1', 'SOSP_DIAG_COD_3','PROB_SALUD']]

,SOSP_DIAG_COD_2,SOSP_DIAG_COD_1,SOSP_DIAG_COD_3,PROB_SALUD
608162,NaN,C34,NaN,Cáncer Pulmón


In [25]:
resultado_filtrado_5 = filtro_dataframe_ca_prostata_inter(dataframe, columnas_codigos, columnas_diagnosticos)

In [26]:
resultado_filtrado_5[['SOSP_DIAG_DESC_2', 'SOSP_DIAG_DESC_1', 'SOSP_DIAG_DESC_3','PROB_SALUD']]

,SOSP_DIAG_DESC_2,SOSP_DIAG_DESC_1,SOSP_DIAG_DESC_3,PROB_SALUD
608236,NaN,Tumor maligno de la prostata,NaN,Cáncer Próstata


In [27]:
resultado_filtrado_5[['SOSP_DIAG_COD_2', 'SOSP_DIAG_COD_1', 'SOSP_DIAG_COD_3','PROB_SALUD']]

,SOSP_DIAG_COD_2,SOSP_DIAG_COD_1,SOSP_DIAG_COD_3,PROB_SALUD
608236,NaN,C61,NaN,Cáncer Próstata


In [28]:
resultado_filtrado_6 = filtro_dataframe_ca_renal_inter(dataframe, columnas_codigos, columnas_diagnosticos)

In [29]:
resultado_filtrado_6[['SOSP_DIAG_DESC_2', 'SOSP_DIAG_DESC_1', 'SOSP_DIAG_DESC_3','PROB_SALUD']]

,SOSP_DIAG_DESC_2,SOSP_DIAG_DESC_1,SOSP_DIAG_DESC_3,PROB_SALUD


In [30]:
resultado_filtrado_6[['SOSP_DIAG_COD_2', 'SOSP_DIAG_COD_1', 'SOSP_DIAG_COD_3','PROB_SALUD']]

,SOSP_DIAG_COD_2,SOSP_DIAG_COD_1,SOSP_DIAG_COD_3,PROB_SALUD


In [31]:
resultado_filtrado_7 = filtro_dataframe_ca_testiculo_inter(dataframe, columnas_codigos, columnas_diagnosticos)

In [32]:
resultado_filtrado_7[['SOSP_DIAG_DESC_2', 'SOSP_DIAG_DESC_1', 'SOSP_DIAG_DESC_3','PROB_SALUD']]

,SOSP_DIAG_DESC_2,SOSP_DIAG_DESC_1,SOSP_DIAG_DESC_3,PROB_SALUD


In [33]:
resultado_filtrado_7[['SOSP_DIAG_COD_2', 'SOSP_DIAG_COD_1', 'SOSP_DIAG_COD_3','PROB_SALUD']]

,SOSP_DIAG_COD_2,SOSP_DIAG_COD_1,SOSP_DIAG_COD_3,PROB_SALUD


In [34]:
resultado_filtrado_8 = filtro_dataframe_ca_leucemia_inter(dataframe, columnas_codigos, columnas_diagnosticos)

In [35]:
resultado_filtrado_8[['SOSP_DIAG_DESC_2', 'SOSP_DIAG_DESC_1', 'SOSP_DIAG_DESC_3','PROB_SALUD']]

,SOSP_DIAG_DESC_2,SOSP_DIAG_DESC_1,SOSP_DIAG_DESC_3,PROB_SALUD


In [36]:
resultado_filtrado_8[['SOSP_DIAG_COD_2', 'SOSP_DIAG_COD_1', 'SOSP_DIAG_COD_3','PROB_SALUD']]

,SOSP_DIAG_COD_2,SOSP_DIAG_COD_1,SOSP_DIAG_COD_3,PROB_SALUD


In [37]:
resultado_filtrado_9 = filtro_dataframe_ca_linfoma_inter(dataframe, columnas_codigos, columnas_diagnosticos)

In [38]:
resultado_filtrado_9[['SOSP_DIAG_DESC_2', 'SOSP_DIAG_DESC_1', 'SOSP_DIAG_DESC_3','PROB_SALUD']]

,SOSP_DIAG_DESC_2,SOSP_DIAG_DESC_1,SOSP_DIAG_DESC_3,PROB_SALUD


In [39]:
resultado_filtrado_9[['SOSP_DIAG_COD_2', 'SOSP_DIAG_COD_1', 'SOSP_DIAG_COD_3','PROB_SALUD']]

,SOSP_DIAG_COD_2,SOSP_DIAG_COD_1,SOSP_DIAG_COD_3,PROB_SALUD


In [40]:
resultado_filtrado_10 = filtro_dataframe_ca_mieloma_inter(dataframe, columnas_codigos, columnas_diagnosticos)

In [41]:
resultado_filtrado_10[['SOSP_DIAG_DESC_2', 'SOSP_DIAG_DESC_1', 'SOSP_DIAG_DESC_3','PROB_SALUD']]

,SOSP_DIAG_DESC_2,SOSP_DIAG_DESC_1,SOSP_DIAG_DESC_3,PROB_SALUD
608155,Dermatitis alergica de contacto,Mieloma multiple,NaN,


In [42]:
resultado_filtrado_10[['SOSP_DIAG_COD_2', 'SOSP_DIAG_COD_1', 'SOSP_DIAG_COD_3','PROB_SALUD']]

,SOSP_DIAG_COD_2,SOSP_DIAG_COD_1,SOSP_DIAG_COD_3,PROB_SALUD
608155,L23,C900,NaN,


In [43]:
resultado_filtrado_11 = filtro_dataframe_ca_vesical_inter(dataframe, columnas_codigos, columnas_diagnosticos)

In [44]:
resultado_filtrado_11[['SOSP_DIAG_DESC_2', 'SOSP_DIAG_DESC_1', 'SOSP_DIAG_DESC_3','PROB_SALUD']]

,SOSP_DIAG_DESC_2,SOSP_DIAG_DESC_1,SOSP_DIAG_DESC_3,PROB_SALUD


In [45]:
resultado_filtrado_11[['SOSP_DIAG_COD_2', 'SOSP_DIAG_COD_1', 'SOSP_DIAG_COD_3','PROB_SALUD']]

,SOSP_DIAG_COD_2,SOSP_DIAG_COD_1,SOSP_DIAG_COD_3,PROB_SALUD


In [46]:
consolidado = [resultado_filtrado]


consolidado.append(resultado_filtrado_2)
consolidado.append(resultado_filtrado_3)
consolidado.append(resultado_filtrado_4)
consolidado.append(resultado_filtrado_5)
consolidado.append(resultado_filtrado_6)
consolidado.append(resultado_filtrado_7)
consolidado.append(resultado_filtrado_8)
consolidado.append(resultado_filtrado_9)
consolidado.append(resultado_filtrado_10)
consolidado.append(resultado_filtrado_11)
df_concatenated = pd.concat(consolidado)


In [47]:
list(df_concatenated.columns)

['PAC_ID',
 'TIPO_INDENTIF',
 'NRO_IDENTIFICADOR',
 'NOM_PAC',
 'APELLPAT_PAC',
 'APELLMAT_PAC',
 'FECHA_NAC_PAC',
 'EDAD_PAC',
 'SEXO',
 'NACIONALIDAD_PAC',
 'FONO_MOVIL',
 'FONO_FIJO_1',
 'FONO_FIJO_2',
 'DIRECCION_PAC',
 'COMUNA',
 'TIPO_PREVISION_EPISODIO',
 'TRAMO',
 'PRAIS',
 'ID_SOLICITUD',
 'FECHA_CREACION_SOL',
 'FECHA_SOLIC',
 'FECHA_DESEABLE_AT',
 'TIPO_SOLICITUD',
 'ESTAB_ORIGEN_COD',
 'ESTAB_ORIGEN_DESC',
 'ESTAB_ORIGEN_LINEA_ASIS',
 'ESPECIALIDAD_ORIGEN',
 'RUN_PROF_SOLICITA',
 'DV_PROF_SOLICITA',
 'PROF_SOLICITANTE',
 'PRIORIDAD_ORIGEN',
 'NRO_GES',
 'DESC_GES',
 'TIPO_DIAG_1',
 'SOSP_DIAG_COD_1',
 'SOSP_DIAG_DESC_1',
 'TIPO_DIAG_2',
 'SOSP_DIAG_COD_2',
 'SOSP_DIAG_DESC_2',
 'TIPO_DIAG_3',
 'SOSP_DIAG_COD_3',
 'SOSP_DIAG_DESC_3',
 'TIPO_DIAG_4',
 'SOSP_DIAG_COD_4',
 'SOSP_DIAG_DESC_4',
 'TIPO_DIAG_5',
 'SOSP_DIAG_COD_5',
 'SOSP_DIAG_DESC_5',
 'ESTAB_DESTINO_COD',
 'ESTAB_DESTINO_DESC',
 'ESTAB_DESTINO_LINEA_ASIS',
 'ESPECIALIDAD_DESTINO',
 'PRIORIDAD_DESTINO',
 'CODIGO_P

In [48]:
df_concatenated = df_concatenated[~(df_concatenated['PROB_SALUD']=='')]

In [49]:
df_concatenated

,PAC_ID,TIPO_INDENTIF,NRO_IDENTIFICADOR,NOM_PAC,APELLPAT_PAC,APELLMAT_PAC,FECHA_NAC_PAC,EDAD_PAC,SEXO,NACIONALIDAD_PAC,...,NOM_PROF_DESTINO_5,ESTADO_SOL,FECHA_AUTORIZ,RUNDV_AUTORIZ,NOM_AUTORIZ,CAUSAL_SALIDA_NRO,CAUSAL_SALIDA_DESC,COMENT_RECHAZO,FECHA_CAUSAL_SALIDA,PROB_SALUD
566200,AC00197389,RUN,51921682,JUAN RAMÓN,SUAZO,ARAVENA,13/05/1950,74,Hombre,Chile,...,NaN,Pendiente de Autorizar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cáncer Colorectal
585759,AC00326267,RUN,101684768,LUIS ALBERTO,ÁLVAREZ,PINO,02/06/1964,60,Hombre,Chile,...,NaN,Pendiente de Autorizar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cáncer Colorectal
585771,AC00436637,RUN,84028290,JUAN ERNESTO,AMAYA,FAURA,04/09/1957,66,Hombre,Chile,...,NaN,Pendiente de Autorizar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cáncer Colorectal
585773,AC00326267,RUN,101684768,LUIS ALBERTO,ÁLVAREZ,PINO,02/06/1964,60,Hombre,Chile,...,NaN,Pendiente de Autorizar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cáncer Colorectal
608162,AC00050633,RUN,78982853,HILDA MILAGRO,LORCA,LUCERO,31/01/1959,65,Mujer,Chile,...,NaN,Pendiente de Autorizar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cáncer Pulmón
608236,AC00015329,RUN,42806021,JORGE LUIS,ABARCA,GÁLVEZ,29/11/1940,83,Hombre,Chile,...,NaN,Pendiente de Autorizar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cáncer Próstata


In [50]:
df_concatenated['ESTADO_SOL'].value_counts()

ESTADO_SOL
Pendiente de Autorizar    6
Name: count, dtype: int64

In [51]:
list(df_concatenated.columns)

['PAC_ID',
 'TIPO_INDENTIF',
 'NRO_IDENTIFICADOR',
 'NOM_PAC',
 'APELLPAT_PAC',
 'APELLMAT_PAC',
 'FECHA_NAC_PAC',
 'EDAD_PAC',
 'SEXO',
 'NACIONALIDAD_PAC',
 'FONO_MOVIL',
 'FONO_FIJO_1',
 'FONO_FIJO_2',
 'DIRECCION_PAC',
 'COMUNA',
 'TIPO_PREVISION_EPISODIO',
 'TRAMO',
 'PRAIS',
 'ID_SOLICITUD',
 'FECHA_CREACION_SOL',
 'FECHA_SOLIC',
 'FECHA_DESEABLE_AT',
 'TIPO_SOLICITUD',
 'ESTAB_ORIGEN_COD',
 'ESTAB_ORIGEN_DESC',
 'ESTAB_ORIGEN_LINEA_ASIS',
 'ESPECIALIDAD_ORIGEN',
 'RUN_PROF_SOLICITA',
 'DV_PROF_SOLICITA',
 'PROF_SOLICITANTE',
 'PRIORIDAD_ORIGEN',
 'NRO_GES',
 'DESC_GES',
 'TIPO_DIAG_1',
 'SOSP_DIAG_COD_1',
 'SOSP_DIAG_DESC_1',
 'TIPO_DIAG_2',
 'SOSP_DIAG_COD_2',
 'SOSP_DIAG_DESC_2',
 'TIPO_DIAG_3',
 'SOSP_DIAG_COD_3',
 'SOSP_DIAG_DESC_3',
 'TIPO_DIAG_4',
 'SOSP_DIAG_COD_4',
 'SOSP_DIAG_DESC_4',
 'TIPO_DIAG_5',
 'SOSP_DIAG_COD_5',
 'SOSP_DIAG_DESC_5',
 'ESTAB_DESTINO_COD',
 'ESTAB_DESTINO_DESC',
 'ESTAB_DESTINO_LINEA_ASIS',
 'ESPECIALIDAD_DESTINO',
 'PRIORIDAD_DESTINO',
 'CODIGO_P

In [52]:
df_concatenated['DV'] = df_concatenated['NRO_IDENTIFICADOR'].str[-1]

    #resultado_filtrado.loc[condicion, 'NRO_IDENTIFICADOR_PACIENTE'] = resultado_filtrado.loc[condicion, 'NRO_IDENTIFICADOR_PACIENTE'].str[:-1]
df_concatenated['RUT'] = df_concatenated['NRO_IDENTIFICADOR'].str[:-1]


In [53]:
df_concatenated

,PAC_ID,TIPO_INDENTIF,NRO_IDENTIFICADOR,NOM_PAC,APELLPAT_PAC,APELLMAT_PAC,FECHA_NAC_PAC,EDAD_PAC,SEXO,NACIONALIDAD_PAC,...,FECHA_AUTORIZ,RUNDV_AUTORIZ,NOM_AUTORIZ,CAUSAL_SALIDA_NRO,CAUSAL_SALIDA_DESC,COMENT_RECHAZO,FECHA_CAUSAL_SALIDA,PROB_SALUD,DV,RUT
566200,AC00197389,RUN,51921682,JUAN RAMÓN,SUAZO,ARAVENA,13/05/1950,74,Hombre,Chile,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cáncer Colorectal,2,5192168
585759,AC00326267,RUN,101684768,LUIS ALBERTO,ÁLVAREZ,PINO,02/06/1964,60,Hombre,Chile,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cáncer Colorectal,8,10168476
585771,AC00436637,RUN,84028290,JUAN ERNESTO,AMAYA,FAURA,04/09/1957,66,Hombre,Chile,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cáncer Colorectal,0,8402829
585773,AC00326267,RUN,101684768,LUIS ALBERTO,ÁLVAREZ,PINO,02/06/1964,60,Hombre,Chile,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cáncer Colorectal,8,10168476
608162,AC00050633,RUN,78982853,HILDA MILAGRO,LORCA,LUCERO,31/01/1959,65,Mujer,Chile,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cáncer Pulmón,3,7898285
608236,AC00015329,RUN,42806021,JORGE LUIS,ABARCA,GÁLVEZ,29/11/1940,83,Hombre,Chile,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cáncer Próstata,1,4280602


In [54]:
df_inter = crear_dataframe_inter(df_concatenated)

In [55]:
df_inter[['RUT','DIAGNOSTICOS']]

,RUT,DIAGNOSTICOS
566200,5192168,"Tumor maligno del colon,CANCER DE COLON"
585759,10168476,"SINDROME DE RESECCION ANTERIOR,Tumor maligno d..."
585771,8402829,"Tumor maligno del recto,CANCER DE RECTO QUE TE..."
585773,10168476,"SINDROME DE RESECCION ANTERIOR,Tumor maligno d..."
608162,7898285,Tumor maligno de los bronquios y del pulmon
608236,4280602,Tumor maligno de la prostata


# Cruce GES HLF

In [56]:

#os.path.join(ruta_ges,'UNIDAD GES', 'SCRIPT_GES','Cruce GES','Cruce.xlsx')
ruta_del_archivo = os.path.join(ruta_ges,'UNIDAD GES', 'SCRIPT_GES','Cruce GES','Cruce.xlsx')


# Carga el archivo XLSX en un DataFrame
dataframe = pd.read_excel(ruta_del_archivo)

# Cruce GES Local

In [57]:
#ruta_del_archivo = os.path.join(dir_datos_hlf, 'V5_Cruce.xlsx')


# Carga el archivo XLSX en un DataFrame
#dataframe = pd.read_excel(ruta_del_archivo)

## Filtrar GES especialidad

In [58]:
#regex_ges = re.compile(r'\b(?:confirmaci[oó]n.*diagn[oó]stica(?! \(sin.*etapificacion\))|confirmaci[oó]n|atenci[óo]n.*de.*especialidad|diagnostico.*consulta.*especialista|evaluaci[oó]n.*por.*especialista|consulta.*especialista)\b'
regex_ges = re.compile(r'\bconfirmaci[oó]n diagn[oó]stica(?! y etapificaci[oó]n)|\bconfirmaci[oó]n\b(?! diagn[oó]stica y etapificaci[oó]n)|confirmaci[oó]n [-\w\s]+ diagn[oó]stica\b|atenci[óo]n.*especiali(sta|dad)|(consulta|evaluaci[óo]n|diagn[oó]stico).*especialista', re.IGNORECASE)

dataframe = dataframe[dataframe['Nombre garantia'].str.contains(regex_ges, na=False)]

In [59]:
dataframe

,Problema de salud,RUT,DV,Nombre,Fecha inicio,Fecha limite,Nombre garantia,Telefonos.x,DIRECCION_NUMERO,BLOCK_DEPTO,DIRECCION.THALAMUS,Telefonos.y
7,Catarátas Proceso de Diagnóstico {decreto nº 228},10005555,4,"FARÍAS MENESES, AURELIA DE LAS MERCE",2024-01-22,2024-07-22,Confirmación-Diagnóstica,- / - / 77504577 / / 52048581 / / -/52048581//.,CALLE-Psje Berilio-Los Industriales-10649,-,"Avenida Psje Berilio 10649,",#ND
8,Leucemia Adulto Leucemia Crónica {decreto nº 44},10042988,8,"ROSALES GUTIÉRREZ, MARGARITA",2024-04-05,2024-06-04,Confirmación Diagnóstica Leucemia Crónica,- / - / 0 / / 93997626 / 86979838 / -/.,PASAJE-laguna del maule-Santa Maria De La Estr...,-,#ND,#ND
9,Cáncer Gástrico . {decreto nº 228},10042997,7,"YÁÑEZ FUENTES, NANCY DEL ROSARIO",2024-06-23,2024-07-23,Consulta Especialista,- / - / 2937386 / 7814536 / 87592780 / - / -/8...,CALLE-Sotero De L Rio-Pad 19 Vmp-756,-13,"Avenida Sotero De L Rio 756,",#ND
11,Catarátas Proceso de Diagnóstico {decreto nº 228},10051904,6,"QUIJADA SÁNCHEZ, SYLVIA",2024-06-04,2024-12-02,Confirmación-Diagnóstica,#ND,#ND,#ND,#ND,#ND
12,Retinopatía Diabética . {decreto nº 228},10057389,K,"DONAIRE ORTIZ, RAIMUNDO ALEJANDRO",2024-03-16,2024-06-14,Confirmación Diagnóstica,- / - / 46528184 / 95718980 / 941366250 / 7674...,PASAJE-LAS ROSAS-La Higuera-3986,-,#ND,FIJO: 63560869 MOVIL: 76742162
...,...,...,...,...,...,...,...,...,...,...,...,...
5137,Vicios de Refracción Sospecha Vicios de Refrac...,9911750,8,"CANIO REYES, ANTONIO",2024-04-22,2024-10-21,Confirmación Diagnóstica,- / - / 94956250 / 0 / 54924512 / 37412934 / 2...,PASAJE-Aillaguriche-San Esteban-1397,-,"Pasaje Aillaguriche 1397,",#ND
5138,Retinopatía Diabética . {decreto nº 228},9935961,7,"NAVARRO PAREDES, MARÍA DE LAS MERCEDES",2024-04-15,2024-07-15,Confirmación Diagnóstica,- / - / 28661981 / 82695382 / 82695382 / 27043...,CALLE-VIÑA DEL MAR-Alberto Larraguibel-6466,-31 A,"Calle VIÑA DEL MAR 6466, DP 31A",#ND
5139,Retinopatía Diabética . {decreto nº 228},9968823,8,"ALEGRÍA HUERTA, MARÍA ALEJANDRINA DEL CARMEN",2024-04-29,2024-07-29,Confirmación Diagnóstica,- / - / 0 / / 78773579 / - / -/78773579//.,PASAJE-El Apero-El Rodeo-1466,-,"Pasaje El Apero 1466,",#ND
5140,Catarátas Proceso de Diagnóstico {decreto nº 228},9968849,1,"ARRIAGADA PINTO, MARCO AURELIO",2023-10-10,2024-04-08,Confirmación-Diagnóstica,- / - / 45100096 / / 45100096 / - / -/.,CALLE-Las Amapolas-Santa Raquel-9648,-,#ND,#ND


In [60]:
dataframe['Fecha inicio'] = pd.to_datetime(dataframe['Fecha inicio'], format='%Y-%m-%d')

In [61]:
dataframe['Fecha inicio'] 

7      2024-01-22
8      2024-04-05
9      2024-06-23
11     2024-06-04
12     2024-03-16
          ...    
5137   2024-04-22
5138   2024-04-15
5139   2024-04-29
5140   2023-10-10
5141   2024-06-27
Name: Fecha inicio, Length: 3547, dtype: datetime64[ns]

## Establecer fecha revisión GES

In [62]:
fecha_hoy = datetime.today()
fecha_inicio = fecha_hoy - timedelta(days=10)

fecha_inicio = pd.to_datetime(fecha_inicio, format='%Y-%m-%d')
    # Filtrar el DataFrame
dataframe = dataframe[dataframe['Fecha inicio'] >= fecha_inicio]

In [63]:
dataframe

,Problema de salud,RUT,DV,Nombre,Fecha inicio,Fecha limite,Nombre garantia,Telefonos.x,DIRECCION_NUMERO,BLOCK_DEPTO,DIRECCION.THALAMUS,Telefonos.y
15,Artritis Reumatoide . {decreto nº 44},10084004,9,"CIFUENTES CALDERÓN, ALEX ANGELO",2024-06-28,2024-09-26,Confirmación Diagnóstica,- / - / 2881532 / / - / 2881532 / -/.,NINGUNO---,-,#ND,#ND
56,Cáncer Gástrico . {decreto nº 228},10492252,K,"VILCHES PÉREZ, MARISA DE LAS MERCED",2024-06-28,2024-07-29,Consulta Especialista,- / - / 22828026 / 66931649 / 66931649 / 22828...,PASAJE-HUALLATIRE-Villa Ohiggins-921,-,"Pasaje HUALLATIRE 921,",#ND
116,Artritis Reumatoide . {decreto nº 44},11375655,1,"VÁSQUEZ ALFARO, XIMENA ROSA",2024-07-01,2024-09-30,Confirmación Diagnóstica,- / - / 62216476 / 67449878 / 67449878 / 32166...,PASAJE-El Galope-El Rodeo-1608,-,"Pasaje El Galope 1608,",#ND
130,Cáncer Colorectal . {decreto n° 4/2013},11667136,0,"FERNÁNDEZ SILVA, IVÁN BLADIMIR",2024-07-03,2024-08-19,Confirmación,- / - / 22620004 / 98769960 / - / 22620004 / 9...,PASAJE-EL CANELO-La Arboleda-9476,-,#ND,#ND
148,Cáncer Gástrico . {decreto nº 228},11904915,6,"ROJAS SÁEZ, FLORENTINO AVELINO",2024-06-27,2024-07-29,Consulta Especialista,#ND,#ND,#ND,#ND,#ND
...,...,...,...,...,...,...,...,...,...,...,...,...
5033,Catarátas Proceso de Diagnóstico {decreto nº 228},9151426,5,"SOTO ALEU, LUIS FELIPE",2024-06-25,2024-12-23,Confirmación-Diagnóstica,- / - / 88183653 / / 88183653 / 88183653 / -/.,CALLE-parina--9582,-,#ND,#ND
5046,Vicios de Refracción Sospecha Vicios de Refrac...,9240913,9,"ITURRIAGA HIDALGO, EUGENIA DEL CARMEN",2024-06-25,2024-12-23,Confirmación Diagnóstica,- / - / 0 / / 33720207 / 57332240 / -/3372020...,PASAJE-la guitarra-Concierto-10919,-,"Pasaje La Guitarra 10919,",#ND
5083,Cáncer de Mama Derecha {decreto nº 228},9479644,K,"RIQUELME PALOMERA, ELIZABETH ANGÉLICA",2024-06-28,2024-08-12,Confirmación Mama Derecha,- / - / 0 / / 57601234 / 59957182 / -/9957601...,AVENIDA-MANUEL MAGALLANES MOURE-Jardin Alto-8292,-,"Pasaje MANUEL MAGALLANES MOURE 8292,",#ND
5126,Hipertensión Arterial . {decreto nº 228},9820079,7,"MORA HERRERA, MARÍA EUGENIA",2024-06-25,2024-08-09,Atención Especialista,- / - / 33164537 / 78031220 / 93548740 / 72504...,PASAJE-isla guamblin-Andes Del Sur-01641,-,#ND,FIJO: 32273462 MOVIL: 32273462


In [64]:
# Construir la ruta completa al archivo CSV
dataframe['RUT']= dataframe['RUT'].apply(lambda x : str(x))
dataframe['RUT COMPLETO'] = dataframe['RUT'].astype(str)  + dataframe['DV'].astype(str)


In [65]:
dataframe['Fecha inicio'].min()

Timestamp('2024-06-25 00:00:00')

## Filtro por problema de salud

In [66]:
import pandas as pd

# Suponiendo que 'dataframe' es tu DataFrame original
# Utilizamos una expresión regular para buscar las palabras clave 'pulmon', 'colorecta', 'linfoma', y 'leucemia'
regex = r'(?i)\b(pulmón|colorectal|linfoma|leucemia|g[aá]strico)\b'

dataframe_tiroides = dataframe[dataframe['Problema de salud'].str.contains(regex, na=False)]


In [67]:
dataframe_tiroides['Problema de salud'].value_counts()

Problema de salud
Cáncer Colorectal . {decreto n° 4/2013}     11
Cáncer Gástrico . {decreto nº 228}           2
Cáncer de Pulmón . {Decreto Nro 22/2019}     2
Name: count, dtype: int64

In [68]:
dataframe_tiroides = dataframe_tiroides.sort_values(by='Fecha inicio', ascending=False)


# Eliminar duplicados basados en la columna "NRO_IDENTIFICADOR_PACIENTE"
dataframe_tiroides = dataframe_tiroides.drop_duplicates(subset=['RUT','Problema de salud'], keep='first')


In [69]:
nuevo_dataframe_2 = crear_dataframe_ges(dataframe_tiroides)

In [70]:
nuevo_dataframe_2

,Problema Oncologico,RUT,DV,Nombre del Paciente,N° Telefono 1,N° Telefono 2,Fecha de SIC,Establecimiento de Origen,Especialidad Destino,Estado de SIC,N° Telefono 3,Nombre garantia,Fecha Proximo Hito,DIAGNOSTICOS,CODIGOS_DIAGNOSTICO,Nombre agenda Medico especialista
130,Cáncer Colorectal . {decreto n° 4/2013},11667136,0,"FERNÁNDEZ SILVA, IVÁN BLADIMIR",- / - / 22620004 / 98769960 / - / 22620004 / 9...,#ND,2024-07-03,NaN,NaN,NaN,NaN,Confirmación,NaN,NaN,NaN,NaN
4707,Cáncer Colorectal . {decreto n° 4/2013},8130920,5,"SILVA HENRÍQUEZ, GINA DE LAS MERCEDES",- / - / 68262953 / 68262953 / 68262953 / 31898...,#ND,2024-07-03,NaN,NaN,NaN,NaN,Confirmación,NaN,NaN,NaN,NaN
2790,Cáncer Colorectal . {decreto n° 4/2013},5869303,0,"RIQUELME PUEBLA, GLORIA MARÍA BERNARD",- / - / 25028287 / 0 / 97158876 / 25028287 / -...,#ND,2024-07-02,NaN,NaN,NaN,NaN,Confirmación,NaN,NaN,NaN,NaN
3021,Cáncer Colorectal . {decreto n° 4/2013},6094724,4,"PIZARRO SERRAT, RICARDO ROLANDO",- / - / / 0 / 93865732 / - / -/9993865732/299...,#ND,2024-07-02,NaN,NaN,NaN,NaN,Confirmación,NaN,NaN,NaN,NaN
4989,Cáncer Colorectal . {decreto n° 4/2013},8957749,7,"LEIVA CEPEDA, RODRIGO EDUARDO",- / - / 0 / 0 / 75530673 / / -/75530673//.,FIJO: NO ESPECIFICADO MOVIL: 75530673,2024-07-02,NaN,NaN,NaN,NaN,Confirmación,NaN,NaN,NaN,NaN
56,Cáncer Gástrico . {decreto nº 228},10492252,K,"VILCHES PÉREZ, MARISA DE LAS MERCED",- / - / 22828026 / 66931649 / 66931649 / 22828...,#ND,2024-06-28,NaN,NaN,NaN,NaN,Consulta Especialista,NaN,NaN,NaN,NaN
4049,Cáncer Colorectal . {decreto n° 4/2013},7196918,5,"GAJARDO DÍAZ, JUANA NATALIA",- / - / 85552235 / 85552235 / 85552235 / 85552...,#ND,2024-06-28,NaN,NaN,NaN,NaN,Confirmación,NaN,NaN,NaN,NaN
4325,Cáncer de Pulmón . {Decreto Nro 22/2019},7540942,7,"VALDEBENITO LOBOS, MARÍA ANGÉLICA",- / - / 0 / / 94364482 / - / -/.,#ND,2024-06-28,NaN,NaN,NaN,NaN,Confirmación Diagnóstica,NaN,NaN,NaN,NaN
4784,Cáncer de Pulmón . {Decreto Nro 22/2019},8317900,7,"MELLI NAVARRETE, ESTER LUCRECIA",- / - / 35766740 / 35766740 / 35766740 / 23140...,FIJO: 23140537 MOVIL: 935766740,2024-06-28,NaN,NaN,NaN,NaN,Confirmación Diagnóstica,NaN,NaN,NaN,NaN
148,Cáncer Gástrico . {decreto nº 228},11904915,6,"ROJAS SÁEZ, FLORENTINO AVELINO",#ND,#ND,2024-06-27,NaN,NaN,NaN,NaN,Consulta Especialista,NaN,NaN,NaN,NaN


In [71]:
df_concatenated_2 = pd.concat([df_inter,nuevo_dataframe_2])

In [72]:
df_inter

,Problema Oncologico,RUT,DV,Nombre del Paciente,N° Telefono 1,N° Telefono 2,N° Telefono 3,Fecha de SIC,Establecimiento de Origen,Especialidad Destino,Estado de SIC,Fecha Proximo Hito,DIAGNOSTICOS,Nombre agenda Medico especialista
566200,Cáncer Colorectal,5192168,2,JUAN RAMÓN SUAZO ARAVENA,981443616,972334059,NaN,2024-06-25,HLF,Cirugía Abdominal Adulto,Pendiente de Autorizar,NaN,"Tumor maligno del colon,CANCER DE COLON",NaN
585759,Cáncer Colorectal,10168476,8,LUIS ALBERTO ÁLVAREZ PINO,96271467,NaN,NaN,2024-06-26,HLF,Cirugia Cabeza y Cuello,Pendiente de Autorizar,NaN,"SINDROME DE RESECCION ANTERIOR,Tumor maligno d...",NaN
585771,Cáncer Colorectal,8402829,0,JUAN ERNESTO AMAYA FAURA,0966644805,0940244902,NaN,2024-06-26,HLF,Urología Adulto,Pendiente de Autorizar,NaN,"Tumor maligno del recto,CANCER DE RECTO QUE TE...",NaN
585773,Cáncer Colorectal,10168476,8,LUIS ALBERTO ÁLVAREZ PINO,96271467,NaN,NaN,2024-06-26,HLF,Endocrinología Adulto,Pendiente de Autorizar,NaN,"SINDROME DE RESECCION ANTERIOR,Tumor maligno d...",NaN
608162,Cáncer Pulmón,7898285,3,HILDA MILAGRO LORCA LUCERO,58944803,22851980,NaN,2024-06-27,HLF,Cirugía Tórax Adulto,Pendiente de Autorizar,NaN,Tumor maligno de los bronquios y del pulmon,NaN
608236,Cáncer Próstata,4280602,1,JORGE LUIS ABARCA GÁLVEZ,979889506,976081118,997783585,2024-06-27,HLF,Cirugía Abdominal Adulto,Pendiente de Autorizar,NaN,Tumor maligno de la prostata,NaN


In [73]:
df_inter['Nombre del Paciente'] = df_inter['Nombre del Paciente'].astype(str)

In [74]:
nuevo_dataframe_2

,Problema Oncologico,RUT,DV,Nombre del Paciente,N° Telefono 1,N° Telefono 2,Fecha de SIC,Establecimiento de Origen,Especialidad Destino,Estado de SIC,N° Telefono 3,Nombre garantia,Fecha Proximo Hito,DIAGNOSTICOS,CODIGOS_DIAGNOSTICO,Nombre agenda Medico especialista
130,Cáncer Colorectal . {decreto n° 4/2013},11667136,0,"FERNÁNDEZ SILVA, IVÁN BLADIMIR",- / - / 22620004 / 98769960 / - / 22620004 / 9...,#ND,2024-07-03,NaN,NaN,NaN,NaN,Confirmación,NaN,NaN,NaN,NaN
4707,Cáncer Colorectal . {decreto n° 4/2013},8130920,5,"SILVA HENRÍQUEZ, GINA DE LAS MERCEDES",- / - / 68262953 / 68262953 / 68262953 / 31898...,#ND,2024-07-03,NaN,NaN,NaN,NaN,Confirmación,NaN,NaN,NaN,NaN
2790,Cáncer Colorectal . {decreto n° 4/2013},5869303,0,"RIQUELME PUEBLA, GLORIA MARÍA BERNARD",- / - / 25028287 / 0 / 97158876 / 25028287 / -...,#ND,2024-07-02,NaN,NaN,NaN,NaN,Confirmación,NaN,NaN,NaN,NaN
3021,Cáncer Colorectal . {decreto n° 4/2013},6094724,4,"PIZARRO SERRAT, RICARDO ROLANDO",- / - / / 0 / 93865732 / - / -/9993865732/299...,#ND,2024-07-02,NaN,NaN,NaN,NaN,Confirmación,NaN,NaN,NaN,NaN
4989,Cáncer Colorectal . {decreto n° 4/2013},8957749,7,"LEIVA CEPEDA, RODRIGO EDUARDO",- / - / 0 / 0 / 75530673 / / -/75530673//.,FIJO: NO ESPECIFICADO MOVIL: 75530673,2024-07-02,NaN,NaN,NaN,NaN,Confirmación,NaN,NaN,NaN,NaN
56,Cáncer Gástrico . {decreto nº 228},10492252,K,"VILCHES PÉREZ, MARISA DE LAS MERCED",- / - / 22828026 / 66931649 / 66931649 / 22828...,#ND,2024-06-28,NaN,NaN,NaN,NaN,Consulta Especialista,NaN,NaN,NaN,NaN
4049,Cáncer Colorectal . {decreto n° 4/2013},7196918,5,"GAJARDO DÍAZ, JUANA NATALIA",- / - / 85552235 / 85552235 / 85552235 / 85552...,#ND,2024-06-28,NaN,NaN,NaN,NaN,Confirmación,NaN,NaN,NaN,NaN
4325,Cáncer de Pulmón . {Decreto Nro 22/2019},7540942,7,"VALDEBENITO LOBOS, MARÍA ANGÉLICA",- / - / 0 / / 94364482 / - / -/.,#ND,2024-06-28,NaN,NaN,NaN,NaN,Confirmación Diagnóstica,NaN,NaN,NaN,NaN
4784,Cáncer de Pulmón . {Decreto Nro 22/2019},8317900,7,"MELLI NAVARRETE, ESTER LUCRECIA",- / - / 35766740 / 35766740 / 35766740 / 23140...,FIJO: 23140537 MOVIL: 935766740,2024-06-28,NaN,NaN,NaN,NaN,Confirmación Diagnóstica,NaN,NaN,NaN,NaN
148,Cáncer Gástrico . {decreto nº 228},11904915,6,"ROJAS SÁEZ, FLORENTINO AVELINO",#ND,#ND,2024-06-27,NaN,NaN,NaN,NaN,Consulta Especialista,NaN,NaN,NaN,NaN


In [75]:
df_concatenated_2

,Problema Oncologico,RUT,DV,Nombre del Paciente,N° Telefono 1,N° Telefono 2,N° Telefono 3,Fecha de SIC,Establecimiento de Origen,Especialidad Destino,Estado de SIC,Fecha Proximo Hito,DIAGNOSTICOS,Nombre agenda Medico especialista,Nombre garantia,CODIGOS_DIAGNOSTICO
566200,Cáncer Colorectal,5192168,2,JUAN RAMÓN SUAZO ARAVENA,981443616,972334059,NaN,2024-06-25,HLF,Cirugía Abdominal Adulto,Pendiente de Autorizar,NaN,"Tumor maligno del colon,CANCER DE COLON",NaN,NaN,NaN
585759,Cáncer Colorectal,10168476,8,LUIS ALBERTO ÁLVAREZ PINO,96271467,NaN,NaN,2024-06-26,HLF,Cirugia Cabeza y Cuello,Pendiente de Autorizar,NaN,"SINDROME DE RESECCION ANTERIOR,Tumor maligno d...",NaN,NaN,NaN
585771,Cáncer Colorectal,8402829,0,JUAN ERNESTO AMAYA FAURA,0966644805,0940244902,NaN,2024-06-26,HLF,Urología Adulto,Pendiente de Autorizar,NaN,"Tumor maligno del recto,CANCER DE RECTO QUE TE...",NaN,NaN,NaN
585773,Cáncer Colorectal,10168476,8,LUIS ALBERTO ÁLVAREZ PINO,96271467,NaN,NaN,2024-06-26,HLF,Endocrinología Adulto,Pendiente de Autorizar,NaN,"SINDROME DE RESECCION ANTERIOR,Tumor maligno d...",NaN,NaN,NaN
608162,Cáncer Pulmón,7898285,3,HILDA MILAGRO LORCA LUCERO,58944803,22851980,NaN,2024-06-27,HLF,Cirugía Tórax Adulto,Pendiente de Autorizar,NaN,Tumor maligno de los bronquios y del pulmon,NaN,NaN,NaN
608236,Cáncer Próstata,4280602,1,JORGE LUIS ABARCA GÁLVEZ,979889506,976081118,997783585,2024-06-27,HLF,Cirugía Abdominal Adulto,Pendiente de Autorizar,NaN,Tumor maligno de la prostata,NaN,NaN,NaN
130,Cáncer Colorectal . {decreto n° 4/2013},11667136,0,"FERNÁNDEZ SILVA, IVÁN BLADIMIR",- / - / 22620004 / 98769960 / - / 22620004 / 9...,#ND,NaN,2024-07-03,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN
4707,Cáncer Colorectal . {decreto n° 4/2013},8130920,5,"SILVA HENRÍQUEZ, GINA DE LAS MERCEDES",- / - / 68262953 / 68262953 / 68262953 / 31898...,#ND,NaN,2024-07-03,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN
2790,Cáncer Colorectal . {decreto n° 4/2013},5869303,0,"RIQUELME PUEBLA, GLORIA MARÍA BERNARD",- / - / 25028287 / 0 / 97158876 / 25028287 / -...,#ND,NaN,2024-07-02,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN
3021,Cáncer Colorectal . {decreto n° 4/2013},6094724,4,"PIZARRO SERRAT, RICARDO ROLANDO",- / - / / 0 / 93865732 / - / -/9993865732/299...,#ND,NaN,2024-07-02,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN


In [76]:
dataframe_tiroides['Problema de salud'].value_counts()

Problema de salud
Cáncer Colorectal . {decreto n° 4/2013}     11
Cáncer Gástrico . {decreto nº 228}           2
Cáncer de Pulmón . {Decreto Nro 22/2019}     2
Name: count, dtype: int64

In [77]:
df_concatenated_2['Problema Oncologico'] = df_concatenated_2['Problema Oncologico'].replace("Cáncer Gástrico . {decreto nº 228}","Cáncer Gástrico")
df_concatenated_2['Problema Oncologico'] = df_concatenated_2['Problema Oncologico'].replace("Cáncer Colorectal . {decreto n° 4/2013}","Cáncer Colorectal")
df_concatenated_2['Problema Oncologico'] = df_concatenated_2['Problema Oncologico'].replace("Linfoma en Adultos .{decreto nº 228}","Linfoma")
df_concatenated_2['Problema Oncologico'] = df_concatenated_2['Problema Oncologico'].replace("Cáncer de Pulmón . {Decreto Nro 22/2019}","Cáncer Pulmón")
df_concatenated_2['Problema Oncologico'] = df_concatenated_2['Problema Oncologico'].replace("Leucemia Adulto Leucemia Crónica {decreto nº 44}","Leucemia Adulto")

In [78]:
df_concatenated_2

,Problema Oncologico,RUT,DV,Nombre del Paciente,N° Telefono 1,N° Telefono 2,N° Telefono 3,Fecha de SIC,Establecimiento de Origen,Especialidad Destino,Estado de SIC,Fecha Proximo Hito,DIAGNOSTICOS,Nombre agenda Medico especialista,Nombre garantia,CODIGOS_DIAGNOSTICO
566200,Cáncer Colorectal,5192168,2,JUAN RAMÓN SUAZO ARAVENA,981443616,972334059,NaN,2024-06-25,HLF,Cirugía Abdominal Adulto,Pendiente de Autorizar,NaN,"Tumor maligno del colon,CANCER DE COLON",NaN,NaN,NaN
585759,Cáncer Colorectal,10168476,8,LUIS ALBERTO ÁLVAREZ PINO,96271467,NaN,NaN,2024-06-26,HLF,Cirugia Cabeza y Cuello,Pendiente de Autorizar,NaN,"SINDROME DE RESECCION ANTERIOR,Tumor maligno d...",NaN,NaN,NaN
585771,Cáncer Colorectal,8402829,0,JUAN ERNESTO AMAYA FAURA,0966644805,0940244902,NaN,2024-06-26,HLF,Urología Adulto,Pendiente de Autorizar,NaN,"Tumor maligno del recto,CANCER DE RECTO QUE TE...",NaN,NaN,NaN
585773,Cáncer Colorectal,10168476,8,LUIS ALBERTO ÁLVAREZ PINO,96271467,NaN,NaN,2024-06-26,HLF,Endocrinología Adulto,Pendiente de Autorizar,NaN,"SINDROME DE RESECCION ANTERIOR,Tumor maligno d...",NaN,NaN,NaN
608162,Cáncer Pulmón,7898285,3,HILDA MILAGRO LORCA LUCERO,58944803,22851980,NaN,2024-06-27,HLF,Cirugía Tórax Adulto,Pendiente de Autorizar,NaN,Tumor maligno de los bronquios y del pulmon,NaN,NaN,NaN
608236,Cáncer Próstata,4280602,1,JORGE LUIS ABARCA GÁLVEZ,979889506,976081118,997783585,2024-06-27,HLF,Cirugía Abdominal Adulto,Pendiente de Autorizar,NaN,Tumor maligno de la prostata,NaN,NaN,NaN
130,Cáncer Colorectal,11667136,0,"FERNÁNDEZ SILVA, IVÁN BLADIMIR",- / - / 22620004 / 98769960 / - / 22620004 / 9...,#ND,NaN,2024-07-03,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN
4707,Cáncer Colorectal,8130920,5,"SILVA HENRÍQUEZ, GINA DE LAS MERCEDES",- / - / 68262953 / 68262953 / 68262953 / 31898...,#ND,NaN,2024-07-03,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN
2790,Cáncer Colorectal,5869303,0,"RIQUELME PUEBLA, GLORIA MARÍA BERNARD",- / - / 25028287 / 0 / 97158876 / 25028287 / -...,#ND,NaN,2024-07-02,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN
3021,Cáncer Colorectal,6094724,4,"PIZARRO SERRAT, RICARDO ROLANDO",- / - / / 0 / 93865732 / - / -/9993865732/299...,#ND,NaN,2024-07-02,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN


In [79]:
resultado_filtrado_2 = df_concatenated_2.copy()#.drop_duplicates(subset='RUT', keep='first')

resultado_filtrado_2['RUT'] = resultado_filtrado_2['RUT'].apply(lambda x: str(x))


In [80]:
resultado_filtrado_2

,Problema Oncologico,RUT,DV,Nombre del Paciente,N° Telefono 1,N° Telefono 2,N° Telefono 3,Fecha de SIC,Establecimiento de Origen,Especialidad Destino,Estado de SIC,Fecha Proximo Hito,DIAGNOSTICOS,Nombre agenda Medico especialista,Nombre garantia,CODIGOS_DIAGNOSTICO
566200,Cáncer Colorectal,5192168,2,JUAN RAMÓN SUAZO ARAVENA,981443616,972334059,NaN,2024-06-25,HLF,Cirugía Abdominal Adulto,Pendiente de Autorizar,NaN,"Tumor maligno del colon,CANCER DE COLON",NaN,NaN,NaN
585759,Cáncer Colorectal,10168476,8,LUIS ALBERTO ÁLVAREZ PINO,96271467,NaN,NaN,2024-06-26,HLF,Cirugia Cabeza y Cuello,Pendiente de Autorizar,NaN,"SINDROME DE RESECCION ANTERIOR,Tumor maligno d...",NaN,NaN,NaN
585771,Cáncer Colorectal,8402829,0,JUAN ERNESTO AMAYA FAURA,0966644805,0940244902,NaN,2024-06-26,HLF,Urología Adulto,Pendiente de Autorizar,NaN,"Tumor maligno del recto,CANCER DE RECTO QUE TE...",NaN,NaN,NaN
585773,Cáncer Colorectal,10168476,8,LUIS ALBERTO ÁLVAREZ PINO,96271467,NaN,NaN,2024-06-26,HLF,Endocrinología Adulto,Pendiente de Autorizar,NaN,"SINDROME DE RESECCION ANTERIOR,Tumor maligno d...",NaN,NaN,NaN
608162,Cáncer Pulmón,7898285,3,HILDA MILAGRO LORCA LUCERO,58944803,22851980,NaN,2024-06-27,HLF,Cirugía Tórax Adulto,Pendiente de Autorizar,NaN,Tumor maligno de los bronquios y del pulmon,NaN,NaN,NaN
608236,Cáncer Próstata,4280602,1,JORGE LUIS ABARCA GÁLVEZ,979889506,976081118,997783585,2024-06-27,HLF,Cirugía Abdominal Adulto,Pendiente de Autorizar,NaN,Tumor maligno de la prostata,NaN,NaN,NaN
130,Cáncer Colorectal,11667136,0,"FERNÁNDEZ SILVA, IVÁN BLADIMIR",- / - / 22620004 / 98769960 / - / 22620004 / 9...,#ND,NaN,2024-07-03,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN
4707,Cáncer Colorectal,8130920,5,"SILVA HENRÍQUEZ, GINA DE LAS MERCEDES",- / - / 68262953 / 68262953 / 68262953 / 31898...,#ND,NaN,2024-07-03,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN
2790,Cáncer Colorectal,5869303,0,"RIQUELME PUEBLA, GLORIA MARÍA BERNARD",- / - / 25028287 / 0 / 97158876 / 25028287 / -...,#ND,NaN,2024-07-02,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN
3021,Cáncer Colorectal,6094724,4,"PIZARRO SERRAT, RICARDO ROLANDO",- / - / / 0 / 93865732 / - / -/9993865732/299...,#ND,NaN,2024-07-02,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN


# DEBUG 2

## Eliminar duplicadas 3 condiciones

In [81]:
resultado_filtrado_2

,Problema Oncologico,RUT,DV,Nombre del Paciente,N° Telefono 1,N° Telefono 2,N° Telefono 3,Fecha de SIC,Establecimiento de Origen,Especialidad Destino,Estado de SIC,Fecha Proximo Hito,DIAGNOSTICOS,Nombre agenda Medico especialista,Nombre garantia,CODIGOS_DIAGNOSTICO
566200,Cáncer Colorectal,5192168,2,JUAN RAMÓN SUAZO ARAVENA,981443616,972334059,NaN,2024-06-25,HLF,Cirugía Abdominal Adulto,Pendiente de Autorizar,NaN,"Tumor maligno del colon,CANCER DE COLON",NaN,NaN,NaN
585759,Cáncer Colorectal,10168476,8,LUIS ALBERTO ÁLVAREZ PINO,96271467,NaN,NaN,2024-06-26,HLF,Cirugia Cabeza y Cuello,Pendiente de Autorizar,NaN,"SINDROME DE RESECCION ANTERIOR,Tumor maligno d...",NaN,NaN,NaN
585771,Cáncer Colorectal,8402829,0,JUAN ERNESTO AMAYA FAURA,0966644805,0940244902,NaN,2024-06-26,HLF,Urología Adulto,Pendiente de Autorizar,NaN,"Tumor maligno del recto,CANCER DE RECTO QUE TE...",NaN,NaN,NaN
585773,Cáncer Colorectal,10168476,8,LUIS ALBERTO ÁLVAREZ PINO,96271467,NaN,NaN,2024-06-26,HLF,Endocrinología Adulto,Pendiente de Autorizar,NaN,"SINDROME DE RESECCION ANTERIOR,Tumor maligno d...",NaN,NaN,NaN
608162,Cáncer Pulmón,7898285,3,HILDA MILAGRO LORCA LUCERO,58944803,22851980,NaN,2024-06-27,HLF,Cirugía Tórax Adulto,Pendiente de Autorizar,NaN,Tumor maligno de los bronquios y del pulmon,NaN,NaN,NaN
608236,Cáncer Próstata,4280602,1,JORGE LUIS ABARCA GÁLVEZ,979889506,976081118,997783585,2024-06-27,HLF,Cirugía Abdominal Adulto,Pendiente de Autorizar,NaN,Tumor maligno de la prostata,NaN,NaN,NaN
130,Cáncer Colorectal,11667136,0,"FERNÁNDEZ SILVA, IVÁN BLADIMIR",- / - / 22620004 / 98769960 / - / 22620004 / 9...,#ND,NaN,2024-07-03,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN
4707,Cáncer Colorectal,8130920,5,"SILVA HENRÍQUEZ, GINA DE LAS MERCEDES",- / - / 68262953 / 68262953 / 68262953 / 31898...,#ND,NaN,2024-07-03,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN
2790,Cáncer Colorectal,5869303,0,"RIQUELME PUEBLA, GLORIA MARÍA BERNARD",- / - / 25028287 / 0 / 97158876 / 25028287 / -...,#ND,NaN,2024-07-02,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN
3021,Cáncer Colorectal,6094724,4,"PIZARRO SERRAT, RICARDO ROLANDO",- / - / / 0 / 93865732 / - / -/9993865732/299...,#ND,NaN,2024-07-02,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN


In [82]:
list(resultado_filtrado_2.columns)

['Problema Oncologico',
 'RUT',
 'DV',
 'Nombre del Paciente',
 'N° Telefono 1',
 'N° Telefono 2',
 'N° Telefono 3',
 'Fecha de SIC',
 'Establecimiento de Origen',
 'Especialidad Destino',
 'Estado de SIC',
 'Fecha Proximo Hito',
 'DIAGNOSTICOS',
 'Nombre agenda Medico especialista',
 'Nombre garantia',
 'CODIGOS_DIAGNOSTICO']

In [83]:
resultado_filtrado_2['Fecha de SIC'] = pd.to_datetime(resultado_filtrado_2['Fecha de SIC'], format='%d/%m/%Y')

In [84]:
resultado_filtrado_2 = resultado_filtrado_2.sort_values(by='Fecha de SIC', ascending=False)

In [85]:
resultado_filtrado_3 = resultado_filtrado_2.drop_duplicates(subset=['Problema Oncologico',
 'RUT',
 #'Fecha de SIC',
 'Especialidad Destino'
 ], keep= 'first')

In [86]:
resultado_filtrado_3

,Problema Oncologico,RUT,DV,Nombre del Paciente,N° Telefono 1,N° Telefono 2,N° Telefono 3,Fecha de SIC,Establecimiento de Origen,Especialidad Destino,Estado de SIC,Fecha Proximo Hito,DIAGNOSTICOS,Nombre agenda Medico especialista,Nombre garantia,CODIGOS_DIAGNOSTICO
130,Cáncer Colorectal,11667136,0,"FERNÁNDEZ SILVA, IVÁN BLADIMIR",- / - / 22620004 / 98769960 / - / 22620004 / 9...,#ND,NaN,2024-07-03,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN
4707,Cáncer Colorectal,8130920,5,"SILVA HENRÍQUEZ, GINA DE LAS MERCEDES",- / - / 68262953 / 68262953 / 68262953 / 31898...,#ND,NaN,2024-07-03,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN
4989,Cáncer Colorectal,8957749,7,"LEIVA CEPEDA, RODRIGO EDUARDO",- / - / 0 / 0 / 75530673 / / -/75530673//.,FIJO: NO ESPECIFICADO MOVIL: 75530673,NaN,2024-07-02,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN
2790,Cáncer Colorectal,5869303,0,"RIQUELME PUEBLA, GLORIA MARÍA BERNARD",- / - / 25028287 / 0 / 97158876 / 25028287 / -...,#ND,NaN,2024-07-02,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN
3021,Cáncer Colorectal,6094724,4,"PIZARRO SERRAT, RICARDO ROLANDO",- / - / / 0 / 93865732 / - / -/9993865732/299...,#ND,NaN,2024-07-02,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN
56,Cáncer Gástrico,10492252,K,"VILCHES PÉREZ, MARISA DE LAS MERCED",- / - / 22828026 / 66931649 / 66931649 / 22828...,#ND,NaN,2024-06-28,NaN,NaN,NaN,NaN,NaN,NaN,Consulta Especialista,NaN
4049,Cáncer Colorectal,7196918,5,"GAJARDO DÍAZ, JUANA NATALIA",- / - / 85552235 / 85552235 / 85552235 / 85552...,#ND,NaN,2024-06-28,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN
4325,Cáncer Pulmón,7540942,7,"VALDEBENITO LOBOS, MARÍA ANGÉLICA",- / - / 0 / / 94364482 / - / -/.,#ND,NaN,2024-06-28,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación Diagnóstica,NaN
4784,Cáncer Pulmón,8317900,7,"MELLI NAVARRETE, ESTER LUCRECIA",- / - / 35766740 / 35766740 / 35766740 / 23140...,FIJO: 23140537 MOVIL: 935766740,NaN,2024-06-28,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación Diagnóstica,NaN
608236,Cáncer Próstata,4280602,1,JORGE LUIS ABARCA GÁLVEZ,979889506,976081118,997783585,2024-06-27,HLF,Cirugía Abdominal Adulto,Pendiente de Autorizar,NaN,Tumor maligno de la prostata,NaN,NaN,NaN


In [87]:
resultado_filtrado_2

,Problema Oncologico,RUT,DV,Nombre del Paciente,N° Telefono 1,N° Telefono 2,N° Telefono 3,Fecha de SIC,Establecimiento de Origen,Especialidad Destino,Estado de SIC,Fecha Proximo Hito,DIAGNOSTICOS,Nombre agenda Medico especialista,Nombre garantia,CODIGOS_DIAGNOSTICO
130,Cáncer Colorectal,11667136,0,"FERNÁNDEZ SILVA, IVÁN BLADIMIR",- / - / 22620004 / 98769960 / - / 22620004 / 9...,#ND,NaN,2024-07-03,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN
4707,Cáncer Colorectal,8130920,5,"SILVA HENRÍQUEZ, GINA DE LAS MERCEDES",- / - / 68262953 / 68262953 / 68262953 / 31898...,#ND,NaN,2024-07-03,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN
4989,Cáncer Colorectal,8957749,7,"LEIVA CEPEDA, RODRIGO EDUARDO",- / - / 0 / 0 / 75530673 / / -/75530673//.,FIJO: NO ESPECIFICADO MOVIL: 75530673,NaN,2024-07-02,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN
2790,Cáncer Colorectal,5869303,0,"RIQUELME PUEBLA, GLORIA MARÍA BERNARD",- / - / 25028287 / 0 / 97158876 / 25028287 / -...,#ND,NaN,2024-07-02,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN
3021,Cáncer Colorectal,6094724,4,"PIZARRO SERRAT, RICARDO ROLANDO",- / - / / 0 / 93865732 / - / -/9993865732/299...,#ND,NaN,2024-07-02,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN
56,Cáncer Gástrico,10492252,K,"VILCHES PÉREZ, MARISA DE LAS MERCED",- / - / 22828026 / 66931649 / 66931649 / 22828...,#ND,NaN,2024-06-28,NaN,NaN,NaN,NaN,NaN,NaN,Consulta Especialista,NaN
4049,Cáncer Colorectal,7196918,5,"GAJARDO DÍAZ, JUANA NATALIA",- / - / 85552235 / 85552235 / 85552235 / 85552...,#ND,NaN,2024-06-28,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN
4325,Cáncer Pulmón,7540942,7,"VALDEBENITO LOBOS, MARÍA ANGÉLICA",- / - / 0 / / 94364482 / - / -/.,#ND,NaN,2024-06-28,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación Diagnóstica,NaN
4784,Cáncer Pulmón,8317900,7,"MELLI NAVARRETE, ESTER LUCRECIA",- / - / 35766740 / 35766740 / 35766740 / 23140...,FIJO: 23140537 MOVIL: 935766740,NaN,2024-06-28,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación Diagnóstica,NaN
608236,Cáncer Próstata,4280602,1,JORGE LUIS ABARCA GÁLVEZ,979889506,976081118,997783585,2024-06-27,HLF,Cirugía Abdominal Adulto,Pendiente de Autorizar,NaN,Tumor maligno de la prostata,NaN,NaN,NaN


In [88]:
duplicados = resultado_filtrado_2.duplicated(subset=['Problema Oncologico',
 'RUT',
 #'Fecha de SIC',
 #'Especialidad Destino'
 ], keep= False)

In [89]:
resultado_filtrado_2

,Problema Oncologico,RUT,DV,Nombre del Paciente,N° Telefono 1,N° Telefono 2,N° Telefono 3,Fecha de SIC,Establecimiento de Origen,Especialidad Destino,Estado de SIC,Fecha Proximo Hito,DIAGNOSTICOS,Nombre agenda Medico especialista,Nombre garantia,CODIGOS_DIAGNOSTICO
130,Cáncer Colorectal,11667136,0,"FERNÁNDEZ SILVA, IVÁN BLADIMIR",- / - / 22620004 / 98769960 / - / 22620004 / 9...,#ND,NaN,2024-07-03,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN
4707,Cáncer Colorectal,8130920,5,"SILVA HENRÍQUEZ, GINA DE LAS MERCEDES",- / - / 68262953 / 68262953 / 68262953 / 31898...,#ND,NaN,2024-07-03,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN
4989,Cáncer Colorectal,8957749,7,"LEIVA CEPEDA, RODRIGO EDUARDO",- / - / 0 / 0 / 75530673 / / -/75530673//.,FIJO: NO ESPECIFICADO MOVIL: 75530673,NaN,2024-07-02,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN
2790,Cáncer Colorectal,5869303,0,"RIQUELME PUEBLA, GLORIA MARÍA BERNARD",- / - / 25028287 / 0 / 97158876 / 25028287 / -...,#ND,NaN,2024-07-02,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN
3021,Cáncer Colorectal,6094724,4,"PIZARRO SERRAT, RICARDO ROLANDO",- / - / / 0 / 93865732 / - / -/9993865732/299...,#ND,NaN,2024-07-02,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN
56,Cáncer Gástrico,10492252,K,"VILCHES PÉREZ, MARISA DE LAS MERCED",- / - / 22828026 / 66931649 / 66931649 / 22828...,#ND,NaN,2024-06-28,NaN,NaN,NaN,NaN,NaN,NaN,Consulta Especialista,NaN
4049,Cáncer Colorectal,7196918,5,"GAJARDO DÍAZ, JUANA NATALIA",- / - / 85552235 / 85552235 / 85552235 / 85552...,#ND,NaN,2024-06-28,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN
4325,Cáncer Pulmón,7540942,7,"VALDEBENITO LOBOS, MARÍA ANGÉLICA",- / - / 0 / / 94364482 / - / -/.,#ND,NaN,2024-06-28,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación Diagnóstica,NaN
4784,Cáncer Pulmón,8317900,7,"MELLI NAVARRETE, ESTER LUCRECIA",- / - / 35766740 / 35766740 / 35766740 / 23140...,FIJO: 23140537 MOVIL: 935766740,NaN,2024-06-28,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación Diagnóstica,NaN
608236,Cáncer Próstata,4280602,1,JORGE LUIS ABARCA GÁLVEZ,979889506,976081118,997783585,2024-06-27,HLF,Cirugía Abdominal Adulto,Pendiente de Autorizar,NaN,Tumor maligno de la prostata,NaN,NaN,NaN


## Eliminar duplicados GES

In [90]:
duplicados = resultado_filtrado_2.duplicated(subset=['Problema Oncologico',
 'RUT',
 #'Fecha de SIC',
 #'Especialidad Destino'
 ], keep= False)

In [91]:
resultado_filtrado_2[duplicados]

,Problema Oncologico,RUT,DV,Nombre del Paciente,N° Telefono 1,N° Telefono 2,N° Telefono 3,Fecha de SIC,Establecimiento de Origen,Especialidad Destino,Estado de SIC,Fecha Proximo Hito,DIAGNOSTICOS,Nombre agenda Medico especialista,Nombre garantia,CODIGOS_DIAGNOSTICO
585773,Cáncer Colorectal,10168476,8,LUIS ALBERTO ÁLVAREZ PINO,96271467,NaN,NaN,2024-06-26,HLF,Endocrinología Adulto,Pendiente de Autorizar,NaN,"SINDROME DE RESECCION ANTERIOR,Tumor maligno d...",NaN,NaN,NaN
585759,Cáncer Colorectal,10168476,8,LUIS ALBERTO ÁLVAREZ PINO,96271467,NaN,NaN,2024-06-26,HLF,Cirugia Cabeza y Cuello,Pendiente de Autorizar,NaN,"SINDROME DE RESECCION ANTERIOR,Tumor maligno d...",NaN,NaN,NaN


In [92]:
resultado_filtrado_2['Especialidad Destino'].value_counts()

Especialidad Destino
Cirugía Abdominal Adulto    2
Cirugía Tórax Adulto        1
Endocrinología Adulto       1
Urología Adulto             1
Cirugia Cabeza y Cuello     1
Name: count, dtype: int64

In [93]:
resultado_filtrado_2[resultado_filtrado_2['Especialidad Destino'].isnull()]

,Problema Oncologico,RUT,DV,Nombre del Paciente,N° Telefono 1,N° Telefono 2,N° Telefono 3,Fecha de SIC,Establecimiento de Origen,Especialidad Destino,Estado de SIC,Fecha Proximo Hito,DIAGNOSTICOS,Nombre agenda Medico especialista,Nombre garantia,CODIGOS_DIAGNOSTICO
130,Cáncer Colorectal,11667136,0,"FERNÁNDEZ SILVA, IVÁN BLADIMIR",- / - / 22620004 / 98769960 / - / 22620004 / 9...,#ND,NaN,2024-07-03,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN
4707,Cáncer Colorectal,8130920,5,"SILVA HENRÍQUEZ, GINA DE LAS MERCEDES",- / - / 68262953 / 68262953 / 68262953 / 31898...,#ND,NaN,2024-07-03,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN
4989,Cáncer Colorectal,8957749,7,"LEIVA CEPEDA, RODRIGO EDUARDO",- / - / 0 / 0 / 75530673 / / -/75530673//.,FIJO: NO ESPECIFICADO MOVIL: 75530673,NaN,2024-07-02,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN
2790,Cáncer Colorectal,5869303,0,"RIQUELME PUEBLA, GLORIA MARÍA BERNARD",- / - / 25028287 / 0 / 97158876 / 25028287 / -...,#ND,NaN,2024-07-02,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN
3021,Cáncer Colorectal,6094724,4,"PIZARRO SERRAT, RICARDO ROLANDO",- / - / / 0 / 93865732 / - / -/9993865732/299...,#ND,NaN,2024-07-02,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN
56,Cáncer Gástrico,10492252,K,"VILCHES PÉREZ, MARISA DE LAS MERCED",- / - / 22828026 / 66931649 / 66931649 / 22828...,#ND,NaN,2024-06-28,NaN,NaN,NaN,NaN,NaN,NaN,Consulta Especialista,NaN
4049,Cáncer Colorectal,7196918,5,"GAJARDO DÍAZ, JUANA NATALIA",- / - / 85552235 / 85552235 / 85552235 / 85552...,#ND,NaN,2024-06-28,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN
4325,Cáncer Pulmón,7540942,7,"VALDEBENITO LOBOS, MARÍA ANGÉLICA",- / - / 0 / / 94364482 / - / -/.,#ND,NaN,2024-06-28,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación Diagnóstica,NaN
4784,Cáncer Pulmón,8317900,7,"MELLI NAVARRETE, ESTER LUCRECIA",- / - / 35766740 / 35766740 / 35766740 / 23140...,FIJO: 23140537 MOVIL: 935766740,NaN,2024-06-28,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación Diagnóstica,NaN
148,Cáncer Gástrico,11904915,6,"ROJAS SÁEZ, FLORENTINO AVELINO",#ND,#ND,NaN,2024-06-27,NaN,NaN,NaN,NaN,NaN,NaN,Consulta Especialista,NaN


In [94]:
resultado_filtrado_2_especialidad = resultado_filtrado_2[~(resultado_filtrado_2['Especialidad Destino'].isnull())]
resultado_filtrado_2_no_especialidad = resultado_filtrado_2[(resultado_filtrado_2['Especialidad Destino'].isnull())]

In [95]:
resultado_filtrado_2_no_especialidad

,Problema Oncologico,RUT,DV,Nombre del Paciente,N° Telefono 1,N° Telefono 2,N° Telefono 3,Fecha de SIC,Establecimiento de Origen,Especialidad Destino,Estado de SIC,Fecha Proximo Hito,DIAGNOSTICOS,Nombre agenda Medico especialista,Nombre garantia,CODIGOS_DIAGNOSTICO
130,Cáncer Colorectal,11667136,0,"FERNÁNDEZ SILVA, IVÁN BLADIMIR",- / - / 22620004 / 98769960 / - / 22620004 / 9...,#ND,NaN,2024-07-03,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN
4707,Cáncer Colorectal,8130920,5,"SILVA HENRÍQUEZ, GINA DE LAS MERCEDES",- / - / 68262953 / 68262953 / 68262953 / 31898...,#ND,NaN,2024-07-03,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN
4989,Cáncer Colorectal,8957749,7,"LEIVA CEPEDA, RODRIGO EDUARDO",- / - / 0 / 0 / 75530673 / / -/75530673//.,FIJO: NO ESPECIFICADO MOVIL: 75530673,NaN,2024-07-02,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN
2790,Cáncer Colorectal,5869303,0,"RIQUELME PUEBLA, GLORIA MARÍA BERNARD",- / - / 25028287 / 0 / 97158876 / 25028287 / -...,#ND,NaN,2024-07-02,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN
3021,Cáncer Colorectal,6094724,4,"PIZARRO SERRAT, RICARDO ROLANDO",- / - / / 0 / 93865732 / - / -/9993865732/299...,#ND,NaN,2024-07-02,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN
56,Cáncer Gástrico,10492252,K,"VILCHES PÉREZ, MARISA DE LAS MERCED",- / - / 22828026 / 66931649 / 66931649 / 22828...,#ND,NaN,2024-06-28,NaN,NaN,NaN,NaN,NaN,NaN,Consulta Especialista,NaN
4049,Cáncer Colorectal,7196918,5,"GAJARDO DÍAZ, JUANA NATALIA",- / - / 85552235 / 85552235 / 85552235 / 85552...,#ND,NaN,2024-06-28,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN
4325,Cáncer Pulmón,7540942,7,"VALDEBENITO LOBOS, MARÍA ANGÉLICA",- / - / 0 / / 94364482 / - / -/.,#ND,NaN,2024-06-28,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación Diagnóstica,NaN
4784,Cáncer Pulmón,8317900,7,"MELLI NAVARRETE, ESTER LUCRECIA",- / - / 35766740 / 35766740 / 35766740 / 23140...,FIJO: 23140537 MOVIL: 935766740,NaN,2024-06-28,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación Diagnóstica,NaN
148,Cáncer Gástrico,11904915,6,"ROJAS SÁEZ, FLORENTINO AVELINO",#ND,#ND,NaN,2024-06-27,NaN,NaN,NaN,NaN,NaN,NaN,Consulta Especialista,NaN


In [96]:
resultado_filtrado_2_especialidad

,Problema Oncologico,RUT,DV,Nombre del Paciente,N° Telefono 1,N° Telefono 2,N° Telefono 3,Fecha de SIC,Establecimiento de Origen,Especialidad Destino,Estado de SIC,Fecha Proximo Hito,DIAGNOSTICOS,Nombre agenda Medico especialista,Nombre garantia,CODIGOS_DIAGNOSTICO
608236,Cáncer Próstata,4280602,1,JORGE LUIS ABARCA GÁLVEZ,979889506,976081118,997783585,2024-06-27,HLF,Cirugía Abdominal Adulto,Pendiente de Autorizar,NaN,Tumor maligno de la prostata,NaN,NaN,NaN
608162,Cáncer Pulmón,7898285,3,HILDA MILAGRO LORCA LUCERO,58944803,22851980,NaN,2024-06-27,HLF,Cirugía Tórax Adulto,Pendiente de Autorizar,NaN,Tumor maligno de los bronquios y del pulmon,NaN,NaN,NaN
585773,Cáncer Colorectal,10168476,8,LUIS ALBERTO ÁLVAREZ PINO,96271467,NaN,NaN,2024-06-26,HLF,Endocrinología Adulto,Pendiente de Autorizar,NaN,"SINDROME DE RESECCION ANTERIOR,Tumor maligno d...",NaN,NaN,NaN
585771,Cáncer Colorectal,8402829,0,JUAN ERNESTO AMAYA FAURA,0966644805,0940244902,NaN,2024-06-26,HLF,Urología Adulto,Pendiente de Autorizar,NaN,"Tumor maligno del recto,CANCER DE RECTO QUE TE...",NaN,NaN,NaN
585759,Cáncer Colorectal,10168476,8,LUIS ALBERTO ÁLVAREZ PINO,96271467,NaN,NaN,2024-06-26,HLF,Cirugia Cabeza y Cuello,Pendiente de Autorizar,NaN,"SINDROME DE RESECCION ANTERIOR,Tumor maligno d...",NaN,NaN,NaN
566200,Cáncer Colorectal,5192168,2,JUAN RAMÓN SUAZO ARAVENA,981443616,972334059,NaN,2024-06-25,HLF,Cirugía Abdominal Adulto,Pendiente de Autorizar,NaN,"Tumor maligno del colon,CANCER DE COLON",NaN,NaN,NaN


In [97]:
resultado_filtrado_2

,Problema Oncologico,RUT,DV,Nombre del Paciente,N° Telefono 1,N° Telefono 2,N° Telefono 3,Fecha de SIC,Establecimiento de Origen,Especialidad Destino,Estado de SIC,Fecha Proximo Hito,DIAGNOSTICOS,Nombre agenda Medico especialista,Nombre garantia,CODIGOS_DIAGNOSTICO
130,Cáncer Colorectal,11667136,0,"FERNÁNDEZ SILVA, IVÁN BLADIMIR",- / - / 22620004 / 98769960 / - / 22620004 / 9...,#ND,NaN,2024-07-03,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN
4707,Cáncer Colorectal,8130920,5,"SILVA HENRÍQUEZ, GINA DE LAS MERCEDES",- / - / 68262953 / 68262953 / 68262953 / 31898...,#ND,NaN,2024-07-03,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN
4989,Cáncer Colorectal,8957749,7,"LEIVA CEPEDA, RODRIGO EDUARDO",- / - / 0 / 0 / 75530673 / / -/75530673//.,FIJO: NO ESPECIFICADO MOVIL: 75530673,NaN,2024-07-02,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN
2790,Cáncer Colorectal,5869303,0,"RIQUELME PUEBLA, GLORIA MARÍA BERNARD",- / - / 25028287 / 0 / 97158876 / 25028287 / -...,#ND,NaN,2024-07-02,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN
3021,Cáncer Colorectal,6094724,4,"PIZARRO SERRAT, RICARDO ROLANDO",- / - / / 0 / 93865732 / - / -/9993865732/299...,#ND,NaN,2024-07-02,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN
56,Cáncer Gástrico,10492252,K,"VILCHES PÉREZ, MARISA DE LAS MERCED",- / - / 22828026 / 66931649 / 66931649 / 22828...,#ND,NaN,2024-06-28,NaN,NaN,NaN,NaN,NaN,NaN,Consulta Especialista,NaN
4049,Cáncer Colorectal,7196918,5,"GAJARDO DÍAZ, JUANA NATALIA",- / - / 85552235 / 85552235 / 85552235 / 85552...,#ND,NaN,2024-06-28,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN
4325,Cáncer Pulmón,7540942,7,"VALDEBENITO LOBOS, MARÍA ANGÉLICA",- / - / 0 / / 94364482 / - / -/.,#ND,NaN,2024-06-28,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación Diagnóstica,NaN
4784,Cáncer Pulmón,8317900,7,"MELLI NAVARRETE, ESTER LUCRECIA",- / - / 35766740 / 35766740 / 35766740 / 23140...,FIJO: 23140537 MOVIL: 935766740,NaN,2024-06-28,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación Diagnóstica,NaN
608236,Cáncer Próstata,4280602,1,JORGE LUIS ABARCA GÁLVEZ,979889506,976081118,997783585,2024-06-27,HLF,Cirugía Abdominal Adulto,Pendiente de Autorizar,NaN,Tumor maligno de la prostata,NaN,NaN,NaN


In [98]:
# Guardar el DataFrame en un archivo de Excel

# Día actual
today = date.today()

# Fecha actual
now = datetime.now()

# Formatear día con ceros a la izquierda si es necesario
day_formatted = "{:02d}".format(today.day)

# Formatear mes con ceros a la izquierda si es necesario
month_formatted = "{:02d}".format(today.month)


nombre_archivo = f'{today.year}{month_formatted}{day_formatted}_derivaciones_antiguas.xlsx'  # Nombre del archivo de Excel
#resultado_filtrado_2.to_excel(os.path.join(dir_actual, '..','DERIVACIONES','Reporte diario casos nuevos',f'{nombre_archivo}'), index=False)  # index=False para no incluir el índice en el archivo de Excel 


# Base Gestora HLF

In [99]:

dataframe = resultado_filtrado_2.copy()

# Obtener la ruta del directorio actual del script
#dir_actual = os.path.dirname(__file__)
#dir_actual = os.getcwd()

dir_actual = os.getcwd()


# Construir la ruta completa al archivo CSV
archivo_excel = os.path.join(dir_actual, '..','DERIVACIONES', 'BASE DE TRABAJO_DERIVACIONES.xlsx') # Gestion_Oncologica -> apolo

dataframe_2 = pd.read_excel(archivo_excel, sheet_name='Data')


# Obtener la cantidad de filas que no son nulas en la columna 'mi_columna'
cantidad_filas_no_nulas = dataframe_2['Rut'].count()



print("Cantidad de pacientes en la base", cantidad_filas_no_nulas)



dataframe_2 = dataframe_2[:cantidad_filas_no_nulas]

dataframe_2.reset_index(drop=True, inplace=True)


# Renombrar la columna 'old_name' a 'new_name'
dataframe_2 = dataframe_2.rename(columns={'Rut': 'RUT'})


dataframe_2 = dataframe_2.rename(columns={'Problema de Salud': 'Problema Oncologico'})

dataframe_2 = dataframe_2.rename(columns={'Nombre': 'Nombre del Paciente'})

# Definir una función para quitar los últimos dos dígitos
def quitar_ultimos_dos(elemento):
    return np.int64(elemento[:-2])

# Convertir todos los elementos de la columna 'RUT' a string
dataframe_2['RUT'] = dataframe_2['RUT'].astype(str)

# Aplicar la función a la columna 'RUT'
#dataframe_2['RUT'] = dataframe_2['RUT'].apply(quitar_ultimos_dos)



# Identificar las filas donde 'DV' es NaN
filas_dv_nan = dataframe_2[dataframe_2['DV'].isna()]

# Iterar sobre estas filas y corregir los valores de 'DV' y 'Rut'
for index, fila in filas_dv_nan.iterrows():
    rut = str(fila['RUT'])
    if True:
      dv = rut[-1]  # Último dígito del Rut
      dataframe_2.at[index, 'DV'] = dv
      dataframe_2.at[index, 'RUT'] = rut[:-1]  # Eliminar el último dígito del Rut
dataframe_2['RUT'] = dataframe_2['RUT'].astype(float)
dataframe_2['RUT'] = dataframe_2['RUT'].astype(int)


Cantidad de pacientes en la base 325


## Base Gestora Local

In [100]:
#dataframe = resultado_filtrado_2.copy()

#dir_actual = os.getcwd()


# Construir la ruta relativa al directorio donde se encuentra el archivo CSV
#dir_archivo_xlsx = os.path.join(dir_actual, '..','..','Datos HLF','BDD Gestoras')

# Construir la ruta completa al archivo CSV
#archivo_excel = os.path.join(dir_archivo_xlsx, 'BASE DE TRABAJO_DERIVACIONES.xlsx') # Gestion_Oncologica -> apolo

#dataframe_2 = pd.read_excel(archivo_excel, sheet_name='Data')


#cantidad_filas_no_nulas = dataframe_2['Rut'].count()



#print("Cantidad de pacientes en la base", cantidad_filas_no_nulas)



#dataframe_2 = dataframe_2[:cantidad_filas_no_nulas]

#dataframe_2.reset_index(drop=True, inplace=True)


# Renombrar la columna 'old_name' a 'new_name'
#dataframe_2 = dataframe_2.rename(columns={'Rut': 'RUT'})


#dataframe_2 = dataframe_2.rename(columns={'Problema de Salud': 'Problema Oncologico'})

#dataframe_2 = dataframe_2.rename(columns={'Nombre': 'Nombre del Paciente'})

# Definir una función para quitar los últimos dos dígitos
#def quitar_ultimos_dos(elemento):
#    return np.int64(elemento[:-2])

# Convertir todos los elementos de la columna 'RUT' a string
#dataframe_2['RUT'] = dataframe_2['RUT'].astype(str)

# Aplicar la función a la columna 'RUT'
#dataframe_2['RUT'] = dataframe_2['RUT'].apply(quitar_ultimos_dos)



# Identificar las filas donde 'DV' es NaN
#filas_dv_nan = dataframe_2[dataframe_2['DV'].isna()]

# Iterar sobre estas filas y corregir los valores de 'DV' y 'Rut'
#for index, fila in filas_dv_nan.iterrows():
#    rut = str(fila['RUT'])
#    if True:
#      dv = rut[-1]  # Último dígito del Rut
#      dataframe_2.at[index, 'DV'] = dv
 #     dataframe_2.at[index, 'RUT'] = rut[:-1]  # Eliminar el último dígito del Rut

#dataframe_2['RUT'] = dataframe_2['RUT'].astype(int)

# Merge

In [101]:
dataframe

,Problema Oncologico,RUT,DV,Nombre del Paciente,N° Telefono 1,N° Telefono 2,N° Telefono 3,Fecha de SIC,Establecimiento de Origen,Especialidad Destino,Estado de SIC,Fecha Proximo Hito,DIAGNOSTICOS,Nombre agenda Medico especialista,Nombre garantia,CODIGOS_DIAGNOSTICO
130,Cáncer Colorectal,11667136,0,"FERNÁNDEZ SILVA, IVÁN BLADIMIR",- / - / 22620004 / 98769960 / - / 22620004 / 9...,#ND,NaN,2024-07-03,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN
4707,Cáncer Colorectal,8130920,5,"SILVA HENRÍQUEZ, GINA DE LAS MERCEDES",- / - / 68262953 / 68262953 / 68262953 / 31898...,#ND,NaN,2024-07-03,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN
4989,Cáncer Colorectal,8957749,7,"LEIVA CEPEDA, RODRIGO EDUARDO",- / - / 0 / 0 / 75530673 / / -/75530673//.,FIJO: NO ESPECIFICADO MOVIL: 75530673,NaN,2024-07-02,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN
2790,Cáncer Colorectal,5869303,0,"RIQUELME PUEBLA, GLORIA MARÍA BERNARD",- / - / 25028287 / 0 / 97158876 / 25028287 / -...,#ND,NaN,2024-07-02,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN
3021,Cáncer Colorectal,6094724,4,"PIZARRO SERRAT, RICARDO ROLANDO",- / - / / 0 / 93865732 / - / -/9993865732/299...,#ND,NaN,2024-07-02,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN
56,Cáncer Gástrico,10492252,K,"VILCHES PÉREZ, MARISA DE LAS MERCED",- / - / 22828026 / 66931649 / 66931649 / 22828...,#ND,NaN,2024-06-28,NaN,NaN,NaN,NaN,NaN,NaN,Consulta Especialista,NaN
4049,Cáncer Colorectal,7196918,5,"GAJARDO DÍAZ, JUANA NATALIA",- / - / 85552235 / 85552235 / 85552235 / 85552...,#ND,NaN,2024-06-28,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación,NaN
4325,Cáncer Pulmón,7540942,7,"VALDEBENITO LOBOS, MARÍA ANGÉLICA",- / - / 0 / / 94364482 / - / -/.,#ND,NaN,2024-06-28,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación Diagnóstica,NaN
4784,Cáncer Pulmón,8317900,7,"MELLI NAVARRETE, ESTER LUCRECIA",- / - / 35766740 / 35766740 / 35766740 / 23140...,FIJO: 23140537 MOVIL: 935766740,NaN,2024-06-28,NaN,NaN,NaN,NaN,NaN,NaN,Confirmación Diagnóstica,NaN
608236,Cáncer Próstata,4280602,1,JORGE LUIS ABARCA GÁLVEZ,979889506,976081118,997783585,2024-06-27,HLF,Cirugía Abdominal Adulto,Pendiente de Autorizar,NaN,Tumor maligno de la prostata,NaN,NaN,NaN


In [102]:
dataframe_2

,Problema Oncologico,RUT,DV,Nombre del Paciente,Estado,Fecha Próximo Hito,Medico Responsable,Especialidad,Fecha de SIC,Estado de SIC,Establecimiento de Origen,Observaciones,Diagnosticos,Telefonos,CONCATENADO,CITAS FUTURAS,MEDICO,ESTADO SIC
0,Cáncer Próstata,5926301,3,ADA VILCHES MORALES,Priorizacion CUD,NaN,NaN,Urología Adulto,2024-05-13,NaN,NaN,NaN,NaN,NaN,5926301Urología Adulto,NaN,NaN,Solicitada
1,Cáncer Tiroides,16698412,2,Canales Marchant Nicole Paulina,Priorizacion CUD,NaN,NaN,Endocrinología Adulto,2024-05-08,NaN,CESFAM BELLAVISTA,NaN,NaN,NaN,16698412Endocrinología Adulto,NaN,NaN,Finalizada
2,Cáncer Próstata,99616630,K,CARLOS AGUILAR SALAS,Priorizacion CUD,NaN,NaN,Urología Adulto,2024-05-02,NaN,NaN,NaN,NaN,NaN,99616630Urología Adulto,NaN,NaN,NaN
3,Cáncer Pulmón,7408006,5,Cuevas Villalobos Eulalia Iris,Rechazada,NaN,NaN,Broncopulmonar Adulto,2024-06-03,rechazada,CESFAM JOSE ALVO,no pertinente (cud),NaN,NaN,7408006Broncopulmonar Adulto,NaN,NaN,NaN
4,NaN,7262387,8,Cuevas Villalobos Rene Edmundo,Priorizacion CUD,NaN,NaN,Dermatología Adulto,2024-06-07,NaN,CESFAM JOSE ALVO,NaN,NaN,NaN,7262387DERMATOLOGIA,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320,Cáncer Colorectal,7196918,5,"GAJARDO DÍAZ, JUANA NATALIA",NaN,NaN,NaN,NaN,2024-06-28,NaN,NaN,NaN,NaN,- / - / 85552235 / 85552235 / 85552235 / 85552...,NaN,NaN,NaN,NaN
321,Cáncer Colorectal,8941727,9,"MEDINA JIMÉNEZ, LUCRECIA DEL CARMEN",NaN,NaN,NaN,NaN,2024-06-25,NaN,NaN,NaN,NaN,- / - / / / 63744000 / 28391288 / -/96374400...,NaN,NaN,NaN,NaN
322,Cáncer Gástrico,11904915,6,"ROJAS SÁEZ, FLORENTINO AVELINO",NaN,NaN,NaN,NaN,2024-06-27,NaN,NaN,NaN,NaN,#ND/#ND/-,NaN,NaN,NaN,NaN
323,Cáncer Colorectal,12108563,1,"GUAJARDO MELLA, JACQUELINE DEL CARMEN",NaN,NaN,NaN,NaN,2024-06-24,NaN,NaN,NaN,NaN,- / - / 42010925 / 42010925 / 67434851 / 67434...,NaN,NaN,NaN,NaN


In [103]:
list(dataframe_2.columns)

['Problema Oncologico',
 'RUT',
 'DV',
 'Nombre del Paciente',
 'Estado',
 'Fecha Próximo Hito',
 'Medico Responsable',
 'Especialidad',
 'Fecha de SIC',
 'Estado de SIC',
 'Establecimiento de Origen',
 'Observaciones',
 'Diagnosticos',
 'Telefonos',
 'CONCATENADO',
 'CITAS FUTURAS',
 'MEDICO',
 'ESTADO SIC']

In [104]:
#dataframe_2 = pd.DataFrame(columns = ['Problema Oncologico',
# 'RUT',
# 'DV',
# 'Nombre del Paciente',
# 'Estado',
# 'Fecha Próximo Hito',
# 'Medico Responsable',
# 'Especialidad',
# 'Fecha de SIC',
# 'Estado de SIC',
# 'Establecimiento de Origen',
# 'Observaciones',
# 'Diagnosticos',
# 'Telefonos',
# 'CITAS FUTURAS',
# 'MEDICO',
# 'ESTADO SIC'])

In [105]:
dataframe_2

,Problema Oncologico,RUT,DV,Nombre del Paciente,Estado,Fecha Próximo Hito,Medico Responsable,Especialidad,Fecha de SIC,Estado de SIC,Establecimiento de Origen,Observaciones,Diagnosticos,Telefonos,CONCATENADO,CITAS FUTURAS,MEDICO,ESTADO SIC
0,Cáncer Próstata,5926301,3,ADA VILCHES MORALES,Priorizacion CUD,NaN,NaN,Urología Adulto,2024-05-13,NaN,NaN,NaN,NaN,NaN,5926301Urología Adulto,NaN,NaN,Solicitada
1,Cáncer Tiroides,16698412,2,Canales Marchant Nicole Paulina,Priorizacion CUD,NaN,NaN,Endocrinología Adulto,2024-05-08,NaN,CESFAM BELLAVISTA,NaN,NaN,NaN,16698412Endocrinología Adulto,NaN,NaN,Finalizada
2,Cáncer Próstata,99616630,K,CARLOS AGUILAR SALAS,Priorizacion CUD,NaN,NaN,Urología Adulto,2024-05-02,NaN,NaN,NaN,NaN,NaN,99616630Urología Adulto,NaN,NaN,NaN
3,Cáncer Pulmón,7408006,5,Cuevas Villalobos Eulalia Iris,Rechazada,NaN,NaN,Broncopulmonar Adulto,2024-06-03,rechazada,CESFAM JOSE ALVO,no pertinente (cud),NaN,NaN,7408006Broncopulmonar Adulto,NaN,NaN,NaN
4,NaN,7262387,8,Cuevas Villalobos Rene Edmundo,Priorizacion CUD,NaN,NaN,Dermatología Adulto,2024-06-07,NaN,CESFAM JOSE ALVO,NaN,NaN,NaN,7262387DERMATOLOGIA,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320,Cáncer Colorectal,7196918,5,"GAJARDO DÍAZ, JUANA NATALIA",NaN,NaN,NaN,NaN,2024-06-28,NaN,NaN,NaN,NaN,- / - / 85552235 / 85552235 / 85552235 / 85552...,NaN,NaN,NaN,NaN
321,Cáncer Colorectal,8941727,9,"MEDINA JIMÉNEZ, LUCRECIA DEL CARMEN",NaN,NaN,NaN,NaN,2024-06-25,NaN,NaN,NaN,NaN,- / - / / / 63744000 / 28391288 / -/96374400...,NaN,NaN,NaN,NaN
322,Cáncer Gástrico,11904915,6,"ROJAS SÁEZ, FLORENTINO AVELINO",NaN,NaN,NaN,NaN,2024-06-27,NaN,NaN,NaN,NaN,#ND/#ND/-,NaN,NaN,NaN,NaN
323,Cáncer Colorectal,12108563,1,"GUAJARDO MELLA, JACQUELINE DEL CARMEN",NaN,NaN,NaN,NaN,2024-06-24,NaN,NaN,NaN,NaN,- / - / 42010925 / 42010925 / 67434851 / 67434...,NaN,NaN,NaN,NaN


In [106]:
# Ejemplo de creación de DataFrame (puedes cargar tus propios DataFrames)
df1 = dataframe.copy()
df2 = dataframe_2.copy()

# Agrega una columna para identificar el origen de cada RUT
df1['_df'] = 'df1'
df2['_df'] = 'df2'

# Convertir la columna 'RUT' en ambos DataFrames al mismo tipo de datos
df1['RUT'] = df1['RUT'].astype(str)
df2['RUT'] = df2['RUT'].astype(str)
df1['RUT'] = df1['RUT'].astype(int)
df2['RUT'] = df2['RUT'].astype(int)


df2.rename(columns={'Especialidad': 'Especialidad Destino'}, inplace=True)




# Realiza un merge de los dos DataFrames en la columna 'RUT' y conserva solo las filas que aparecen en 'left_only'
df_merge = pd.merge(df1, df2, on=['RUT','Problema Oncologico','Especialidad Destino'], how='outer', indicator=True)


# Combinar las columnas 'DV_x' y 'DV_y' en una sola columna 'DV'
df_merge['Nombre del Paciente_x'].fillna(df_merge['Nombre del Paciente_y'], inplace=True)

# Renombrar la columna 'DV_x' a 'DV'
df_merge.rename(columns={'Nombre del Paciente_x': 'Nombre del Paciente'}, inplace=True)

# Eliminar la columna 'DV_y' si ya no es necesaria
#df_merge.drop(columns=['Nombre del Paciente_y'], inplace=True)

# Combinar las columnas 'DV_x' y 'DV_y' en una sola columna 'DV'
#df_merge['Problema Oncologico_x'].fillna(df_merge['Problema Oncologico_y'], inplace=True)

# Renombrar la columna 'DV_x' a 'DV'
df_merge.rename(columns={'Problema Oncologico_x': 'Problema Oncologico'}, inplace=True)

# Eliminar la columna 'DV_y' si ya no es necesaria
#df_merge.drop(columns=['Problema Oncologico_y'], inplace=True)


# Combinar las columnas 'DV_x' y 'DV_y' en una sola columna 'DV'
df_merge['DV_x'].fillna(df_merge['DV_y'], inplace=True)

# Renombrar la columna 'DV_x' a 'DV'
df_merge.rename(columns={'DV_x': 'DV'}, inplace=True)

# Eliminar la columna 'DV_y' si ya no es necesaria
df_merge.drop(columns=['DV_y'], inplace=True)

df_ca_tiroide = df_merge[df_merge['_merge']=='left_only']

# Reemplazar los valores NaN por '-'
df_ca_tiroide['N° Telefono 1'] = df_ca_tiroide['N° Telefono 1'].fillna('-')
df_ca_tiroide['N° Telefono 2'] = df_ca_tiroide['N° Telefono 2'].fillna('-')
df_ca_tiroide['N° Telefono 3'] = df_ca_tiroide['N° Telefono 3'].fillna('-')

def combinar_telefonos(row):
    return '/'.join(filter(None, [str(row['N° Telefono 1']), str(row['N° Telefono 2']), str(row['N° Telefono 3'])]))

# Aplicar la función a lo largo de las filas del DataFrame para crear la columna 'Telefono Consolidado'
df_ca_tiroide_telefono = df_ca_tiroide[['N° Telefono 1','N° Telefono 2','N° Telefono 3']]
df_ca_tiroide['Telefono Consolidado2'] = df_ca_tiroide_telefono.apply(combinar_telefonos, axis=1)

# Eliminar las columnas originales de los números de teléfono si ya no son necesarias
df_ca_tiroide.drop(columns=['N° Telefono 1', 'N° Telefono 2', 'N° Telefono 3'], inplace=True)

df_ca_tiroide['Etapa'] = np.nan

df_ca_tiroide['Estado'] = np.nan

df_ca_tiroide['Fecha Proximo Hito'] = np.nan

df_ca_tiroide['Observacion'] = np.nan

df_ca_tiroide['Medico Responsable'] = np.nan

df_ca_tiroide['Fecha Ultima atencion'] = np.nan

df_ca_tiroide['Profesional Ultima atencion'] = np.nan

df_ca_tiroide['MEDICO'] = np.nan

df_ca_tiroide['CITAS FUTURAS'] = np.nan


In [107]:
df_merge[df_merge['_merge']=='left_only']

,Problema Oncologico,RUT,DV,Nombre del Paciente,N° Telefono 1,N° Telefono 2,N° Telefono 3,Fecha de SIC_x,Establecimiento de Origen_x,Especialidad Destino,...,Establecimiento de Origen_y,Observaciones,Diagnosticos,Telefonos,CONCATENADO,CITAS FUTURAS,MEDICO,ESTADO SIC,_df_y,_merge
7,Cáncer Colorectal,3198181,6,"MURÚA NAVARRO, CARLOS DEL CARMEN",- / - / 22823728 / / 76690716 / 22823728 / -/...,#ND,NaN,2024-06-27,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
29,Cáncer Próstata,4280602,1,JORGE LUIS ABARCA GÁLVEZ,979889506,976081118,997783585,2024-06-27,HLF,Cirugía Abdominal Adulto,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
51,Cáncer Colorectal,5192168,2,JUAN RAMÓN SUAZO ARAVENA,981443616,972334059,NaN,2024-06-25,HLF,Cirugía Abdominal Adulto,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
86,Cáncer Colorectal,5869303,0,"RIQUELME PUEBLA, GLORIA MARÍA BERNARD",- / - / 25028287 / 0 / 97158876 / 25028287 / -...,#ND,NaN,2024-07-02,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
100,Cáncer Colorectal,6094724,4,"PIZARRO SERRAT, RICARDO ROLANDO",- / - / / 0 / 93865732 / - / -/9993865732/299...,#ND,NaN,2024-07-02,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
174,Cáncer Pulmón,7540942,7,"VALDEBENITO LOBOS, MARÍA ANGÉLICA",- / - / 0 / / 94364482 / - / -/.,#ND,NaN,2024-06-28,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
203,Cáncer Colorectal,8130920,5,"SILVA HENRÍQUEZ, GINA DE LAS MERCEDES",- / - / 68262953 / 68262953 / 68262953 / 31898...,#ND,NaN,2024-07-03,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
207,Cáncer Pulmón,8317900,7,"MELLI NAVARRETE, ESTER LUCRECIA",- / - / 35766740 / 35766740 / 35766740 / 23140...,FIJO: 23140537 MOVIL: 935766740,NaN,2024-06-28,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
211,Cáncer Colorectal,8402829,0,JUAN ERNESTO AMAYA FAURA,0966644805,0940244902,NaN,2024-06-26,HLF,Urología Adulto,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
223,Cáncer Colorectal,8957749,7,"LEIVA CEPEDA, RODRIGO EDUARDO",- / - / 0 / 0 / 75530673 / / -/75530673//.,FIJO: NO ESPECIFICADO MOVIL: 75530673,NaN,2024-07-02,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [108]:
list(df_ca_tiroide.columns)

['Problema Oncologico',
 'RUT',
 'DV',
 'Nombre del Paciente',
 'Fecha de SIC_x',
 'Establecimiento de Origen_x',
 'Especialidad Destino',
 'Estado de SIC_x',
 'Fecha Proximo Hito',
 'DIAGNOSTICOS',
 'Nombre agenda Medico especialista',
 'Nombre garantia',
 'CODIGOS_DIAGNOSTICO',
 '_df_x',
 'Nombre del Paciente_y',
 'Estado',
 'Fecha Próximo Hito',
 'Medico Responsable',
 'Fecha de SIC_y',
 'Estado de SIC_y',
 'Establecimiento de Origen_y',
 'Observaciones',
 'Diagnosticos',
 'Telefonos',
 'CONCATENADO',
 'CITAS FUTURAS',
 'MEDICO',
 'ESTADO SIC',
 '_df_y',
 '_merge',
 'Telefono Consolidado2',
 'Etapa',
 'Observacion',
 'Fecha Ultima atencion',
 'Profesional Ultima atencion']

In [109]:
list(df_ca_tiroide.columns)

['Problema Oncologico',
 'RUT',
 'DV',
 'Nombre del Paciente',
 'Fecha de SIC_x',
 'Establecimiento de Origen_x',
 'Especialidad Destino',
 'Estado de SIC_x',
 'Fecha Proximo Hito',
 'DIAGNOSTICOS',
 'Nombre agenda Medico especialista',
 'Nombre garantia',
 'CODIGOS_DIAGNOSTICO',
 '_df_x',
 'Nombre del Paciente_y',
 'Estado',
 'Fecha Próximo Hito',
 'Medico Responsable',
 'Fecha de SIC_y',
 'Estado de SIC_y',
 'Establecimiento de Origen_y',
 'Observaciones',
 'Diagnosticos',
 'Telefonos',
 'CONCATENADO',
 'CITAS FUTURAS',
 'MEDICO',
 'ESTADO SIC',
 '_df_y',
 '_merge',
 'Telefono Consolidado2',
 'Etapa',
 'Observacion',
 'Fecha Ultima atencion',
 'Profesional Ultima atencion']

In [110]:

#df_ca_tiroide = df_ca_tiroide[['Problema Oncologico', 'RUT', 'DV', 'Nombre del Paciente','Etapa','Nombre garantia','Estado','Fecha Proximo Hito','Observacion','Medico Responsable','Fecha Ultima atencion', 'Profesional Ultima atencion','DIAGNOSTICOS', 'Telefono Consolidado2']]
df_ca_tiroide = df_ca_tiroide.rename(columns={'Fecha de SIC_x': 'Fecha de SIC'})
df_ca_tiroide = df_ca_tiroide.rename(columns={'Estado de SIC_x': 'Estado de SIC'})
df_ca_tiroide = df_ca_tiroide.rename(columns={'Establecimiento de Origen_x': 'Establecimiento de Origen'})

df_ca_tiroide = df_ca_tiroide[['Problema Oncologico',
 'RUT',
 'DV',
 'Nombre del Paciente',
 'Estado',
 'Fecha Próximo Hito',
 'Medico Responsable',
 'Especialidad Destino',
 'Fecha de SIC',
 'Estado de SIC',
 'Establecimiento de Origen',
 'DIAGNOSTICOS',
 'Observaciones',
 'Telefono Consolidado2',
 'CITAS FUTURAS',
 'MEDICO',
 'ESTADO SIC']]

In [111]:
df_ca_tiroide

,Problema Oncologico,RUT,DV,Nombre del Paciente,Estado,Fecha Próximo Hito,Medico Responsable,Especialidad Destino,Fecha de SIC,Estado de SIC,Establecimiento de Origen,DIAGNOSTICOS,Observaciones,Telefono Consolidado2,CITAS FUTURAS,MEDICO,ESTADO SIC
7,Cáncer Colorectal,3198181,6,"MURÚA NAVARRO, CARLOS DEL CARMEN",NaN,NaN,NaN,NaN,2024-06-27,NaN,NaN,NaN,NaN,- / - / 22823728 / / 76690716 / 22823728 / -/...,NaN,NaN,NaN
29,Cáncer Próstata,4280602,1,JORGE LUIS ABARCA GÁLVEZ,NaN,NaN,NaN,Cirugía Abdominal Adulto,2024-06-27,Pendiente de Autorizar,HLF,Tumor maligno de la prostata,NaN,979889506/976081118/997783585,NaN,NaN,NaN
51,Cáncer Colorectal,5192168,2,JUAN RAMÓN SUAZO ARAVENA,NaN,NaN,NaN,Cirugía Abdominal Adulto,2024-06-25,Pendiente de Autorizar,HLF,"Tumor maligno del colon,CANCER DE COLON",NaN,981443616/972334059/-,NaN,NaN,NaN
86,Cáncer Colorectal,5869303,0,"RIQUELME PUEBLA, GLORIA MARÍA BERNARD",NaN,NaN,NaN,NaN,2024-07-02,NaN,NaN,NaN,NaN,- / - / 25028287 / 0 / 97158876 / 25028287 / -...,NaN,NaN,NaN
100,Cáncer Colorectal,6094724,4,"PIZARRO SERRAT, RICARDO ROLANDO",NaN,NaN,NaN,NaN,2024-07-02,NaN,NaN,NaN,NaN,- / - / / 0 / 93865732 / - / -/9993865732/299...,NaN,NaN,NaN
174,Cáncer Pulmón,7540942,7,"VALDEBENITO LOBOS, MARÍA ANGÉLICA",NaN,NaN,NaN,NaN,2024-06-28,NaN,NaN,NaN,NaN,- / - / 0 / / 94364482 / - / -/./#ND/-,NaN,NaN,NaN
203,Cáncer Colorectal,8130920,5,"SILVA HENRÍQUEZ, GINA DE LAS MERCEDES",NaN,NaN,NaN,NaN,2024-07-03,NaN,NaN,NaN,NaN,- / - / 68262953 / 68262953 / 68262953 / 31898...,NaN,NaN,NaN
207,Cáncer Pulmón,8317900,7,"MELLI NAVARRETE, ESTER LUCRECIA",NaN,NaN,NaN,NaN,2024-06-28,NaN,NaN,NaN,NaN,- / - / 35766740 / 35766740 / 35766740 / 23140...,NaN,NaN,NaN
211,Cáncer Colorectal,8402829,0,JUAN ERNESTO AMAYA FAURA,NaN,NaN,NaN,Urología Adulto,2024-06-26,Pendiente de Autorizar,HLF,"Tumor maligno del recto,CANCER DE RECTO QUE TE...",NaN,0966644805/0940244902/-,NaN,NaN,NaN
223,Cáncer Colorectal,8957749,7,"LEIVA CEPEDA, RODRIGO EDUARDO",NaN,NaN,NaN,NaN,2024-07-02,NaN,NaN,NaN,NaN,- / - / 0 / 0 / 75530673 / / -/75530673//./FI...,NaN,NaN,NaN


## Deprecado

In [112]:
# Guardar el DataFrame en un archivo de Excel

# Día actual
today = date.today()

# Fecha actual
now = datetime.now()

# Formatear día con ceros a la izquierda si es necesario
day_formatted = "{:02d}".format(today.day)

# Formatear mes con ceros a la izquierda si es necesario
month_formatted = "{:02d}".format(today.month)


nombre_archivo = f'{today.year}{month_formatted}{day_formatted}_derivaciones_nuevas.xlsx'  # Nombre del archivo de Excel
#df_ca_tiroide.to_excel(os.path.join(dir_actual, '..','DERIVACIONES','Reporte diario casos nuevos',f'{nombre_archivo}'), index=False)  # index=False para no incluir el índice en el archivo de Excel 

# Convertir la columna 'RUT' en DataFrame_2 a tipo object si es necesario


df_concatenated_2 = pd.concat([dataframe_2,df_ca_tiroide])

#print(list[df_concatenated_2.columns])




df_concatenated_2['RUT'] = df_concatenated_2['RUT'].astype(str)

In [113]:
df_ca_tiroide

,Problema Oncologico,RUT,DV,Nombre del Paciente,Estado,Fecha Próximo Hito,Medico Responsable,Especialidad Destino,Fecha de SIC,Estado de SIC,Establecimiento de Origen,DIAGNOSTICOS,Observaciones,Telefono Consolidado2,CITAS FUTURAS,MEDICO,ESTADO SIC
7,Cáncer Colorectal,3198181,6,"MURÚA NAVARRO, CARLOS DEL CARMEN",NaN,NaN,NaN,NaN,2024-06-27,NaN,NaN,NaN,NaN,- / - / 22823728 / / 76690716 / 22823728 / -/...,NaN,NaN,NaN
29,Cáncer Próstata,4280602,1,JORGE LUIS ABARCA GÁLVEZ,NaN,NaN,NaN,Cirugía Abdominal Adulto,2024-06-27,Pendiente de Autorizar,HLF,Tumor maligno de la prostata,NaN,979889506/976081118/997783585,NaN,NaN,NaN
51,Cáncer Colorectal,5192168,2,JUAN RAMÓN SUAZO ARAVENA,NaN,NaN,NaN,Cirugía Abdominal Adulto,2024-06-25,Pendiente de Autorizar,HLF,"Tumor maligno del colon,CANCER DE COLON",NaN,981443616/972334059/-,NaN,NaN,NaN
86,Cáncer Colorectal,5869303,0,"RIQUELME PUEBLA, GLORIA MARÍA BERNARD",NaN,NaN,NaN,NaN,2024-07-02,NaN,NaN,NaN,NaN,- / - / 25028287 / 0 / 97158876 / 25028287 / -...,NaN,NaN,NaN
100,Cáncer Colorectal,6094724,4,"PIZARRO SERRAT, RICARDO ROLANDO",NaN,NaN,NaN,NaN,2024-07-02,NaN,NaN,NaN,NaN,- / - / / 0 / 93865732 / - / -/9993865732/299...,NaN,NaN,NaN
174,Cáncer Pulmón,7540942,7,"VALDEBENITO LOBOS, MARÍA ANGÉLICA",NaN,NaN,NaN,NaN,2024-06-28,NaN,NaN,NaN,NaN,- / - / 0 / / 94364482 / - / -/./#ND/-,NaN,NaN,NaN
203,Cáncer Colorectal,8130920,5,"SILVA HENRÍQUEZ, GINA DE LAS MERCEDES",NaN,NaN,NaN,NaN,2024-07-03,NaN,NaN,NaN,NaN,- / - / 68262953 / 68262953 / 68262953 / 31898...,NaN,NaN,NaN
207,Cáncer Pulmón,8317900,7,"MELLI NAVARRETE, ESTER LUCRECIA",NaN,NaN,NaN,NaN,2024-06-28,NaN,NaN,NaN,NaN,- / - / 35766740 / 35766740 / 35766740 / 23140...,NaN,NaN,NaN
211,Cáncer Colorectal,8402829,0,JUAN ERNESTO AMAYA FAURA,NaN,NaN,NaN,Urología Adulto,2024-06-26,Pendiente de Autorizar,HLF,"Tumor maligno del recto,CANCER DE RECTO QUE TE...",NaN,0966644805/0940244902/-,NaN,NaN,NaN
223,Cáncer Colorectal,8957749,7,"LEIVA CEPEDA, RODRIGO EDUARDO",NaN,NaN,NaN,NaN,2024-07-02,NaN,NaN,NaN,NaN,- / - / 0 / 0 / 75530673 / / -/75530673//./FI...,NaN,NaN,NaN


# Fecha Agenda Futura HLF

In [114]:

# Día actual
today = date.today()

# Fecha actual
now = datetime.now()

# Formatear día con ceros a la izquierda si es necesario
day_formatted = "{:02d}".format(today.day)

# Formatear mes con ceros a la izquierda si es necesario
month_formatted = "{:02d}".format(today.month)



ruta_del_archivo = os.path.join(ruta_reporteria,'Reportes_Ambulatorio','AgendasFuturas', f'{today.year}{month_formatted}{day_formatted}_Citas_Futuras.csv')


# Carga el archivo XLSX en un DataFrame
dataframe_fu = cargar_dataframe_ambulatorio(ruta_del_archivo)


# Mover los nombres de las columnas una posición hacia la izquierda
new_columns = list(dataframe_fu.columns)
new_columns = new_columns[1:] + [new_columns[0]]
dataframe_fu.columns = new_columns


# Eliminar duplicados por índice y mantener el primero
df_sin_duplicados = dataframe_fu.drop_duplicates(keep='first')

# Fecha Agenda Futura Local

In [115]:
#dir_actual = os.getcwd()

# Construir la ruta relativa al directorio donde se encuentra el archivo CSV
#dir_archivo_csv = os.path.join(dir_actual, '..','..')

# Entrar a la carpeta "Datos HLF"
#dir_datos_hlf = os.path.join(dir_archivo_csv, 'Datos HLF')



#ruta_del_archivo = os.path.join(dir_datos_hlf, '20240502_Citas_Futuras.csv')


# Carga el archivo XLSX en un DataFrame
#dataframe_fu = cargar_dataframe_ambulatorio(ruta_del_archivo)

# Mover los nombres de las columnas una posición hacia la izquierda
#new_columns = list(dataframe_fu.columns)
#new_columns = new_columns[1:] + [new_columns[0]]
#dataframe_fu.columns = new_columns


# Eliminar duplicados por índice y mantener el primero
#df_sin_duplicados = dataframe_fu.drop_duplicates(keep='first')



In [116]:
df_sin_duplicados['ESPECIALIDAD_AGENDA'].unique()

array(['Oftalmología', 'Enfermería', 'TACO', 'Medicina Interna',
       'Urología Adulto', 'Enfermería Multiespecialidad',
       'Broncopulmonar Adulto', 'Cirugía Adulto', 'Imagenología AMB',
       'Kinesiología MFR', 'Unidad Alivio Dolor y Cuidados Paliativos',
       'Traumatología Adulto', 'Med. Física y Rehab. Adultos',
       'Medicina Familiar', 'Ginecología Adulto',
       'Gastroenterología Adulto', 'Rehabilitación: Prótesis Removible',
       'Radiografías Dentales', 'Otorrinolaringología',
       'Anestesiología Adulto', 'Hospital de Día', 'Cardiología Adulto',
       'Hematología Adulto', 'Diabetes', 'Dermatología Adulto',
       'Psiquiatría Adulto', 'T. Temporomandibulares y Dolor Orofacial',
       'Cirugía de mamas', 'Nutricionista Adulto',
       'Unidad de Patología Mamaria', 'Servicio Social',
       'Nefrología Adulto', 'Periodoncia', 'Unidad de Patología Cervical',
       'Fonoaudiología MFR', 'Alto Riesgo Obstétrico',
       'Inmunología Adulto', 'Hospital de Día

In [117]:
regex_esp = re.compile(r'(?i)\bimagenolog[ií]a|\bbroncopulmonar|\bcirug[íi]a.*(abdominal|adulto|cabeza.*cuello|t[óo]rax|digestiva|general)|coloproctolog[íi]a|dermatolog[íi]a|endocrinolog[íi]a|gastroenterolog[íi]a|hematolog[íi]a|medicina.*interna|traumatolog[íi]a|urolog[íi]a', re.IGNORECASE)


endocrinologia_df_fu = df_sin_duplicados[df_sin_duplicados['ESPECIALIDAD_AGENDA'].str.contains(regex_esp, na=False)]


    # Filtrar las filas que no contienen 'No realizada' o 'Anulada' en la columna 'ESTADO_CITA'
endocrinologia_df_fu = endocrinologia_df_fu[~endocrinologia_df_fu['ESTADO_CITA'].isin(['Anulada', 'En buzón de reasignación'])]

endocrinologia_df_fu['DV'] = endocrinologia_df_fu['DOCUMENTO'].str[-1]

endocrinologia_df_fu['RUT'] = endocrinologia_df_fu['DOCUMENTO'].str[:-1]

endocrinologia_df_fu = endocrinologia_df_fu[endocrinologia_df_fu['NOMBRE_TIPO_DOCUMENTO']=='RUN']

endocrinologia_df_fu = endocrinologia_df_fu[endocrinologia_df_fu['NOMBRE_AGENDA'].str.contains(r'\bDR.')]

In [118]:
endocrinologia_df_fu['DESCRIPCION'].value_counts()

DESCRIPCION
Control - Consulta integral de especialidades    4360
Derivacion desde APS                              729
Control postquirurgico                            403
Interconsulta                                     399
CONSULTA ABREVIADA-RECETA                         266
CONSULTA ABREVIADA-LICENCIA MEDICA                 87
Ingreso Traumatología Poli de Choque AMB           38
CONSULTA ABREVIADA-CERTIFICADO MEDICO              33
Ingreso Médico  (transitorio)                      25
Control Sueño                                      19
Crioterapia hasta 5 lesiones AMB                    9
Crioterapia 6 -10 lesiones AMB                      1
Control Post Alta                                   1
Name: count, dtype: int64

In [119]:
#endocrinologia_df_fu = endocrinologia_df_fu[endocrinologia_df_fu['DESCRIPCION'].isin(['Derivacion desde APS', 'Interconsulta', 'Ingreso Médico  (transitorio)','Ingreso Traumatología Poli de Choque AMB', 'Ingreso Médico no programado'])]

In [120]:
endocrinologia_df_fu['DESCRIPCION'].value_counts()

DESCRIPCION
Control - Consulta integral de especialidades    4360
Derivacion desde APS                              729
Control postquirurgico                            403
Interconsulta                                     399
CONSULTA ABREVIADA-RECETA                         266
CONSULTA ABREVIADA-LICENCIA MEDICA                 87
Ingreso Traumatología Poli de Choque AMB           38
CONSULTA ABREVIADA-CERTIFICADO MEDICO              33
Ingreso Médico  (transitorio)                      25
Control Sueño                                      19
Crioterapia hasta 5 lesiones AMB                    9
Crioterapia 6 -10 lesiones AMB                      1
Control Post Alta                                   1
Name: count, dtype: int64

In [121]:
list(endocrinologia_df_fu.columns)

['NOMBRE_AGENDA',
 'ESPECIALIDAD_AGENDA',
 'CITAFECH',
 'CITAHORA',
 'ESTADO_CITA',
 'DESCRIPCION',
 'ORIGEN',
 'GES_CITA',
 'GES_SOL',
 'PATOLOGIA_GES_SOL',
 'PACIENTE',
 'CITA_CODUTEN',
 'DOCUMENTO',
 'NOMBRE_TIPO_DOCUMENTO',
 'NUM_DOCUMENTO_TUTOR',
 'NOMBRE_DOCUMENTO_TUTOR',
 'CITAPACIFECHNACI',
 'SEXO',
 'NACIONALIDAD',
 'FACT_FINANCIADOR',
 'FACT_COBERTURA',
 'GES_EPISODIO',
 'MOVIL_HABITUAL',
 'TELEFONO1_HABITUAL',
 'TELEFONO2_HABITUAL',
 'TELEFONO3_HABITUALDIRECCION_HABITUAL',
 'COMUNA_HABITUAL',
 'MOVIL_CONTACTO',
 'TELEFONO1_CONTACTO',
 'TELEFONO2_CONTACTO',
 'TELEFONO3_CONTACTO',
 'CITA_KEY',
 'DV',
 'RUT']

In [122]:
endocrinologia_df_fu[['ESPECIALIDAD_AGENDA','RUT']]

,ESPECIALIDAD_AGENDA,RUT
1714736932004310243,Medicina Interna,3747816
1717780731277500026,Cirugía Adulto,3681341
1713879964147300042,Urología Adulto,3460934
1718986686055510263,Traumatología Adulto,5094335
1718820660828510377,Cirugía Adulto,4989061
...,...,...
1711637800965910740,Urología Adulto,5812444
1713888455873300835,Urología Adulto,26819044
1711570784927900514,Urología Adulto,3727939
1711981070379900773,Urología Adulto,7893201


In [123]:
endocrinologia_df_fu['ESPECIALIDAD_AGENDA'].unique()

array(['Medicina Interna', 'Cirugía Adulto', 'Urología Adulto',
       'Traumatología Adulto', 'Gastroenterología Adulto',
       'Hematología Adulto', 'Broncopulmonar Infantil',
       'Traumatología Infantil', 'Cirugía Tórax Adulto',
       'Cirugia Cabeza y Cuello', 'Endocrinología Adulto',
       'Dermatología Adulto', 'Neurología Adulto',
       'Cirugía Abdominal Adulto', 'Broncopulmonar Adulto',
       'Gastroenterología Infantil', 'Neurología Infantil',
       'Endocrinología Infantil', 'Hematología Infantil',
       'Cirugía Plástica Adulto'], dtype=object)

In [124]:
endocrinologia_df_fu = endocrinologia_df_fu[['RUT','NOMBRE_AGENDA','ESPECIALIDAD_AGENDA','CITAFECH','CITAHORA','ESTADO_CITA','DESCRIPCION','ORIGEN','GES_CITA','GES_SOL','PATOLOGIA_GES_SOL','PACIENTE','CITA_CODUTEN','DOCUMENTO']]

In [125]:
df_concatenated_2

,Problema Oncologico,RUT,DV,Nombre del Paciente,Estado,Fecha Próximo Hito,Medico Responsable,Especialidad,Fecha de SIC,Estado de SIC,...,Observaciones,Diagnosticos,Telefonos,CONCATENADO,CITAS FUTURAS,MEDICO,ESTADO SIC,Especialidad Destino,DIAGNOSTICOS,Telefono Consolidado2
0,Cáncer Próstata,5926301,3,ADA VILCHES MORALES,Priorizacion CUD,NaN,NaN,Urología Adulto,2024-05-13,NaN,...,NaN,NaN,NaN,5926301Urología Adulto,NaN,NaN,Solicitada,NaN,NaN,NaN
1,Cáncer Tiroides,16698412,2,Canales Marchant Nicole Paulina,Priorizacion CUD,NaN,NaN,Endocrinología Adulto,2024-05-08,NaN,...,NaN,NaN,NaN,16698412Endocrinología Adulto,NaN,NaN,Finalizada,NaN,NaN,NaN
2,Cáncer Próstata,99616630,K,CARLOS AGUILAR SALAS,Priorizacion CUD,NaN,NaN,Urología Adulto,2024-05-02,NaN,...,NaN,NaN,NaN,99616630Urología Adulto,NaN,NaN,NaN,NaN,NaN,NaN
3,Cáncer Pulmón,7408006,5,Cuevas Villalobos Eulalia Iris,Rechazada,NaN,NaN,Broncopulmonar Adulto,2024-06-03,rechazada,...,no pertinente (cud),NaN,NaN,7408006Broncopulmonar Adulto,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,7262387,8,Cuevas Villalobos Rene Edmundo,Priorizacion CUD,NaN,NaN,Dermatología Adulto,2024-06-07,NaN,...,NaN,NaN,NaN,7262387DERMATOLOGIA,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,Cáncer Colorectal,8957749,7,"LEIVA CEPEDA, RODRIGO EDUARDO",NaN,NaN,NaN,NaN,2024-07-02,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,- / - / 0 / 0 / 75530673 / / -/75530673//./FI...
250,Cáncer Colorectal,10168476,8,LUIS ALBERTO ÁLVAREZ PINO,NaN,NaN,NaN,NaN,2024-06-26,Pendiente de Autorizar,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cirugia Cabeza y Cuello,"SINDROME DE RESECCION ANTERIOR,Tumor maligno d...",96271467/-/-
251,Cáncer Colorectal,10168476,8,LUIS ALBERTO ÁLVAREZ PINO,NaN,NaN,NaN,NaN,2024-06-26,Pendiente de Autorizar,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Endocrinología Adulto,"SINDROME DE RESECCION ANTERIOR,Tumor maligno d...",96271467/-/-
258,Cáncer Gástrico,10492252,K,"VILCHES PÉREZ, MARISA DE LAS MERCED",NaN,NaN,NaN,NaN,2024-06-28,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,- / - / 22828026 / 66931649 / 66931649 / 22828...


In [126]:
df_concatenated_2['Problema Oncologico'].unique()

array(['Cáncer Próstata', 'Cáncer Tiroides', 'Cáncer Pulmón', nan,
       'Cáncer Gástrico', 'Cáncer colorectal', 'Tumor maligno de piel',
       'Cáncer Renal', 'Mieloma', 'Linfoma', 'Cáncer Testículo',
       'Cáncer Gastrico ', 'Cáncer Vesical', 'Leucemia',
       'Cáncer Colorectal'], dtype=object)

In [127]:
df_concatenated_2

,Problema Oncologico,RUT,DV,Nombre del Paciente,Estado,Fecha Próximo Hito,Medico Responsable,Especialidad,Fecha de SIC,Estado de SIC,...,Observaciones,Diagnosticos,Telefonos,CONCATENADO,CITAS FUTURAS,MEDICO,ESTADO SIC,Especialidad Destino,DIAGNOSTICOS,Telefono Consolidado2
0,Cáncer Próstata,5926301,3,ADA VILCHES MORALES,Priorizacion CUD,NaN,NaN,Urología Adulto,2024-05-13,NaN,...,NaN,NaN,NaN,5926301Urología Adulto,NaN,NaN,Solicitada,NaN,NaN,NaN
1,Cáncer Tiroides,16698412,2,Canales Marchant Nicole Paulina,Priorizacion CUD,NaN,NaN,Endocrinología Adulto,2024-05-08,NaN,...,NaN,NaN,NaN,16698412Endocrinología Adulto,NaN,NaN,Finalizada,NaN,NaN,NaN
2,Cáncer Próstata,99616630,K,CARLOS AGUILAR SALAS,Priorizacion CUD,NaN,NaN,Urología Adulto,2024-05-02,NaN,...,NaN,NaN,NaN,99616630Urología Adulto,NaN,NaN,NaN,NaN,NaN,NaN
3,Cáncer Pulmón,7408006,5,Cuevas Villalobos Eulalia Iris,Rechazada,NaN,NaN,Broncopulmonar Adulto,2024-06-03,rechazada,...,no pertinente (cud),NaN,NaN,7408006Broncopulmonar Adulto,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,7262387,8,Cuevas Villalobos Rene Edmundo,Priorizacion CUD,NaN,NaN,Dermatología Adulto,2024-06-07,NaN,...,NaN,NaN,NaN,7262387DERMATOLOGIA,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,Cáncer Colorectal,8957749,7,"LEIVA CEPEDA, RODRIGO EDUARDO",NaN,NaN,NaN,NaN,2024-07-02,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,- / - / 0 / 0 / 75530673 / / -/75530673//./FI...
250,Cáncer Colorectal,10168476,8,LUIS ALBERTO ÁLVAREZ PINO,NaN,NaN,NaN,NaN,2024-06-26,Pendiente de Autorizar,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cirugia Cabeza y Cuello,"SINDROME DE RESECCION ANTERIOR,Tumor maligno d...",96271467/-/-
251,Cáncer Colorectal,10168476,8,LUIS ALBERTO ÁLVAREZ PINO,NaN,NaN,NaN,NaN,2024-06-26,Pendiente de Autorizar,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Endocrinología Adulto,"SINDROME DE RESECCION ANTERIOR,Tumor maligno d...",96271467/-/-
258,Cáncer Gástrico,10492252,K,"VILCHES PÉREZ, MARISA DE LAS MERCED",NaN,NaN,NaN,NaN,2024-06-28,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,- / - / 22828026 / 66931649 / 66931649 / 22828...


In [128]:
df_ca_tiroide

,Problema Oncologico,RUT,DV,Nombre del Paciente,Estado,Fecha Próximo Hito,Medico Responsable,Especialidad Destino,Fecha de SIC,Estado de SIC,Establecimiento de Origen,DIAGNOSTICOS,Observaciones,Telefono Consolidado2,CITAS FUTURAS,MEDICO,ESTADO SIC
7,Cáncer Colorectal,3198181,6,"MURÚA NAVARRO, CARLOS DEL CARMEN",NaN,NaN,NaN,NaN,2024-06-27,NaN,NaN,NaN,NaN,- / - / 22823728 / / 76690716 / 22823728 / -/...,NaN,NaN,NaN
29,Cáncer Próstata,4280602,1,JORGE LUIS ABARCA GÁLVEZ,NaN,NaN,NaN,Cirugía Abdominal Adulto,2024-06-27,Pendiente de Autorizar,HLF,Tumor maligno de la prostata,NaN,979889506/976081118/997783585,NaN,NaN,NaN
51,Cáncer Colorectal,5192168,2,JUAN RAMÓN SUAZO ARAVENA,NaN,NaN,NaN,Cirugía Abdominal Adulto,2024-06-25,Pendiente de Autorizar,HLF,"Tumor maligno del colon,CANCER DE COLON",NaN,981443616/972334059/-,NaN,NaN,NaN
86,Cáncer Colorectal,5869303,0,"RIQUELME PUEBLA, GLORIA MARÍA BERNARD",NaN,NaN,NaN,NaN,2024-07-02,NaN,NaN,NaN,NaN,- / - / 25028287 / 0 / 97158876 / 25028287 / -...,NaN,NaN,NaN
100,Cáncer Colorectal,6094724,4,"PIZARRO SERRAT, RICARDO ROLANDO",NaN,NaN,NaN,NaN,2024-07-02,NaN,NaN,NaN,NaN,- / - / / 0 / 93865732 / - / -/9993865732/299...,NaN,NaN,NaN
174,Cáncer Pulmón,7540942,7,"VALDEBENITO LOBOS, MARÍA ANGÉLICA",NaN,NaN,NaN,NaN,2024-06-28,NaN,NaN,NaN,NaN,- / - / 0 / / 94364482 / - / -/./#ND/-,NaN,NaN,NaN
203,Cáncer Colorectal,8130920,5,"SILVA HENRÍQUEZ, GINA DE LAS MERCEDES",NaN,NaN,NaN,NaN,2024-07-03,NaN,NaN,NaN,NaN,- / - / 68262953 / 68262953 / 68262953 / 31898...,NaN,NaN,NaN
207,Cáncer Pulmón,8317900,7,"MELLI NAVARRETE, ESTER LUCRECIA",NaN,NaN,NaN,NaN,2024-06-28,NaN,NaN,NaN,NaN,- / - / 35766740 / 35766740 / 35766740 / 23140...,NaN,NaN,NaN
211,Cáncer Colorectal,8402829,0,JUAN ERNESTO AMAYA FAURA,NaN,NaN,NaN,Urología Adulto,2024-06-26,Pendiente de Autorizar,HLF,"Tumor maligno del recto,CANCER DE RECTO QUE TE...",NaN,0966644805/0940244902/-,NaN,NaN,NaN
223,Cáncer Colorectal,8957749,7,"LEIVA CEPEDA, RODRIGO EDUARDO",NaN,NaN,NaN,NaN,2024-07-02,NaN,NaN,NaN,NaN,- / - / 0 / 0 / 75530673 / / -/75530673//./FI...,NaN,NaN,NaN


In [129]:
df_ca_tiroide

,Problema Oncologico,RUT,DV,Nombre del Paciente,Estado,Fecha Próximo Hito,Medico Responsable,Especialidad Destino,Fecha de SIC,Estado de SIC,Establecimiento de Origen,DIAGNOSTICOS,Observaciones,Telefono Consolidado2,CITAS FUTURAS,MEDICO,ESTADO SIC
7,Cáncer Colorectal,3198181,6,"MURÚA NAVARRO, CARLOS DEL CARMEN",NaN,NaN,NaN,NaN,2024-06-27,NaN,NaN,NaN,NaN,- / - / 22823728 / / 76690716 / 22823728 / -/...,NaN,NaN,NaN
29,Cáncer Próstata,4280602,1,JORGE LUIS ABARCA GÁLVEZ,NaN,NaN,NaN,Cirugía Abdominal Adulto,2024-06-27,Pendiente de Autorizar,HLF,Tumor maligno de la prostata,NaN,979889506/976081118/997783585,NaN,NaN,NaN
51,Cáncer Colorectal,5192168,2,JUAN RAMÓN SUAZO ARAVENA,NaN,NaN,NaN,Cirugía Abdominal Adulto,2024-06-25,Pendiente de Autorizar,HLF,"Tumor maligno del colon,CANCER DE COLON",NaN,981443616/972334059/-,NaN,NaN,NaN
86,Cáncer Colorectal,5869303,0,"RIQUELME PUEBLA, GLORIA MARÍA BERNARD",NaN,NaN,NaN,NaN,2024-07-02,NaN,NaN,NaN,NaN,- / - / 25028287 / 0 / 97158876 / 25028287 / -...,NaN,NaN,NaN
100,Cáncer Colorectal,6094724,4,"PIZARRO SERRAT, RICARDO ROLANDO",NaN,NaN,NaN,NaN,2024-07-02,NaN,NaN,NaN,NaN,- / - / / 0 / 93865732 / - / -/9993865732/299...,NaN,NaN,NaN
174,Cáncer Pulmón,7540942,7,"VALDEBENITO LOBOS, MARÍA ANGÉLICA",NaN,NaN,NaN,NaN,2024-06-28,NaN,NaN,NaN,NaN,- / - / 0 / / 94364482 / - / -/./#ND/-,NaN,NaN,NaN
203,Cáncer Colorectal,8130920,5,"SILVA HENRÍQUEZ, GINA DE LAS MERCEDES",NaN,NaN,NaN,NaN,2024-07-03,NaN,NaN,NaN,NaN,- / - / 68262953 / 68262953 / 68262953 / 31898...,NaN,NaN,NaN
207,Cáncer Pulmón,8317900,7,"MELLI NAVARRETE, ESTER LUCRECIA",NaN,NaN,NaN,NaN,2024-06-28,NaN,NaN,NaN,NaN,- / - / 35766740 / 35766740 / 35766740 / 23140...,NaN,NaN,NaN
211,Cáncer Colorectal,8402829,0,JUAN ERNESTO AMAYA FAURA,NaN,NaN,NaN,Urología Adulto,2024-06-26,Pendiente de Autorizar,HLF,"Tumor maligno del recto,CANCER DE RECTO QUE TE...",NaN,0966644805/0940244902/-,NaN,NaN,NaN
223,Cáncer Colorectal,8957749,7,"LEIVA CEPEDA, RODRIGO EDUARDO",NaN,NaN,NaN,NaN,2024-07-02,NaN,NaN,NaN,NaN,- / - / 0 / 0 / 75530673 / / -/75530673//./FI...,NaN,NaN,NaN


In [130]:
pacientes_ges = df_ca_tiroide[df_ca_tiroide['Especialidad Destino'].isnull()]

In [131]:
pacientes_ges

,Problema Oncologico,RUT,DV,Nombre del Paciente,Estado,Fecha Próximo Hito,Medico Responsable,Especialidad Destino,Fecha de SIC,Estado de SIC,Establecimiento de Origen,DIAGNOSTICOS,Observaciones,Telefono Consolidado2,CITAS FUTURAS,MEDICO,ESTADO SIC
7,Cáncer Colorectal,3198181,6,"MURÚA NAVARRO, CARLOS DEL CARMEN",NaN,NaN,NaN,NaN,2024-06-27,NaN,NaN,NaN,NaN,- / - / 22823728 / / 76690716 / 22823728 / -/...,NaN,NaN,NaN
86,Cáncer Colorectal,5869303,0,"RIQUELME PUEBLA, GLORIA MARÍA BERNARD",NaN,NaN,NaN,NaN,2024-07-02,NaN,NaN,NaN,NaN,- / - / 25028287 / 0 / 97158876 / 25028287 / -...,NaN,NaN,NaN
100,Cáncer Colorectal,6094724,4,"PIZARRO SERRAT, RICARDO ROLANDO",NaN,NaN,NaN,NaN,2024-07-02,NaN,NaN,NaN,NaN,- / - / / 0 / 93865732 / - / -/9993865732/299...,NaN,NaN,NaN
174,Cáncer Pulmón,7540942,7,"VALDEBENITO LOBOS, MARÍA ANGÉLICA",NaN,NaN,NaN,NaN,2024-06-28,NaN,NaN,NaN,NaN,- / - / 0 / / 94364482 / - / -/./#ND/-,NaN,NaN,NaN
203,Cáncer Colorectal,8130920,5,"SILVA HENRÍQUEZ, GINA DE LAS MERCEDES",NaN,NaN,NaN,NaN,2024-07-03,NaN,NaN,NaN,NaN,- / - / 68262953 / 68262953 / 68262953 / 31898...,NaN,NaN,NaN
207,Cáncer Pulmón,8317900,7,"MELLI NAVARRETE, ESTER LUCRECIA",NaN,NaN,NaN,NaN,2024-06-28,NaN,NaN,NaN,NaN,- / - / 35766740 / 35766740 / 35766740 / 23140...,NaN,NaN,NaN
223,Cáncer Colorectal,8957749,7,"LEIVA CEPEDA, RODRIGO EDUARDO",NaN,NaN,NaN,NaN,2024-07-02,NaN,NaN,NaN,NaN,- / - / 0 / 0 / 75530673 / / -/75530673//./FI...,NaN,NaN,NaN
258,Cáncer Gástrico,10492252,K,"VILCHES PÉREZ, MARISA DE LAS MERCED",NaN,NaN,NaN,NaN,2024-06-28,NaN,NaN,NaN,NaN,- / - / 22828026 / 66931649 / 66931649 / 22828...,NaN,NaN,NaN
272,Cáncer Colorectal,11667136,0,"FERNÁNDEZ SILVA, IVÁN BLADIMIR",NaN,NaN,NaN,NaN,2024-07-03,NaN,NaN,NaN,NaN,- / - / 22620004 / 98769960 / - / 22620004 / 9...,NaN,NaN,NaN


In [132]:
import pandas as pd
import re



# Mapeo de enfermedades a expresiones regulares de especialidades
mapeo_especialidades = {
    'Cáncer Colorectal': re.compile(r'coloproctolog[í]a|proctol[óo]gica', re.IGNORECASE),
    'Mieloma': re.compile(r'Hematolog[ií]a', re.IGNORECASE),
    'Leucemia': re.compile(r'Hematolog[ií]a', re.IGNORECASE),
    'Linfoma': re.compile(r'Hematolog[ií]a', re.IGNORECASE),
    'Cáncer Gástrico': re.compile(r'(medicina.*interna|cirug[ií]a.*abdominal)', re.IGNORECASE),
    'Cáncer Tiroides': re.compile(r'(cirug[ií]a.*cabeza.*cuello|endocrinolog[íi]a.*adulto)', re.IGNORECASE),
    'Cáncer Pulmón': re.compile(r'(broncopulmonar|cirug[ií]a.*t[óo]rax)', re.IGNORECASE),
    'Cáncer Próstata': re.compile(r'urolog[íi]a', re.IGNORECASE),
    'Cáncer Testículo': re.compile(r'urolog[íi]a', re.IGNORECASE),
    'Cáncer Vesical': re.compile(r'urolog[íi]a', re.IGNORECASE),
    'Cáncer Renal': re.compile(r'urolog[íi]a', re.IGNORECASE),
}

# Filtrar y mergear
df_merged = pd.merge(
    df_ca_tiroide,
    df_ca_tiroide,
    on='RUT',
    how='inner'
)

In [133]:
df_merged[['Problema Oncologico_x','Especialidad Destino_y']]

,Problema Oncologico_x,Especialidad Destino_y
0,Cáncer Colorectal,NaN
1,Cáncer Próstata,Cirugía Abdominal Adulto
2,Cáncer Colorectal,Cirugía Abdominal Adulto
3,Cáncer Colorectal,NaN
4,Cáncer Colorectal,NaN
5,Cáncer Pulmón,NaN
6,Cáncer Colorectal,NaN
7,Cáncer Pulmón,NaN
8,Cáncer Colorectal,Urología Adulto
9,Cáncer Colorectal,NaN


In [134]:
df_merged['Problema Oncologico_x'] = df_merged['Problema Oncologico_x'].astype(str)
df_merged['Especialidad Destino_y'] = df_merged['Especialidad Destino_y'].astype(str)

In [135]:
# Filtrar el dataframe mergeado para que cumpla las condiciones de mapeo
def filtrar_por_especialidad(row):
    pattern = mapeo_especialidades.get(row['Problema Oncologico_x'])
    
    if pattern:
        return bool(pattern.search(row['Especialidad Destino_y']))
    return False




df_merged_filtrado_todo = df_merged[df_merged.apply(filtrar_por_especialidad, axis=1)]
df_merged_filtrado_todo = df_merged_filtrado_todo.rename(columns={'DV_x': 'DV'})

In [136]:
df_merged_filtrado_todo

,Problema Oncologico_x,RUT,DV,Nombre del Paciente_x,Estado_x,Fecha Próximo Hito_x,Medico Responsable_x,Especialidad Destino_x,Fecha de SIC_x,Estado de SIC_x,...,Especialidad Destino_y,Fecha de SIC_y,Estado de SIC_y,Establecimiento de Origen_y,DIAGNOSTICOS_y,Observaciones_y,Telefono Consolidado2_y,CITAS FUTURAS_y,MEDICO_y,ESTADO SIC_y


In [137]:
df_ca_tiroide['Problema Oncologico'] = df_ca_tiroide['Problema Oncologico'].astype(str)
df_ca_tiroide['Especialidad Destino'] = df_ca_tiroide['Especialidad Destino'].astype(str)

# Filtrar el dataframe mergeado para que cumpla las condiciones de mapeo
def filtrar_por_especialidad_otra(row):
    pattern = mapeo_especialidades.get(row['Problema Oncologico'])
    if pattern:
        return bool(pattern.search(row['Especialidad Destino']))
    return False




df_merged_filtrado_todo = df_ca_tiroide[df_ca_tiroide.apply(filtrar_por_especialidad_otra, axis=1)]


In [138]:
df_merged_filtrado_todo

,Problema Oncologico,RUT,DV,Nombre del Paciente,Estado,Fecha Próximo Hito,Medico Responsable,Especialidad Destino,Fecha de SIC,Estado de SIC,Establecimiento de Origen,DIAGNOSTICOS,Observaciones,Telefono Consolidado2,CITAS FUTURAS,MEDICO,ESTADO SIC


In [139]:
df_merged_filtrado_todo = df_merged_filtrado_todo.drop_duplicates(subset=['Problema Oncologico',
 'RUT',
 'Fecha de SIC',
 'Especialidad Destino'
 ], keep= 'first')

In [140]:
df_merged_filtrado_todo

,Problema Oncologico,RUT,DV,Nombre del Paciente,Estado,Fecha Próximo Hito,Medico Responsable,Especialidad Destino,Fecha de SIC,Estado de SIC,Establecimiento de Origen,DIAGNOSTICOS,Observaciones,Telefono Consolidado2,CITAS FUTURAS,MEDICO,ESTADO SIC


In [141]:
df_merged_filtrado_todo[df_merged_filtrado_todo['Estado de SIC'].isin(['Solicitada','Pendiente de Autorizar', 'Citada'])]


,Problema Oncologico,RUT,DV,Nombre del Paciente,Estado,Fecha Próximo Hito,Medico Responsable,Especialidad Destino,Fecha de SIC,Estado de SIC,Establecimiento de Origen,DIAGNOSTICOS,Observaciones,Telefono Consolidado2,CITAS FUTURAS,MEDICO,ESTADO SIC


In [142]:
df_pacientes_nuevos_activas = df_merged_filtrado_todo[df_merged_filtrado_todo['Estado de SIC'].isin(['Solicitada','Pendiente de Autorizar', 'Citada'])][['Problema Oncologico',
 'RUT',
 'DV',
 'Nombre del Paciente',
 'Estado',
 'Fecha Próximo Hito',
 'Medico Responsable',
 'Especialidad Destino',
 'Fecha de SIC',
 'Estado de SIC',
 'Establecimiento de Origen',
 'DIAGNOSTICOS',
 'Observaciones',
 'Telefono Consolidado2']]


In [143]:
pacientes_ges

,Problema Oncologico,RUT,DV,Nombre del Paciente,Estado,Fecha Próximo Hito,Medico Responsable,Especialidad Destino,Fecha de SIC,Estado de SIC,Establecimiento de Origen,DIAGNOSTICOS,Observaciones,Telefono Consolidado2,CITAS FUTURAS,MEDICO,ESTADO SIC
7,Cáncer Colorectal,3198181,6,"MURÚA NAVARRO, CARLOS DEL CARMEN",NaN,NaN,NaN,NaN,2024-06-27,NaN,NaN,NaN,NaN,- / - / 22823728 / / 76690716 / 22823728 / -/...,NaN,NaN,NaN
86,Cáncer Colorectal,5869303,0,"RIQUELME PUEBLA, GLORIA MARÍA BERNARD",NaN,NaN,NaN,NaN,2024-07-02,NaN,NaN,NaN,NaN,- / - / 25028287 / 0 / 97158876 / 25028287 / -...,NaN,NaN,NaN
100,Cáncer Colorectal,6094724,4,"PIZARRO SERRAT, RICARDO ROLANDO",NaN,NaN,NaN,NaN,2024-07-02,NaN,NaN,NaN,NaN,- / - / / 0 / 93865732 / - / -/9993865732/299...,NaN,NaN,NaN
174,Cáncer Pulmón,7540942,7,"VALDEBENITO LOBOS, MARÍA ANGÉLICA",NaN,NaN,NaN,NaN,2024-06-28,NaN,NaN,NaN,NaN,- / - / 0 / / 94364482 / - / -/./#ND/-,NaN,NaN,NaN
203,Cáncer Colorectal,8130920,5,"SILVA HENRÍQUEZ, GINA DE LAS MERCEDES",NaN,NaN,NaN,NaN,2024-07-03,NaN,NaN,NaN,NaN,- / - / 68262953 / 68262953 / 68262953 / 31898...,NaN,NaN,NaN
207,Cáncer Pulmón,8317900,7,"MELLI NAVARRETE, ESTER LUCRECIA",NaN,NaN,NaN,NaN,2024-06-28,NaN,NaN,NaN,NaN,- / - / 35766740 / 35766740 / 35766740 / 23140...,NaN,NaN,NaN
223,Cáncer Colorectal,8957749,7,"LEIVA CEPEDA, RODRIGO EDUARDO",NaN,NaN,NaN,NaN,2024-07-02,NaN,NaN,NaN,NaN,- / - / 0 / 0 / 75530673 / / -/75530673//./FI...,NaN,NaN,NaN
258,Cáncer Gástrico,10492252,K,"VILCHES PÉREZ, MARISA DE LAS MERCED",NaN,NaN,NaN,NaN,2024-06-28,NaN,NaN,NaN,NaN,- / - / 22828026 / 66931649 / 66931649 / 22828...,NaN,NaN,NaN
272,Cáncer Colorectal,11667136,0,"FERNÁNDEZ SILVA, IVÁN BLADIMIR",NaN,NaN,NaN,NaN,2024-07-03,NaN,NaN,NaN,NaN,- / - / 22620004 / 98769960 / - / 22620004 / 9...,NaN,NaN,NaN


In [144]:
list(pacientes_ges.columns)

['Problema Oncologico',
 'RUT',
 'DV',
 'Nombre del Paciente',
 'Estado',
 'Fecha Próximo Hito',
 'Medico Responsable',
 'Especialidad Destino',
 'Fecha de SIC',
 'Estado de SIC',
 'Establecimiento de Origen',
 'DIAGNOSTICOS',
 'Observaciones',
 'Telefono Consolidado2',
 'CITAS FUTURAS',
 'MEDICO',
 'ESTADO SIC']

In [145]:
pacientes_ges

,Problema Oncologico,RUT,DV,Nombre del Paciente,Estado,Fecha Próximo Hito,Medico Responsable,Especialidad Destino,Fecha de SIC,Estado de SIC,Establecimiento de Origen,DIAGNOSTICOS,Observaciones,Telefono Consolidado2,CITAS FUTURAS,MEDICO,ESTADO SIC
7,Cáncer Colorectal,3198181,6,"MURÚA NAVARRO, CARLOS DEL CARMEN",NaN,NaN,NaN,NaN,2024-06-27,NaN,NaN,NaN,NaN,- / - / 22823728 / / 76690716 / 22823728 / -/...,NaN,NaN,NaN
86,Cáncer Colorectal,5869303,0,"RIQUELME PUEBLA, GLORIA MARÍA BERNARD",NaN,NaN,NaN,NaN,2024-07-02,NaN,NaN,NaN,NaN,- / - / 25028287 / 0 / 97158876 / 25028287 / -...,NaN,NaN,NaN
100,Cáncer Colorectal,6094724,4,"PIZARRO SERRAT, RICARDO ROLANDO",NaN,NaN,NaN,NaN,2024-07-02,NaN,NaN,NaN,NaN,- / - / / 0 / 93865732 / - / -/9993865732/299...,NaN,NaN,NaN
174,Cáncer Pulmón,7540942,7,"VALDEBENITO LOBOS, MARÍA ANGÉLICA",NaN,NaN,NaN,NaN,2024-06-28,NaN,NaN,NaN,NaN,- / - / 0 / / 94364482 / - / -/./#ND/-,NaN,NaN,NaN
203,Cáncer Colorectal,8130920,5,"SILVA HENRÍQUEZ, GINA DE LAS MERCEDES",NaN,NaN,NaN,NaN,2024-07-03,NaN,NaN,NaN,NaN,- / - / 68262953 / 68262953 / 68262953 / 31898...,NaN,NaN,NaN
207,Cáncer Pulmón,8317900,7,"MELLI NAVARRETE, ESTER LUCRECIA",NaN,NaN,NaN,NaN,2024-06-28,NaN,NaN,NaN,NaN,- / - / 35766740 / 35766740 / 35766740 / 23140...,NaN,NaN,NaN
223,Cáncer Colorectal,8957749,7,"LEIVA CEPEDA, RODRIGO EDUARDO",NaN,NaN,NaN,NaN,2024-07-02,NaN,NaN,NaN,NaN,- / - / 0 / 0 / 75530673 / / -/75530673//./FI...,NaN,NaN,NaN
258,Cáncer Gástrico,10492252,K,"VILCHES PÉREZ, MARISA DE LAS MERCED",NaN,NaN,NaN,NaN,2024-06-28,NaN,NaN,NaN,NaN,- / - / 22828026 / 66931649 / 66931649 / 22828...,NaN,NaN,NaN
272,Cáncer Colorectal,11667136,0,"FERNÁNDEZ SILVA, IVÁN BLADIMIR",NaN,NaN,NaN,NaN,2024-07-03,NaN,NaN,NaN,NaN,- / - / 22620004 / 98769960 / - / 22620004 / 9...,NaN,NaN,NaN


In [146]:
df_pacientes_nuevos_activas

,Problema Oncologico,RUT,DV,Nombre del Paciente,Estado,Fecha Próximo Hito,Medico Responsable,Especialidad Destino,Fecha de SIC,Estado de SIC,Establecimiento de Origen,DIAGNOSTICOS,Observaciones,Telefono Consolidado2


In [147]:
df_pacientes_nuevos_activas = pd.concat([df_pacientes_nuevos_activas,pacientes_ges])

In [148]:
df_pacientes_nuevos_activas['CONCATENADO'] = df_pacientes_nuevos_activas['RUT'].astype(str)+df_pacientes_nuevos_activas['Especialidad Destino']

In [149]:
list(df_pacientes_nuevos_activas.columns)

['Problema Oncologico',
 'RUT',
 'DV',
 'Nombre del Paciente',
 'Estado',
 'Fecha Próximo Hito',
 'Medico Responsable',
 'Especialidad Destino',
 'Fecha de SIC',
 'Estado de SIC',
 'Establecimiento de Origen',
 'DIAGNOSTICOS',
 'Observaciones',
 'Telefono Consolidado2',
 'CITAS FUTURAS',
 'MEDICO',
 'ESTADO SIC',
 'CONCATENADO']

In [150]:
df_pacientes_nuevos_activas = df_pacientes_nuevos_activas[['Problema Oncologico',
 'RUT',
 'DV',
 'Nombre del Paciente',
 'Estado',
 'Fecha Próximo Hito',
 'Medico Responsable',
 'Especialidad Destino',
 'Fecha de SIC',
 'Estado de SIC',
 'Establecimiento de Origen',
 'Observaciones',
 'DIAGNOSTICOS',
 'Telefono Consolidado2',
 'CONCATENADO',
 'CITAS FUTURAS',
 'MEDICO',
 'ESTADO SIC'
 ]]

In [151]:
list(dataframe_2.columns)

['Problema Oncologico',
 'RUT',
 'DV',
 'Nombre del Paciente',
 'Estado',
 'Fecha Próximo Hito',
 'Medico Responsable',
 'Especialidad',
 'Fecha de SIC',
 'Estado de SIC',
 'Establecimiento de Origen',
 'Observaciones',
 'Diagnosticos',
 'Telefonos',
 'CONCATENADO',
 'CITAS FUTURAS',
 'MEDICO',
 'ESTADO SIC']

In [152]:
df_pacientes_nuevos_activas

,Problema Oncologico,RUT,DV,Nombre del Paciente,Estado,Fecha Próximo Hito,Medico Responsable,Especialidad Destino,Fecha de SIC,Estado de SIC,Establecimiento de Origen,DIAGNOSTICOS,Observaciones,Telefono Consolidado2,CONCATENADO,CITAS FUTURAS,MEDICO,ESTADO SIC
7,Cáncer Colorectal,3198181,6,"MURÚA NAVARRO, CARLOS DEL CARMEN",NaN,NaN,NaN,NaN,2024-06-27,NaN,NaN,NaN,NaN,- / - / 22823728 / / 76690716 / 22823728 / -/...,NaN,NaN,NaN,NaN
86,Cáncer Colorectal,5869303,0,"RIQUELME PUEBLA, GLORIA MARÍA BERNARD",NaN,NaN,NaN,NaN,2024-07-02,NaN,NaN,NaN,NaN,- / - / 25028287 / 0 / 97158876 / 25028287 / -...,NaN,NaN,NaN,NaN
100,Cáncer Colorectal,6094724,4,"PIZARRO SERRAT, RICARDO ROLANDO",NaN,NaN,NaN,NaN,2024-07-02,NaN,NaN,NaN,NaN,- / - / / 0 / 93865732 / - / -/9993865732/299...,NaN,NaN,NaN,NaN
174,Cáncer Pulmón,7540942,7,"VALDEBENITO LOBOS, MARÍA ANGÉLICA",NaN,NaN,NaN,NaN,2024-06-28,NaN,NaN,NaN,NaN,- / - / 0 / / 94364482 / - / -/./#ND/-,NaN,NaN,NaN,NaN
203,Cáncer Colorectal,8130920,5,"SILVA HENRÍQUEZ, GINA DE LAS MERCEDES",NaN,NaN,NaN,NaN,2024-07-03,NaN,NaN,NaN,NaN,- / - / 68262953 / 68262953 / 68262953 / 31898...,NaN,NaN,NaN,NaN
207,Cáncer Pulmón,8317900,7,"MELLI NAVARRETE, ESTER LUCRECIA",NaN,NaN,NaN,NaN,2024-06-28,NaN,NaN,NaN,NaN,- / - / 35766740 / 35766740 / 35766740 / 23140...,NaN,NaN,NaN,NaN
223,Cáncer Colorectal,8957749,7,"LEIVA CEPEDA, RODRIGO EDUARDO",NaN,NaN,NaN,NaN,2024-07-02,NaN,NaN,NaN,NaN,- / - / 0 / 0 / 75530673 / / -/75530673//./FI...,NaN,NaN,NaN,NaN
258,Cáncer Gástrico,10492252,K,"VILCHES PÉREZ, MARISA DE LAS MERCED",NaN,NaN,NaN,NaN,2024-06-28,NaN,NaN,NaN,NaN,- / - / 22828026 / 66931649 / 66931649 / 22828...,NaN,NaN,NaN,NaN
272,Cáncer Colorectal,11667136,0,"FERNÁNDEZ SILVA, IVÁN BLADIMIR",NaN,NaN,NaN,NaN,2024-07-03,NaN,NaN,NaN,NaN,- / - / 22620004 / 98769960 / - / 22620004 / 9...,NaN,NaN,NaN,NaN


## Guardar pacientes activos

In [153]:
# Guardar el DataFrame en un archivo de Excel

# Día actual
today = date.today()

# Fecha actual
now = datetime.now()

# Formatear día con ceros a la izquierda si es necesario
day_formatted = "{:02d}".format(today.day)

# Formatear mes con ceros a la izquierda si es necesario
month_formatted = "{:02d}".format(today.month)


nombre_archivo = f'{today.year}{month_formatted}{day_formatted}_derivaciones_nuevas.xlsx'  # Nombre del archivo de Excel
df_pacientes_nuevos_activas.to_excel(os.path.join(dir_actual, '..','DERIVACIONES','Reporte diario casos nuevos',f'{nombre_archivo}'), index=False)  # index=False para no incluir el índice en el archivo de Excel 


# Citas futuras

In [154]:
df_pacientes_nuevos_activas

,Problema Oncologico,RUT,DV,Nombre del Paciente,Estado,Fecha Próximo Hito,Medico Responsable,Especialidad Destino,Fecha de SIC,Estado de SIC,Establecimiento de Origen,DIAGNOSTICOS,Observaciones,Telefono Consolidado2,CONCATENADO,CITAS FUTURAS,MEDICO,ESTADO SIC
7,Cáncer Colorectal,3198181,6,"MURÚA NAVARRO, CARLOS DEL CARMEN",NaN,NaN,NaN,NaN,2024-06-27,NaN,NaN,NaN,NaN,- / - / 22823728 / / 76690716 / 22823728 / -/...,NaN,NaN,NaN,NaN
86,Cáncer Colorectal,5869303,0,"RIQUELME PUEBLA, GLORIA MARÍA BERNARD",NaN,NaN,NaN,NaN,2024-07-02,NaN,NaN,NaN,NaN,- / - / 25028287 / 0 / 97158876 / 25028287 / -...,NaN,NaN,NaN,NaN
100,Cáncer Colorectal,6094724,4,"PIZARRO SERRAT, RICARDO ROLANDO",NaN,NaN,NaN,NaN,2024-07-02,NaN,NaN,NaN,NaN,- / - / / 0 / 93865732 / - / -/9993865732/299...,NaN,NaN,NaN,NaN
174,Cáncer Pulmón,7540942,7,"VALDEBENITO LOBOS, MARÍA ANGÉLICA",NaN,NaN,NaN,NaN,2024-06-28,NaN,NaN,NaN,NaN,- / - / 0 / / 94364482 / - / -/./#ND/-,NaN,NaN,NaN,NaN
203,Cáncer Colorectal,8130920,5,"SILVA HENRÍQUEZ, GINA DE LAS MERCEDES",NaN,NaN,NaN,NaN,2024-07-03,NaN,NaN,NaN,NaN,- / - / 68262953 / 68262953 / 68262953 / 31898...,NaN,NaN,NaN,NaN
207,Cáncer Pulmón,8317900,7,"MELLI NAVARRETE, ESTER LUCRECIA",NaN,NaN,NaN,NaN,2024-06-28,NaN,NaN,NaN,NaN,- / - / 35766740 / 35766740 / 35766740 / 23140...,NaN,NaN,NaN,NaN
223,Cáncer Colorectal,8957749,7,"LEIVA CEPEDA, RODRIGO EDUARDO",NaN,NaN,NaN,NaN,2024-07-02,NaN,NaN,NaN,NaN,- / - / 0 / 0 / 75530673 / / -/75530673//./FI...,NaN,NaN,NaN,NaN
258,Cáncer Gástrico,10492252,K,"VILCHES PÉREZ, MARISA DE LAS MERCED",NaN,NaN,NaN,NaN,2024-06-28,NaN,NaN,NaN,NaN,- / - / 22828026 / 66931649 / 66931649 / 22828...,NaN,NaN,NaN,NaN
272,Cáncer Colorectal,11667136,0,"FERNÁNDEZ SILVA, IVÁN BLADIMIR",NaN,NaN,NaN,NaN,2024-07-03,NaN,NaN,NaN,NaN,- / - / 22620004 / 98769960 / - / 22620004 / 9...,NaN,NaN,NaN,NaN


In [155]:
list(df_pacientes_nuevos_activas.columns)

['Problema Oncologico',
 'RUT',
 'DV',
 'Nombre del Paciente',
 'Estado',
 'Fecha Próximo Hito',
 'Medico Responsable',
 'Especialidad Destino',
 'Fecha de SIC',
 'Estado de SIC',
 'Establecimiento de Origen',
 'DIAGNOSTICOS',
 'Observaciones',
 'Telefono Consolidado2',
 'CONCATENADO',
 'CITAS FUTURAS',
 'MEDICO',
 'ESTADO SIC']

In [156]:
list(endocrinologia_df_fu.columns)

['RUT',
 'NOMBRE_AGENDA',
 'ESPECIALIDAD_AGENDA',
 'CITAFECH',
 'CITAHORA',
 'ESTADO_CITA',
 'DESCRIPCION',
 'ORIGEN',
 'GES_CITA',
 'GES_SOL',
 'PATOLOGIA_GES_SOL',
 'PACIENTE',
 'CITA_CODUTEN',
 'DOCUMENTO']

In [157]:
list(df_ca_tiroide.columns)

['Problema Oncologico',
 'RUT',
 'DV',
 'Nombre del Paciente',
 'Estado',
 'Fecha Próximo Hito',
 'Medico Responsable',
 'Especialidad Destino',
 'Fecha de SIC',
 'Estado de SIC',
 'Establecimiento de Origen',
 'DIAGNOSTICOS',
 'Observaciones',
 'Telefono Consolidado2',
 'CITAS FUTURAS',
 'MEDICO',
 'ESTADO SIC']

In [158]:
df_pacientes_nuevos_activas

,Problema Oncologico,RUT,DV,Nombre del Paciente,Estado,Fecha Próximo Hito,Medico Responsable,Especialidad Destino,Fecha de SIC,Estado de SIC,Establecimiento de Origen,DIAGNOSTICOS,Observaciones,Telefono Consolidado2,CONCATENADO,CITAS FUTURAS,MEDICO,ESTADO SIC
7,Cáncer Colorectal,3198181,6,"MURÚA NAVARRO, CARLOS DEL CARMEN",NaN,NaN,NaN,NaN,2024-06-27,NaN,NaN,NaN,NaN,- / - / 22823728 / / 76690716 / 22823728 / -/...,NaN,NaN,NaN,NaN
86,Cáncer Colorectal,5869303,0,"RIQUELME PUEBLA, GLORIA MARÍA BERNARD",NaN,NaN,NaN,NaN,2024-07-02,NaN,NaN,NaN,NaN,- / - / 25028287 / 0 / 97158876 / 25028287 / -...,NaN,NaN,NaN,NaN
100,Cáncer Colorectal,6094724,4,"PIZARRO SERRAT, RICARDO ROLANDO",NaN,NaN,NaN,NaN,2024-07-02,NaN,NaN,NaN,NaN,- / - / / 0 / 93865732 / - / -/9993865732/299...,NaN,NaN,NaN,NaN
174,Cáncer Pulmón,7540942,7,"VALDEBENITO LOBOS, MARÍA ANGÉLICA",NaN,NaN,NaN,NaN,2024-06-28,NaN,NaN,NaN,NaN,- / - / 0 / / 94364482 / - / -/./#ND/-,NaN,NaN,NaN,NaN
203,Cáncer Colorectal,8130920,5,"SILVA HENRÍQUEZ, GINA DE LAS MERCEDES",NaN,NaN,NaN,NaN,2024-07-03,NaN,NaN,NaN,NaN,- / - / 68262953 / 68262953 / 68262953 / 31898...,NaN,NaN,NaN,NaN
207,Cáncer Pulmón,8317900,7,"MELLI NAVARRETE, ESTER LUCRECIA",NaN,NaN,NaN,NaN,2024-06-28,NaN,NaN,NaN,NaN,- / - / 35766740 / 35766740 / 35766740 / 23140...,NaN,NaN,NaN,NaN
223,Cáncer Colorectal,8957749,7,"LEIVA CEPEDA, RODRIGO EDUARDO",NaN,NaN,NaN,NaN,2024-07-02,NaN,NaN,NaN,NaN,- / - / 0 / 0 / 75530673 / / -/75530673//./FI...,NaN,NaN,NaN,NaN
258,Cáncer Gástrico,10492252,K,"VILCHES PÉREZ, MARISA DE LAS MERCED",NaN,NaN,NaN,NaN,2024-06-28,NaN,NaN,NaN,NaN,- / - / 22828026 / 66931649 / 66931649 / 22828...,NaN,NaN,NaN,NaN
272,Cáncer Colorectal,11667136,0,"FERNÁNDEZ SILVA, IVÁN BLADIMIR",NaN,NaN,NaN,NaN,2024-07-03,NaN,NaN,NaN,NaN,- / - / 22620004 / 98769960 / - / 22620004 / 9...,NaN,NaN,NaN,NaN


In [159]:
dataframe_2

,Problema Oncologico,RUT,DV,Nombre del Paciente,Estado,Fecha Próximo Hito,Medico Responsable,Especialidad,Fecha de SIC,Estado de SIC,Establecimiento de Origen,Observaciones,Diagnosticos,Telefonos,CONCATENADO,CITAS FUTURAS,MEDICO,ESTADO SIC
0,Cáncer Próstata,5926301,3,ADA VILCHES MORALES,Priorizacion CUD,NaN,NaN,Urología Adulto,2024-05-13,NaN,NaN,NaN,NaN,NaN,5926301Urología Adulto,NaN,NaN,Solicitada
1,Cáncer Tiroides,16698412,2,Canales Marchant Nicole Paulina,Priorizacion CUD,NaN,NaN,Endocrinología Adulto,2024-05-08,NaN,CESFAM BELLAVISTA,NaN,NaN,NaN,16698412Endocrinología Adulto,NaN,NaN,Finalizada
2,Cáncer Próstata,99616630,K,CARLOS AGUILAR SALAS,Priorizacion CUD,NaN,NaN,Urología Adulto,2024-05-02,NaN,NaN,NaN,NaN,NaN,99616630Urología Adulto,NaN,NaN,NaN
3,Cáncer Pulmón,7408006,5,Cuevas Villalobos Eulalia Iris,Rechazada,NaN,NaN,Broncopulmonar Adulto,2024-06-03,rechazada,CESFAM JOSE ALVO,no pertinente (cud),NaN,NaN,7408006Broncopulmonar Adulto,NaN,NaN,NaN
4,NaN,7262387,8,Cuevas Villalobos Rene Edmundo,Priorizacion CUD,NaN,NaN,Dermatología Adulto,2024-06-07,NaN,CESFAM JOSE ALVO,NaN,NaN,NaN,7262387DERMATOLOGIA,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320,Cáncer Colorectal,7196918,5,"GAJARDO DÍAZ, JUANA NATALIA",NaN,NaN,NaN,NaN,2024-06-28,NaN,NaN,NaN,NaN,- / - / 85552235 / 85552235 / 85552235 / 85552...,NaN,NaN,NaN,NaN
321,Cáncer Colorectal,8941727,9,"MEDINA JIMÉNEZ, LUCRECIA DEL CARMEN",NaN,NaN,NaN,NaN,2024-06-25,NaN,NaN,NaN,NaN,- / - / / / 63744000 / 28391288 / -/96374400...,NaN,NaN,NaN,NaN
322,Cáncer Gástrico,11904915,6,"ROJAS SÁEZ, FLORENTINO AVELINO",NaN,NaN,NaN,NaN,2024-06-27,NaN,NaN,NaN,NaN,#ND/#ND/-,NaN,NaN,NaN,NaN
323,Cáncer Colorectal,12108563,1,"GUAJARDO MELLA, JACQUELINE DEL CARMEN",NaN,NaN,NaN,NaN,2024-06-24,NaN,NaN,NaN,NaN,- / - / 42010925 / 42010925 / 67434851 / 67434...,NaN,NaN,NaN,NaN


In [160]:
df_pacientes_nuevos_activas

,Problema Oncologico,RUT,DV,Nombre del Paciente,Estado,Fecha Próximo Hito,Medico Responsable,Especialidad Destino,Fecha de SIC,Estado de SIC,Establecimiento de Origen,DIAGNOSTICOS,Observaciones,Telefono Consolidado2,CONCATENADO,CITAS FUTURAS,MEDICO,ESTADO SIC
7,Cáncer Colorectal,3198181,6,"MURÚA NAVARRO, CARLOS DEL CARMEN",NaN,NaN,NaN,NaN,2024-06-27,NaN,NaN,NaN,NaN,- / - / 22823728 / / 76690716 / 22823728 / -/...,NaN,NaN,NaN,NaN
86,Cáncer Colorectal,5869303,0,"RIQUELME PUEBLA, GLORIA MARÍA BERNARD",NaN,NaN,NaN,NaN,2024-07-02,NaN,NaN,NaN,NaN,- / - / 25028287 / 0 / 97158876 / 25028287 / -...,NaN,NaN,NaN,NaN
100,Cáncer Colorectal,6094724,4,"PIZARRO SERRAT, RICARDO ROLANDO",NaN,NaN,NaN,NaN,2024-07-02,NaN,NaN,NaN,NaN,- / - / / 0 / 93865732 / - / -/9993865732/299...,NaN,NaN,NaN,NaN
174,Cáncer Pulmón,7540942,7,"VALDEBENITO LOBOS, MARÍA ANGÉLICA",NaN,NaN,NaN,NaN,2024-06-28,NaN,NaN,NaN,NaN,- / - / 0 / / 94364482 / - / -/./#ND/-,NaN,NaN,NaN,NaN
203,Cáncer Colorectal,8130920,5,"SILVA HENRÍQUEZ, GINA DE LAS MERCEDES",NaN,NaN,NaN,NaN,2024-07-03,NaN,NaN,NaN,NaN,- / - / 68262953 / 68262953 / 68262953 / 31898...,NaN,NaN,NaN,NaN
207,Cáncer Pulmón,8317900,7,"MELLI NAVARRETE, ESTER LUCRECIA",NaN,NaN,NaN,NaN,2024-06-28,NaN,NaN,NaN,NaN,- / - / 35766740 / 35766740 / 35766740 / 23140...,NaN,NaN,NaN,NaN
223,Cáncer Colorectal,8957749,7,"LEIVA CEPEDA, RODRIGO EDUARDO",NaN,NaN,NaN,NaN,2024-07-02,NaN,NaN,NaN,NaN,- / - / 0 / 0 / 75530673 / / -/75530673//./FI...,NaN,NaN,NaN,NaN
258,Cáncer Gástrico,10492252,K,"VILCHES PÉREZ, MARISA DE LAS MERCED",NaN,NaN,NaN,NaN,2024-06-28,NaN,NaN,NaN,NaN,- / - / 22828026 / 66931649 / 66931649 / 22828...,NaN,NaN,NaN,NaN
272,Cáncer Colorectal,11667136,0,"FERNÁNDEZ SILVA, IVÁN BLADIMIR",NaN,NaN,NaN,NaN,2024-07-03,NaN,NaN,NaN,NaN,- / - / 22620004 / 98769960 / - / 22620004 / 9...,NaN,NaN,NaN,NaN


In [161]:
df_pacientes_nuevos_activas

,Problema Oncologico,RUT,DV,Nombre del Paciente,Estado,Fecha Próximo Hito,Medico Responsable,Especialidad Destino,Fecha de SIC,Estado de SIC,Establecimiento de Origen,DIAGNOSTICOS,Observaciones,Telefono Consolidado2,CONCATENADO,CITAS FUTURAS,MEDICO,ESTADO SIC
7,Cáncer Colorectal,3198181,6,"MURÚA NAVARRO, CARLOS DEL CARMEN",NaN,NaN,NaN,NaN,2024-06-27,NaN,NaN,NaN,NaN,- / - / 22823728 / / 76690716 / 22823728 / -/...,NaN,NaN,NaN,NaN
86,Cáncer Colorectal,5869303,0,"RIQUELME PUEBLA, GLORIA MARÍA BERNARD",NaN,NaN,NaN,NaN,2024-07-02,NaN,NaN,NaN,NaN,- / - / 25028287 / 0 / 97158876 / 25028287 / -...,NaN,NaN,NaN,NaN
100,Cáncer Colorectal,6094724,4,"PIZARRO SERRAT, RICARDO ROLANDO",NaN,NaN,NaN,NaN,2024-07-02,NaN,NaN,NaN,NaN,- / - / / 0 / 93865732 / - / -/9993865732/299...,NaN,NaN,NaN,NaN
174,Cáncer Pulmón,7540942,7,"VALDEBENITO LOBOS, MARÍA ANGÉLICA",NaN,NaN,NaN,NaN,2024-06-28,NaN,NaN,NaN,NaN,- / - / 0 / / 94364482 / - / -/./#ND/-,NaN,NaN,NaN,NaN
203,Cáncer Colorectal,8130920,5,"SILVA HENRÍQUEZ, GINA DE LAS MERCEDES",NaN,NaN,NaN,NaN,2024-07-03,NaN,NaN,NaN,NaN,- / - / 68262953 / 68262953 / 68262953 / 31898...,NaN,NaN,NaN,NaN
207,Cáncer Pulmón,8317900,7,"MELLI NAVARRETE, ESTER LUCRECIA",NaN,NaN,NaN,NaN,2024-06-28,NaN,NaN,NaN,NaN,- / - / 35766740 / 35766740 / 35766740 / 23140...,NaN,NaN,NaN,NaN
223,Cáncer Colorectal,8957749,7,"LEIVA CEPEDA, RODRIGO EDUARDO",NaN,NaN,NaN,NaN,2024-07-02,NaN,NaN,NaN,NaN,- / - / 0 / 0 / 75530673 / / -/75530673//./FI...,NaN,NaN,NaN,NaN
258,Cáncer Gástrico,10492252,K,"VILCHES PÉREZ, MARISA DE LAS MERCED",NaN,NaN,NaN,NaN,2024-06-28,NaN,NaN,NaN,NaN,- / - / 22828026 / 66931649 / 66931649 / 22828...,NaN,NaN,NaN,NaN
272,Cáncer Colorectal,11667136,0,"FERNÁNDEZ SILVA, IVÁN BLADIMIR",NaN,NaN,NaN,NaN,2024-07-03,NaN,NaN,NaN,NaN,- / - / 22620004 / 98769960 / - / 22620004 / 9...,NaN,NaN,NaN,NaN


In [162]:
df_pacientes_nuevos_activas = df_pacientes_nuevos_activas.rename(columns={'Problema Oncologico_x': 'Problema Oncologico'})
df_pacientes_nuevos_activas = df_pacientes_nuevos_activas.rename(columns={'Nombre del Paciente_x': 'Nombre del Paciente'})
df_pacientes_nuevos_activas = df_pacientes_nuevos_activas.rename(columns={'Estado_x': 'Estado'})
df_pacientes_nuevos_activas = df_pacientes_nuevos_activas.rename(columns={'Fecha Próximo Hito_x': 'Fecha Próximo Hito'})
df_pacientes_nuevos_activas = df_pacientes_nuevos_activas.rename(columns={'Medico Responsable_x': 'Medico Responsable'})
df_pacientes_nuevos_activas = df_pacientes_nuevos_activas.rename(columns={'Fecha de SIC_x': 'Fecha de SIC'})
df_pacientes_nuevos_activas = df_pacientes_nuevos_activas.rename(columns={'Estado de SIC_x': 'Estado de SIC'})
df_pacientes_nuevos_activas = df_pacientes_nuevos_activas.rename(columns={'Establecimiento de Origen_x': 'Establecimiento de Origen'})
df_pacientes_nuevos_activas = df_pacientes_nuevos_activas.rename(columns={'DIAGNOSTICOS_x': 'DIAGNOSTICOS'})
df_pacientes_nuevos_activas = df_pacientes_nuevos_activas.rename(columns={'Observaciones_x': 'Observaciones'})
df_pacientes_nuevos_activas = df_pacientes_nuevos_activas.rename(columns={'Telefono Consolidado2_x': 'Telefono Consolidado2'})
df_pacientes_nuevos_activas = df_pacientes_nuevos_activas.rename(columns={'CITAS FUTURAS_x': 'CITAS FUTURAS'})
df_pacientes_nuevos_activas = df_pacientes_nuevos_activas.rename(columns={'MEDICO_x': 'MEDICO'})
df_pacientes_nuevos_activas = df_pacientes_nuevos_activas.rename(columns={'ESTADO SIC_x': 'ESTADO SIC'})
df_pacientes_nuevos_activas = df_pacientes_nuevos_activas.rename(columns={'Especialidad Destino_x': 'Especialidad'})

In [163]:
df_concatenated_3 = pd.concat([dataframe_2,df_pacientes_nuevos_activas])

In [164]:
df_concatenated_3

,Problema Oncologico,RUT,DV,Nombre del Paciente,Estado,Fecha Próximo Hito,Medico Responsable,Especialidad,Fecha de SIC,Estado de SIC,...,Observaciones,Diagnosticos,Telefonos,CONCATENADO,CITAS FUTURAS,MEDICO,ESTADO SIC,Especialidad Destino,DIAGNOSTICOS,Telefono Consolidado2
0,Cáncer Próstata,5926301,3,ADA VILCHES MORALES,Priorizacion CUD,NaN,NaN,Urología Adulto,2024-05-13,NaN,...,NaN,NaN,NaN,5926301Urología Adulto,NaN,NaN,Solicitada,NaN,NaN,NaN
1,Cáncer Tiroides,16698412,2,Canales Marchant Nicole Paulina,Priorizacion CUD,NaN,NaN,Endocrinología Adulto,2024-05-08,NaN,...,NaN,NaN,NaN,16698412Endocrinología Adulto,NaN,NaN,Finalizada,NaN,NaN,NaN
2,Cáncer Próstata,99616630,K,CARLOS AGUILAR SALAS,Priorizacion CUD,NaN,NaN,Urología Adulto,2024-05-02,NaN,...,NaN,NaN,NaN,99616630Urología Adulto,NaN,NaN,NaN,NaN,NaN,NaN
3,Cáncer Pulmón,7408006,5,Cuevas Villalobos Eulalia Iris,Rechazada,NaN,NaN,Broncopulmonar Adulto,2024-06-03,rechazada,...,no pertinente (cud),NaN,NaN,7408006Broncopulmonar Adulto,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,7262387,8,Cuevas Villalobos Rene Edmundo,Priorizacion CUD,NaN,NaN,Dermatología Adulto,2024-06-07,NaN,...,NaN,NaN,NaN,7262387DERMATOLOGIA,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,Cáncer Colorectal,8130920,5,"SILVA HENRÍQUEZ, GINA DE LAS MERCEDES",NaN,NaN,NaN,NaN,2024-07-03,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,- / - / 68262953 / 68262953 / 68262953 / 31898...
207,Cáncer Pulmón,8317900,7,"MELLI NAVARRETE, ESTER LUCRECIA",NaN,NaN,NaN,NaN,2024-06-28,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,- / - / 35766740 / 35766740 / 35766740 / 23140...
223,Cáncer Colorectal,8957749,7,"LEIVA CEPEDA, RODRIGO EDUARDO",NaN,NaN,NaN,NaN,2024-07-02,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,- / - / 0 / 0 / 75530673 / / -/75530673//./FI...
258,Cáncer Gástrico,10492252,K,"VILCHES PÉREZ, MARISA DE LAS MERCED",NaN,NaN,NaN,NaN,2024-06-28,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,- / - / 22828026 / 66931649 / 66931649 / 22828...


In [165]:
import pandas as pd
import re



# Mapeo de enfermedades a expresiones regulares de especialidades
mapeo_especialidades = {
    'Cáncer Colorectal': re.compile(r'coloproctolog[í]a|proctol[óo]gica', re.IGNORECASE),
    'Mieloma': re.compile(r'Hematolog[ií]a', re.IGNORECASE),
    'Leucemia': re.compile(r'Hematolog[ií]a', re.IGNORECASE),
    'Linfoma': re.compile(r'Hematolog[ií]a', re.IGNORECASE),
    'Cáncer Gástrico': re.compile(r'(medicina.*interna|cirug[ií]a.*abdominal)', re.IGNORECASE),
    'Cáncer Tiroides': re.compile(r'(cirug[ií]a.*cabeza.*cuello|endocrinolog[íi]a.*adulto)', re.IGNORECASE),
    'Cáncer Pulmón': re.compile(r'(broncopulmonar|cirug[ií]a.*t[óo]rax)', re.IGNORECASE),
    'Cáncer Próstata': re.compile(r'urolog[íi]a', re.IGNORECASE),
    'Cáncer Testículo': re.compile(r'urolog[íi]a', re.IGNORECASE),
    'Cáncer Vesical': re.compile(r'urolog[íi]a', re.IGNORECASE),
    'Cáncer Renal': re.compile(r'urolog[íi]a', re.IGNORECASE),
}

df_concatenated_3['RUT'] = df_concatenated_3['RUT'].astype(int)
endocrinologia_df_fu['RUT'] = endocrinologia_df_fu['RUT'].astype(int)
# Filtrar y mergear
df_merged = pd.merge(
    df_concatenated_3,
    endocrinologia_df_fu,
    on='RUT',
    how='inner'
)

# Filtrar el dataframe mergeado para que cumpla las condiciones de mapeo
def filtrar_por_especialidad(row):
    pattern = mapeo_especialidades.get(row['Problema Oncologico'])
    if pattern:
        return bool(pattern.search(row['ESPECIALIDAD_AGENDA']))
    return False




df_merged_filtrado = df_merged[df_merged.apply(filtrar_por_especialidad, axis=1)]
df_merged_filtrado = df_merged_filtrado.rename(columns={'DV_x': 'DV'})



In [166]:
df_merged

,Problema Oncologico,RUT,DV,Nombre del Paciente,Estado,Fecha Próximo Hito,Medico Responsable,Especialidad,Fecha de SIC,Estado de SIC,...,CITAHORA,ESTADO_CITA,DESCRIPCION,ORIGEN,GES_CITA,GES_SOL,PATOLOGIA_GES_SOL,PACIENTE,CITA_CODUTEN,DOCUMENTO
0,Cáncer Gástrico,17097810,2,Galaz Aguilera Johanna Lorena,Atendido,2024-06-19 00:00:00,NaN,Cirugía Abdominal Adulto,2024-06-06,Atendido,...,1630,Agendada,Control postquirurgico,CEX,N,N,NaN,JOHANNA LORENA GALAZ AGUILERA,AC00548103,170978102
1,Cáncer Tiroides,7816012,8,JOHANNA GALAZ AGUILERA,Atendido,03-09-2024,NaN,Endocrinología Adulto,2024-06-03,NaN,...,920,Agendada,Control - Consulta integral de especialidades,CEX,N,N,NaN,CARLOS ERNESTO HERRERA BRIONES,AC00173258,78160128
2,NaN,5814383,9,JOSE VILLEGAS QUIROGA,Priorizacion CUD,NaN,NaN,Traumatología Adulto,2024-05-02,NaN,...,1130,Agendada,Control - Consulta integral de especialidades,CEX,N,N,NaN,MARÍA DEL CARMEN MENA FUENZALIDA,AC00544741,58143839
3,NaN,6999136,K,MARCELINO HERNANDEZ RODRIGUEZ,Priorizacion CUD,NaN,NaN,Cirugía Cabeza y cuello,2024-05-07,NaN,...,840,Agendada,Derivacion desde APS,PRI,N,N,NaN,ELIZABETH DEL CARMEN TRONCOSO IRARRÁZABAL,AC00454451,6999136K
4,Cáncer Gástrico,9214167,5,MARCELO GALVEZ GARAY,Priorizacion CUD,NaN,NaN,Cirugía Abdominal Adulto,NaT,NaN,...,1330,Agendada,Derivacion desde APS,CEX,N,N,NaN,Miguel Araham Albornoz Cifuentes,AC00060002,92141675
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,Cáncer Testículo,18547328,7,Daniel Alejandro Mundaca Carrasco,NaN,NaN,NaN,Urología Adulto,2024-06-17,Solicitada,...,900,Agendada,Derivacion desde APS,PRI,S,S,Cáncer De Testículo En Personas De 15 Años Y Más,Daniel Alejandro Mundaca Carrasco,AC00117760,185473287
82,Cáncer Tiroides,21129381,0,KATHERINE ALEJANDRA PALACIOS BARRIGA,NaN,NaN,NaN,Cirugia Cabeza y Cuello,2024-06-17,Pendiente de Autorizar,...,1145,Agendada,Interconsulta,CEX,S,S,CÁNCER DE TIROIDES DIFERENCIADO Y MEDULAR EN P...,KATHERINE ALEJANDRA PALACIOS BARRIGA,AC00382122,211293810
83,Cáncer Gástrico,6227519,7,"GARRIDO NAVARRETE, GEORGINA MERCEDES",NaN,NaN,NaN,NaN,2024-06-20,NaN,...,1330,Agendada,Derivacion desde APS,PRI,S,S,Cáncer Gástrico,Georgina Garrido Navarrete,AC00036072,62275197
84,Cáncer Gástrico,10042997,7,"YÁÑEZ FUENTES, NANCY DEL ROSARIO",NaN,NaN,NaN,NaN,2024-06-23,NaN,...,1400,Agendada,Derivacion desde APS,PRI,S,S,Cáncer Gástrico,Nancy Del Rosario Yáñez Fuentes,AC00065990,100429977


In [167]:
df_merged_filtrado[['RUT','ESPECIALIDAD_AGENDA','Problema Oncologico','ESTADO_CITA','NOMBRE_AGENDA','CITAFECH','CITAHORA','ESTADO_CITA','DESCRIPCION','ORIGEN','GES_CITA','GES_SOL','PATOLOGIA_GES_SOL','PACIENTE','CITA_CODUTEN','DOCUMENTO']]

,RUT,ESPECIALIDAD_AGENDA,Problema Oncologico,ESTADO_CITA,NOMBRE_AGENDA,CITAFECH,CITAHORA,ESTADO_CITA,DESCRIPCION,ORIGEN,GES_CITA,GES_SOL,PATOLOGIA_GES_SOL,PACIENTE,CITA_CODUTEN,DOCUMENTO
0,17097810,Cirugía Abdominal Adulto,Cáncer Gástrico,Agendada,DR. RUBÉN DARÍO ÁVILA SEPÚLVEDA,17-07-2024,1630,Agendada,Control postquirurgico,CEX,N,N,NaN,JOHANNA LORENA GALAZ AGUILERA,AC00548103,170978102
1,7816012,Endocrinología Adulto,Cáncer Tiroides,Agendada,DR. NICOLE ALEXANDRA BUSTIOS CASANOVA,03-09-2024,920,Agendada,Control - Consulta integral de especialidades,CEX,N,N,NaN,CARLOS ERNESTO HERRERA BRIONES,AC00173258,78160128
4,9214167,Medicina Interna,Cáncer Gástrico,Agendada,DR. MARIA GABRIELA CAMPOVERDE BRAVO,18-07-2024,1330,Agendada,Derivacion desde APS,CEX,N,N,NaN,Miguel Araham Albornoz Cifuentes,AC00060002,92141675
7,6811765,Cirugía Abdominal Adulto,Cáncer Gástrico,Agendada,DR. RUBÉN DARÍO ÁVILA SEPÚLVEDA,24-07-2024,1500,Agendada,Control - Consulta integral de especialidades,CEX,N,N,NaN,EDMUNDO FLORENTINO PARRA HERMOSILLA,AC00547102,68117658
12,7380981,Urología Adulto,Cáncer Renal,Agendada,DR. RODRIGO ANTONIO GONZÁLEZ IRRIBARRA,01-08-2024,1320,Agendada,Interconsulta,CEX,N,N,NaN,BERNARDINO SEGUNDO CAYUPE BELTRÁN,AC00475641,73809819
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,18547328,Urología Adulto,Cáncer Testículo,Agendada,DR. VÍCTOR MANUEL AVILLO CAROCA,08-07-2024,900,Agendada,Derivacion desde APS,PRI,S,S,Cáncer De Testículo En Personas De 15 Años Y Más,Daniel Alejandro Mundaca Carrasco,AC00117760,185473287
82,21129381,Cirugia Cabeza y Cuello,Cáncer Tiroides,Agendada,DR. PABLO ALEJANDRO DELGADO ARNEZ,19-07-2024,1145,Agendada,Interconsulta,CEX,S,S,CÁNCER DE TIROIDES DIFERENCIADO Y MEDULAR EN P...,KATHERINE ALEJANDRA PALACIOS BARRIGA,AC00382122,211293810
83,6227519,Medicina Interna,Cáncer Gástrico,Agendada,DR. MARIA GABRIELA CAMPOVERDE BRAVO,22-07-2024,1330,Agendada,Derivacion desde APS,PRI,S,S,Cáncer Gástrico,Georgina Garrido Navarrete,AC00036072,62275197
84,10042997,Medicina Interna,Cáncer Gástrico,Agendada,DR. MARIA GABRIELA CAMPOVERDE BRAVO,17-07-2024,1400,Agendada,Derivacion desde APS,PRI,S,S,Cáncer Gástrico,Nancy Del Rosario Yáñez Fuentes,AC00065990,100429977


In [168]:
df_merged_filtrado['ESTADO_CITA'].value_counts()

ESTADO_CITA
Agendada    65
Name: count, dtype: int64

In [169]:
list(endocrinologia_df_fu.columns)

['RUT',
 'NOMBRE_AGENDA',
 'ESPECIALIDAD_AGENDA',
 'CITAFECH',
 'CITAHORA',
 'ESTADO_CITA',
 'DESCRIPCION',
 'ORIGEN',
 'GES_CITA',
 'GES_SOL',
 'PATOLOGIA_GES_SOL',
 'PACIENTE',
 'CITA_CODUTEN',
 'DOCUMENTO']

In [170]:
df_merged_filtrado[endocrinologia_df_fu.columns]


,RUT,NOMBRE_AGENDA,ESPECIALIDAD_AGENDA,CITAFECH,CITAHORA,ESTADO_CITA,DESCRIPCION,ORIGEN,GES_CITA,GES_SOL,PATOLOGIA_GES_SOL,PACIENTE,CITA_CODUTEN,DOCUMENTO
0,17097810,DR. RUBÉN DARÍO ÁVILA SEPÚLVEDA,Cirugía Abdominal Adulto,17-07-2024,1630,Agendada,Control postquirurgico,CEX,N,N,NaN,JOHANNA LORENA GALAZ AGUILERA,AC00548103,170978102
1,7816012,DR. NICOLE ALEXANDRA BUSTIOS CASANOVA,Endocrinología Adulto,03-09-2024,920,Agendada,Control - Consulta integral de especialidades,CEX,N,N,NaN,CARLOS ERNESTO HERRERA BRIONES,AC00173258,78160128
4,9214167,DR. MARIA GABRIELA CAMPOVERDE BRAVO,Medicina Interna,18-07-2024,1330,Agendada,Derivacion desde APS,CEX,N,N,NaN,Miguel Araham Albornoz Cifuentes,AC00060002,92141675
7,6811765,DR. RUBÉN DARÍO ÁVILA SEPÚLVEDA,Cirugía Abdominal Adulto,24-07-2024,1500,Agendada,Control - Consulta integral de especialidades,CEX,N,N,NaN,EDMUNDO FLORENTINO PARRA HERMOSILLA,AC00547102,68117658
12,7380981,DR. RODRIGO ANTONIO GONZÁLEZ IRRIBARRA,Urología Adulto,01-08-2024,1320,Agendada,Interconsulta,CEX,N,N,NaN,BERNARDINO SEGUNDO CAYUPE BELTRÁN,AC00475641,73809819
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,18547328,DR. VÍCTOR MANUEL AVILLO CAROCA,Urología Adulto,08-07-2024,900,Agendada,Derivacion desde APS,PRI,S,S,Cáncer De Testículo En Personas De 15 Años Y Más,Daniel Alejandro Mundaca Carrasco,AC00117760,185473287
82,21129381,DR. PABLO ALEJANDRO DELGADO ARNEZ,Cirugia Cabeza y Cuello,19-07-2024,1145,Agendada,Interconsulta,CEX,S,S,CÁNCER DE TIROIDES DIFERENCIADO Y MEDULAR EN P...,KATHERINE ALEJANDRA PALACIOS BARRIGA,AC00382122,211293810
83,6227519,DR. MARIA GABRIELA CAMPOVERDE BRAVO,Medicina Interna,22-07-2024,1330,Agendada,Derivacion desde APS,PRI,S,S,Cáncer Gástrico,Georgina Garrido Navarrete,AC00036072,62275197
84,10042997,DR. MARIA GABRIELA CAMPOVERDE BRAVO,Medicina Interna,17-07-2024,1400,Agendada,Derivacion desde APS,PRI,S,S,Cáncer Gástrico,Nancy Del Rosario Yáñez Fuentes,AC00065990,100429977


In [171]:
df_citas_futuras = df_merged_filtrado[endocrinologia_df_fu.columns]

In [172]:
df_citas_futuras

,RUT,NOMBRE_AGENDA,ESPECIALIDAD_AGENDA,CITAFECH,CITAHORA,ESTADO_CITA,DESCRIPCION,ORIGEN,GES_CITA,GES_SOL,PATOLOGIA_GES_SOL,PACIENTE,CITA_CODUTEN,DOCUMENTO
0,17097810,DR. RUBÉN DARÍO ÁVILA SEPÚLVEDA,Cirugía Abdominal Adulto,17-07-2024,1630,Agendada,Control postquirurgico,CEX,N,N,NaN,JOHANNA LORENA GALAZ AGUILERA,AC00548103,170978102
1,7816012,DR. NICOLE ALEXANDRA BUSTIOS CASANOVA,Endocrinología Adulto,03-09-2024,920,Agendada,Control - Consulta integral de especialidades,CEX,N,N,NaN,CARLOS ERNESTO HERRERA BRIONES,AC00173258,78160128
4,9214167,DR. MARIA GABRIELA CAMPOVERDE BRAVO,Medicina Interna,18-07-2024,1330,Agendada,Derivacion desde APS,CEX,N,N,NaN,Miguel Araham Albornoz Cifuentes,AC00060002,92141675
7,6811765,DR. RUBÉN DARÍO ÁVILA SEPÚLVEDA,Cirugía Abdominal Adulto,24-07-2024,1500,Agendada,Control - Consulta integral de especialidades,CEX,N,N,NaN,EDMUNDO FLORENTINO PARRA HERMOSILLA,AC00547102,68117658
12,7380981,DR. RODRIGO ANTONIO GONZÁLEZ IRRIBARRA,Urología Adulto,01-08-2024,1320,Agendada,Interconsulta,CEX,N,N,NaN,BERNARDINO SEGUNDO CAYUPE BELTRÁN,AC00475641,73809819
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,18547328,DR. VÍCTOR MANUEL AVILLO CAROCA,Urología Adulto,08-07-2024,900,Agendada,Derivacion desde APS,PRI,S,S,Cáncer De Testículo En Personas De 15 Años Y Más,Daniel Alejandro Mundaca Carrasco,AC00117760,185473287
82,21129381,DR. PABLO ALEJANDRO DELGADO ARNEZ,Cirugia Cabeza y Cuello,19-07-2024,1145,Agendada,Interconsulta,CEX,S,S,CÁNCER DE TIROIDES DIFERENCIADO Y MEDULAR EN P...,KATHERINE ALEJANDRA PALACIOS BARRIGA,AC00382122,211293810
83,6227519,DR. MARIA GABRIELA CAMPOVERDE BRAVO,Medicina Interna,22-07-2024,1330,Agendada,Derivacion desde APS,PRI,S,S,Cáncer Gástrico,Georgina Garrido Navarrete,AC00036072,62275197
84,10042997,DR. MARIA GABRIELA CAMPOVERDE BRAVO,Medicina Interna,17-07-2024,1400,Agendada,Derivacion desde APS,PRI,S,S,Cáncer Gástrico,Nancy Del Rosario Yáñez Fuentes,AC00065990,100429977


In [173]:
list(df_citas_futuras.columns)

['RUT',
 'NOMBRE_AGENDA',
 'ESPECIALIDAD_AGENDA',
 'CITAFECH',
 'CITAHORA',
 'ESTADO_CITA',
 'DESCRIPCION',
 'ORIGEN',
 'GES_CITA',
 'GES_SOL',
 'PATOLOGIA_GES_SOL',
 'PACIENTE',
 'CITA_CODUTEN',
 'DOCUMENTO']

In [174]:
df_citas_futuras = df_citas_futuras[['RUT',
 'NOMBRE_AGENDA',
 'ESPECIALIDAD_AGENDA',
 'CITAFECH',
 'DESCRIPCION']]

In [175]:
df_citas_futuras['NOMBRE_AGENDA'] = df_citas_futuras['DESCRIPCION']+'-'+df_citas_futuras['NOMBRE_AGENDA']

In [176]:
df_citas_futuras

,RUT,NOMBRE_AGENDA,ESPECIALIDAD_AGENDA,CITAFECH,DESCRIPCION
0,17097810,Control postquirurgico-DR. RUBÉN DARÍO ÁVILA S...,Cirugía Abdominal Adulto,17-07-2024,Control postquirurgico
1,7816012,Control - Consulta integral de especialidades-...,Endocrinología Adulto,03-09-2024,Control - Consulta integral de especialidades
4,9214167,Derivacion desde APS-DR. MARIA GABRIELA CAMPOV...,Medicina Interna,18-07-2024,Derivacion desde APS
7,6811765,Control - Consulta integral de especialidades-...,Cirugía Abdominal Adulto,24-07-2024,Control - Consulta integral de especialidades
12,7380981,Interconsulta-DR. RODRIGO ANTONIO GONZÁLEZ IRR...,Urología Adulto,01-08-2024,Interconsulta
...,...,...,...,...,...
81,18547328,Derivacion desde APS-DR. VÍCTOR MANUEL AVILLO ...,Urología Adulto,08-07-2024,Derivacion desde APS
82,21129381,Interconsulta-DR. PABLO ALEJANDRO DELGADO ARNEZ,Cirugia Cabeza y Cuello,19-07-2024,Interconsulta
83,6227519,Derivacion desde APS-DR. MARIA GABRIELA CAMPOV...,Medicina Interna,22-07-2024,Derivacion desde APS
84,10042997,Derivacion desde APS-DR. MARIA GABRIELA CAMPOV...,Medicina Interna,17-07-2024,Derivacion desde APS


# Guardar Citas Futuras

In [177]:
# Día actual
today = date.today()

# Fecha actual
now = datetime.now()

# Formatear día con ceros a la izquierda si es necesario
day_formatted = "{:02d}".format(today.day)

In [178]:
wb = Workbook()
ws = wb.active

for r in dataframe_to_rows(df_citas_futuras, index= False, header = True):
    ws.append(r)

for cell in ws['D'][1:]:
    cell.number_format = 'DD-MM-YYYY'
nombre_archivo = f'{today.year}{month_formatted}{day_formatted}_citas_futuras_derivaciones.xlsx'
wb.save(os.path.join(dir_actual, '..','DERIVACIONES',f'{nombre_archivo}'))

# Actualizar estados

In [179]:
df_concatenated_3

,Problema Oncologico,RUT,DV,Nombre del Paciente,Estado,Fecha Próximo Hito,Medico Responsable,Especialidad,Fecha de SIC,Estado de SIC,...,Observaciones,Diagnosticos,Telefonos,CONCATENADO,CITAS FUTURAS,MEDICO,ESTADO SIC,Especialidad Destino,DIAGNOSTICOS,Telefono Consolidado2
0,Cáncer Próstata,5926301,3,ADA VILCHES MORALES,Priorizacion CUD,NaN,NaN,Urología Adulto,2024-05-13,NaN,...,NaN,NaN,NaN,5926301Urología Adulto,NaN,NaN,Solicitada,NaN,NaN,NaN
1,Cáncer Tiroides,16698412,2,Canales Marchant Nicole Paulina,Priorizacion CUD,NaN,NaN,Endocrinología Adulto,2024-05-08,NaN,...,NaN,NaN,NaN,16698412Endocrinología Adulto,NaN,NaN,Finalizada,NaN,NaN,NaN
2,Cáncer Próstata,99616630,K,CARLOS AGUILAR SALAS,Priorizacion CUD,NaN,NaN,Urología Adulto,2024-05-02,NaN,...,NaN,NaN,NaN,99616630Urología Adulto,NaN,NaN,NaN,NaN,NaN,NaN
3,Cáncer Pulmón,7408006,5,Cuevas Villalobos Eulalia Iris,Rechazada,NaN,NaN,Broncopulmonar Adulto,2024-06-03,rechazada,...,no pertinente (cud),NaN,NaN,7408006Broncopulmonar Adulto,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,7262387,8,Cuevas Villalobos Rene Edmundo,Priorizacion CUD,NaN,NaN,Dermatología Adulto,2024-06-07,NaN,...,NaN,NaN,NaN,7262387DERMATOLOGIA,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,Cáncer Colorectal,8130920,5,"SILVA HENRÍQUEZ, GINA DE LAS MERCEDES",NaN,NaN,NaN,NaN,2024-07-03,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,- / - / 68262953 / 68262953 / 68262953 / 31898...
207,Cáncer Pulmón,8317900,7,"MELLI NAVARRETE, ESTER LUCRECIA",NaN,NaN,NaN,NaN,2024-06-28,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,- / - / 35766740 / 35766740 / 35766740 / 23140...
223,Cáncer Colorectal,8957749,7,"LEIVA CEPEDA, RODRIGO EDUARDO",NaN,NaN,NaN,NaN,2024-07-02,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,- / - / 0 / 0 / 75530673 / / -/75530673//./FI...
258,Cáncer Gástrico,10492252,K,"VILCHES PÉREZ, MARISA DE LAS MERCED",NaN,NaN,NaN,NaN,2024-06-28,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,- / - / 22828026 / 66931649 / 66931649 / 22828...


In [180]:
dataframe = cargar_dataframe_ambulatorio(rutas_archivos_4)

In [181]:
dataframe[dataframe['NRO_IDENTIFICADOR'] == '92117405'][['ESPECIALIDAD_DESTINO','ESTADO_SOL','FECHA_SOLIC']]

,ESPECIALIDAD_DESTINO,ESTADO_SOL,FECHA_SOLIC
2105,Anestesiología Adulto,Rechazada,01/02/2024
12390,Cirugía Proctológica,Finalizada,08/06/2015
79353,Psiquiatría Adulto,Finalizada,04/10/2016
147503,Periodoncia,Finalizada,25/02/2015
160296,Dermatología Adulto,Solicitada,21/09/2022
179856,Traumatología Adulto,Finalizada,07/09/2021
221557,Medicina Interna,Finalizada,03/07/2023
248311,Gastroenterología Adulto,Finalizada,10/08/2022
332628,Urología Adulto,Finalizada,15/02/2024
332795,Urología Adulto,Rechazada,15/02/2024


In [182]:
dataframe['NRO_IDENTIFICADOR'] = dataframe['NRO_IDENTIFICADOR'].astype(str)
dataframe['NRO_IDENTIFICADOR'] = dataframe['NRO_IDENTIFICADOR'].str.replace('.','', regex = False)
dataframe['NRO_IDENTIFICADOR'] = dataframe['NRO_IDENTIFICADOR'].str.replace('-','', regex = False)
dataframe = dataframe[dataframe['ESPECIALIDAD_DESTINO'].str.contains(regex_esp, na=False)]
dataframe['FECHA_SOLIC'] = pd.to_datetime(dataframe['FECHA_SOLIC'], format='%d/%m/%Y')

In [183]:
dataframe = dataframe[dataframe['TIPO_INDENTIF'] == 'RUN']

In [184]:
dataframe['DV'] = dataframe['NRO_IDENTIFICADOR'].str[-1]

    #resultado_filtrado.loc[condicion, 'NRO_IDENTIFICADOR_PACIENTE'] = resultado_filtrado.loc[condicion, 'NRO_IDENTIFICADOR_PACIENTE'].str[:-1]
dataframe['RUT'] = dataframe['NRO_IDENTIFICADOR'].str[:-1]

In [185]:
dataframe[['RUT','ESTADO_SOL','FECHA_SOLIC','ESPECIALIDAD_DESTINO']]

,RUT,ESTADO_SOL,FECHA_SOLIC,ESPECIALIDAD_DESTINO
2,23999606,Finalizada,2016-01-01,Broncopulmonar Infantil
10,6977536,Finalizada,2019-12-30,Cirugía Adulto
11,5853319,Finalizada,2019-12-27,Traumatología Adulto
12,15378626,Rechazada,2019-12-30,Neurología Adulto
14,26618530,Finalizada,2020-01-01,Neurología Infantil
...,...,...,...,...
697800,17783918,Finalizada,2022-12-31,Hematología Adulto
697801,28322632,Finalizada,2023-12-31,Traumatología Infantil
697803,23458325,Finalizada,2023-12-31,Endocrinología Infantil
697809,6131263,Finalizada,2023-12-31,Medicina Interna


In [186]:
dataframe

,PAC_ID,TIPO_INDENTIF,NRO_IDENTIFICADOR,NOM_PAC,APELLPAT_PAC,APELLMAT_PAC,FECHA_NAC_PAC,EDAD_PAC,SEXO,NACIONALIDAD_PAC,...,ESTADO_SOL,FECHA_AUTORIZ,RUNDV_AUTORIZ,NOM_AUTORIZ,CAUSAL_SALIDA_NRO,CAUSAL_SALIDA_DESC,COMENT_RECHAZO,FECHA_CAUSAL_SALIDA,DV,RUT
2,AC00243286,RUN,23999606K,AGUSTINA ISIDORA,GARRIDO,GARRIDO,30/06/2012,"3 Años, 6 Mes(es) y 1 días",Mujer,Chile,...,Finalizada,04-01-2016,116250470,PAULA ANDREA ARRIAGADA MITROVIC,NaN,NaN,NaN,17/02/2016,K,23999606
10,AC00042909,RUN,69775365,GLADYS DEL ROSARIO,ARRUÉ,ACEVEDO,13/08/1955,64,Mujer,Chile,...,Finalizada,04-05-2020,137072025,MILITZA JANE AGUILERA PASTENE,NaN,NaN,NaN,16/12/2020,5,6977536
11,AC00032409,RUN,5853319K,NORA DEL CARMEN,FERNÁNDEZ,CHANDÍA,15/08/1947,72,Mujer,Chile,...,Finalizada,06-01-2020,132508852,HÉCTOR DANIEL LARA ESPINA,NaN,NaN,NaN,03/03/2020,K,5853319
12,AC00095121,RUN,153786267,MARCO ANTONIO,ACEVEDO,MORALES,01/05/1982,37,Hombre,Chile,...,Rechazada,08-01-2020,118947886,JOSÉ FRANCISCO VALLEJOS CASTRO,1,Atención Realizada,pcte atendido 05-03-2018|ya en control en neur...,05/04/2022,7,15378626
14,AC00370995,RUN,266185308,JUAN JEREMÍAS,BERRIOS,TAIMANE,11/12/2018,"1 Años, 0 Mes(es) y 20 días",Hombre,Chile,...,Finalizada,06-01-2020,116250470,PAULA ANDREA ARRIAGADA MITROVIC,NaN,NaN,NaN,11/03/2020,8,26618530
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
697800,AC00427101,RUN,177839183,JAVIER FRANCISCO,CASTILLO,CANIO,02/07/1991,31,Hombre,Chile,...,Finalizada,03-01-2023,166517214,FELIPE ALFONSO PALACIOS CÁRDENAS,NaN,NaN,NaN,24/01/2023,3,17783918
697801,AC00533794,RUN,283226328,CONSUELO NOEMÍ,ZAMORANO,DUARTE,29/12/2023,"0 Años, 0 Mes(es) y 2 días",Mujer,Chile,...,Finalizada,08-02-2024,100584344,ANA MARÍA ARAVENA PARAS,NaN,NaN,NaN,20/02/2024,8,28322632
697803,AC00425532,RUN,234583255,SOFÍA IGNACIA,SOLÍS,ÁVALOS,22/10/2010,"13 Años, 2 Mes(es) y 9 días",Mujer,Chile,...,Finalizada,02-01-2024,155913657,CRISTIAN ANDRÉS SEILTGENS SORREL,NaN,NaN,NaN,19/03/2024,5,23458325
697809,AC00533874,RUN,61312633,ANA FELICIDAD,MELÉNDEZ,ROJAS,20/01/1947,76,Mujer,Chile,...,Finalizada,02-01-2024,129750413,ANA XIMENA GARRIDO JARA,NaN,NaN,NaN,19/01/2024,3,6131263


In [187]:
dataframe[dataframe['NRO_IDENTIFICADOR'] == '92117405'][['ESPECIALIDAD_DESTINO','ESTADO_SOL','FECHA_SOLIC']]

,ESPECIALIDAD_DESTINO,ESTADO_SOL,FECHA_SOLIC
160296,Dermatología Adulto,Solicitada,2022-09-21
179856,Traumatología Adulto,Finalizada,2021-09-07
221557,Medicina Interna,Finalizada,2023-07-03
248311,Gastroenterología Adulto,Finalizada,2022-08-10
332628,Urología Adulto,Finalizada,2024-02-15
332795,Urología Adulto,Rechazada,2024-02-15
358396,Traumatología Adulto,Finalizada,2022-03-18
644239,Urología Adulto,Rechazada,2024-01-29


In [188]:
dataframe[dataframe['RUT'] == '9211740'][['RUT','ESPECIALIDAD_DESTINO','ESTADO_SOL','FECHA_SOLIC']]

,RUT,ESPECIALIDAD_DESTINO,ESTADO_SOL,FECHA_SOLIC
160296,9211740,Dermatología Adulto,Solicitada,2022-09-21
179856,9211740,Traumatología Adulto,Finalizada,2021-09-07
221557,9211740,Medicina Interna,Finalizada,2023-07-03
248311,9211740,Gastroenterología Adulto,Finalizada,2022-08-10
332628,9211740,Urología Adulto,Finalizada,2024-02-15
332795,9211740,Urología Adulto,Rechazada,2024-02-15
358396,9211740,Traumatología Adulto,Finalizada,2022-03-18
644239,9211740,Urología Adulto,Rechazada,2024-01-29


In [189]:
# Convertir FECHA_SOLIC a tipo datetime para poder comparar fechas
dataframe['FECHA_SOLIC'] = pd.to_datetime(dataframe['FECHA_SOLIC'])

# Ordenar el DataFrame por RUT, ESPECIALIDAD_DESTINO y FECHA_SOLIC descendientemente
dataframe = dataframe.sort_values(by=['RUT', 'ESPECIALIDAD_DESTINO', 'FECHA_SOLIC'], ascending=[True, True, False])

# Quitar duplicados manteniendo el primero (que es el más reciente por la ordenación anterior)
dataframe = dataframe.drop_duplicates(subset=['RUT', 'ESPECIALIDAD_DESTINO'], keep='first')

In [ ]:
dataframe[dataframe['RUT'] == '9211740'][['ESPECIALIDAD_DESTINO','ESTADO_SOL','FECHA_SOLIC']]

,ESPECIALIDAD_DESTINO,ESTADO_SOL,FECHA_SOLIC
160296,Dermatología Adulto,Solicitada,2022-09-21
248311,Gastroenterología Adulto,Finalizada,2022-08-10
221557,Medicina Interna,Finalizada,2023-07-03
358396,Traumatología Adulto,Finalizada,2022-03-18
332628,Urología Adulto,Finalizada,2024-02-15


In [ ]:
list(df_concatenated_3.columns)

['Problema Oncologico',
 'RUT',
 'DV',
 'Nombre del Paciente',
 'Estado',
 'Fecha Próximo Hito',
 'Medico Responsable',
 'Especialidad',
 'Fecha de SIC',
 'Estado de SIC',
 'Establecimiento de Origen',
 'Observaciones',
 'Diagnosticos',
 'Telefonos',
 'CONCATENADO',
 'CITAS FUTURAS',
 'MEDICO',
 'ESTADO SIC',
 'Especialidad Destino',
 'DIAGNOSTICOS',
 'Telefono Consolidado2']

In [ ]:
data_original = df_concatenated_3.copy()

In [ ]:
list(data_original.columns)

['Problema Oncologico',
 'RUT',
 'DV',
 'Nombre del Paciente',
 'Estado',
 'Fecha Próximo Hito',
 'Medico Responsable',
 'Especialidad',
 'Fecha de SIC',
 'Estado de SIC',
 'Establecimiento de Origen',
 'Observaciones',
 'Diagnosticos',
 'Telefonos',
 'CONCATENADO',
 'CITAS FUTURAS',
 'MEDICO',
 'ESTADO SIC',
 'Especialidad Destino',
 'DIAGNOSTICOS',
 'Telefono Consolidado2']

In [ ]:
data_original = data_original.rename(columns={'Estado de SIC': 'ESTADO_SOL'})
data_original = data_original.rename(columns={'Fecha de SIC': 'FECHA_SOLIC'})
data_original = data_original.rename(columns={'Especialidad': 'ESPECIALIDAD_DESTINO'})

In [ ]:
df_original = data_original[['RUT','ESTADO_SOL','FECHA_SOLIC','ESPECIALIDAD_DESTINO']].copy()

In [ ]:
df_actualizado = dataframe[['RUT','ESTADO_SOL','FECHA_SOLIC','ESPECIALIDAD_DESTINO']].copy()

In [ ]:
df_actualizado[dataframe['RUT'] == '9211740'][['ESPECIALIDAD_DESTINO','ESTADO_SOL','FECHA_SOLIC']]

,ESPECIALIDAD_DESTINO,ESTADO_SOL,FECHA_SOLIC
160296,Dermatología Adulto,Solicitada,2022-09-21
248311,Gastroenterología Adulto,Finalizada,2022-08-10
221557,Medicina Interna,Finalizada,2023-07-03
358396,Traumatología Adulto,Finalizada,2022-03-18
332628,Urología Adulto,Finalizada,2024-02-15


In [ ]:
df_actualizado['FECHA_SOLIC'] = pd.to_datetime(df_actualizado['FECHA_SOLIC'], format='%d/%m/%Y')
df_original['FECHA_SOLIC'] = pd.to_datetime(df_original['FECHA_SOLIC'], format='%d/%m/%Y')

In [ ]:
df_actualizado['RUT'] = df_actualizado['RUT'].astype(int)
df_original['RUT'] = df_original['RUT'].astype(int)

In [ ]:
import pandas as pd




# Función para agregar la columna de comparación
def agregar_columna_comparacion(df_original, df_actualizado):
    # Unir los dataframes por las columnas 'RUT', 'FECHA_SOLIC' y 'ESPECIALIDAD_DESTINO'
    df_combinado = df_original.merge(df_actualizado, on=['RUT', 
                                                         #'FECHA_SOLIC', 
                                                         'ESPECIALIDAD_DESTINO'], suffixes=('_antiguo', '_nuevo'))
    
    # Crear la columna de comparación
    df_combinado['CAMBIO_ESTADO'] = df_combinado['ESTADO_SOL_antiguo'] != df_combinado['ESTADO_SOL_nuevo']
    
    return df_combinado

# Usar la función
df_combinado = agregar_columna_comparacion(df_original, df_actualizado)



In [ ]:
df_combinado

,RUT,ESTADO_SOL_antiguo,FECHA_SOLIC_antiguo,ESPECIALIDAD_DESTINO,ESTADO_SOL_nuevo,FECHA_SOLIC_nuevo,CAMBIO_ESTADO
0,6402728,Citada,2024-04-11,Urología Adulto,Solicitada,2024-04-11,True
1,7380981,Citada,2023-12-18,Urología Adulto,Citada,2023-12-18,False
2,9211740,Finalizada,2024-01-29,Urología Adulto,Finalizada,2024-02-15,False
3,5743029,NaN,2024-06-05,Cirugía Abdominal Adulto,Finalizada,2024-05-31,True
4,7816012,NaN,2024-06-03,Endocrinología Adulto,Finalizada,2024-05-28,True
...,...,...,...,...,...,...,...
256,8029404,Solicitada,2024-06-17,Urología Adulto,Citada,2024-06-17,True
257,9034836,Solicitada,2024-06-17,Urología Adulto,Citada,2024-06-17,True
258,10675861,Pendiente de Autorizar,2024-06-20,Endocrinología Adulto,Pendiente de Autorizar,2024-06-20,False
259,18547328,Solicitada,2024-06-17,Urología Adulto,Citada,2024-06-17,True


In [ ]:
df_combinado[df_combinado['CAMBIO_ESTADO']== True]

,RUT,ESTADO_SOL_antiguo,FECHA_SOLIC_antiguo,ESPECIALIDAD_DESTINO,ESTADO_SOL_nuevo,FECHA_SOLIC_nuevo,CAMBIO_ESTADO
0,6402728,Citada,2024-04-11,Urología Adulto,Solicitada,2024-04-11,True
3,5743029,NaN,2024-06-05,Cirugía Abdominal Adulto,Finalizada,2024-05-31,True
4,7816012,NaN,2024-06-03,Endocrinología Adulto,Finalizada,2024-05-28,True
11,19733822,Citada,2024-02-29,Urología Adulto,Solicitada,2024-02-29,True
19,21408209,Solicitada,2024-05-29,Endocrinología Adulto,Citada,2024-05-29,True
...,...,...,...,...,...,...,...
254,4022868,Pendiente de Autorizar,2024-06-18,Hematología Adulto,Solicitada,2024-06-18,True
256,8029404,Solicitada,2024-06-17,Urología Adulto,Citada,2024-06-17,True
257,9034836,Solicitada,2024-06-17,Urología Adulto,Citada,2024-06-17,True
259,18547328,Solicitada,2024-06-17,Urología Adulto,Citada,2024-06-17,True


In [ ]:
list(df_combinado.columns)

['RUT',
 'ESTADO_SOL_antiguo',
 'FECHA_SOLIC_antiguo',
 'ESPECIALIDAD_DESTINO',
 'ESTADO_SOL_nuevo',
 'FECHA_SOLIC_nuevo',
 'CAMBIO_ESTADO']

In [ ]:
list(df_combinado.columns)

['RUT',
 'ESTADO_SOL_antiguo',
 'FECHA_SOLIC_antiguo',
 'ESPECIALIDAD_DESTINO',
 'ESTADO_SOL_nuevo',
 'FECHA_SOLIC_nuevo',
 'CAMBIO_ESTADO']

In [ ]:
df_estados = df_combinado[['RUT',
 'FECHA_SOLIC_nuevo',
 'ESPECIALIDAD_DESTINO',
 'ESTADO_SOL_nuevo']].copy()

In [ ]:
df_estados

,RUT,FECHA_SOLIC_nuevo,ESPECIALIDAD_DESTINO,ESTADO_SOL_nuevo
0,6402728,2024-04-11,Urología Adulto,Solicitada
1,7380981,2023-12-18,Urología Adulto,Citada
2,9211740,2024-02-15,Urología Adulto,Finalizada
3,5743029,2024-05-31,Cirugía Abdominal Adulto,Finalizada
4,7816012,2024-05-28,Endocrinología Adulto,Finalizada
...,...,...,...,...
256,8029404,2024-06-17,Urología Adulto,Citada
257,9034836,2024-06-17,Urología Adulto,Citada
258,10675861,2024-06-20,Endocrinología Adulto,Pendiente de Autorizar
259,18547328,2024-06-17,Urología Adulto,Citada


In [ ]:
df_estados[df_estados['RUT']== 9211740]

,RUT,FECHA_SOLIC_nuevo,ESPECIALIDAD_DESTINO,ESTADO_SOL_nuevo
2,9211740,2024-02-15,Urología Adulto,Finalizada


In [ ]:
df_estados['CONCATENADO'] = df_estados['RUT'].astype(str) + df_estados['ESPECIALIDAD_DESTINO'] 

In [ ]:
df_estados

,RUT,FECHA_SOLIC_nuevo,ESPECIALIDAD_DESTINO,ESTADO_SOL_nuevo,CONCATENADO
0,6402728,2024-04-11,Urología Adulto,Solicitada,6402728Urología Adulto
1,7380981,2023-12-18,Urología Adulto,Citada,7380981Urología Adulto
2,9211740,2024-02-15,Urología Adulto,Finalizada,9211740Urología Adulto
3,5743029,2024-05-31,Cirugía Abdominal Adulto,Finalizada,5743029Cirugía Abdominal Adulto
4,7816012,2024-05-28,Endocrinología Adulto,Finalizada,7816012Endocrinología Adulto
...,...,...,...,...,...
256,8029404,2024-06-17,Urología Adulto,Citada,8029404Urología Adulto
257,9034836,2024-06-17,Urología Adulto,Citada,9034836Urología Adulto
258,10675861,2024-06-20,Endocrinología Adulto,Pendiente de Autorizar,10675861Endocrinología Adulto
259,18547328,2024-06-17,Urología Adulto,Citada,18547328Urología Adulto


In [ ]:
list(df_estados.columns)

['RUT',
 'FECHA_SOLIC_nuevo',
 'ESPECIALIDAD_DESTINO',
 'ESTADO_SOL_nuevo',
 'CONCATENADO']

In [ ]:
lista = ['CONCATENADO','RUT',
 'FECHA_SOLIC_nuevo',
 'ESPECIALIDAD_DESTINO',
 'ESTADO_SOL_nuevo']

In [ ]:
df_estados

,RUT,FECHA_SOLIC_nuevo,ESPECIALIDAD_DESTINO,ESTADO_SOL_nuevo,CONCATENADO
0,6402728,2024-04-11,Urología Adulto,Solicitada,6402728Urología Adulto
1,7380981,2023-12-18,Urología Adulto,Citada,7380981Urología Adulto
2,9211740,2024-02-15,Urología Adulto,Finalizada,9211740Urología Adulto
3,5743029,2024-05-31,Cirugía Abdominal Adulto,Finalizada,5743029Cirugía Abdominal Adulto
4,7816012,2024-05-28,Endocrinología Adulto,Finalizada,7816012Endocrinología Adulto
...,...,...,...,...,...
256,8029404,2024-06-17,Urología Adulto,Citada,8029404Urología Adulto
257,9034836,2024-06-17,Urología Adulto,Citada,9034836Urología Adulto
258,10675861,2024-06-20,Endocrinología Adulto,Pendiente de Autorizar,10675861Endocrinología Adulto
259,18547328,2024-06-17,Urología Adulto,Citada,18547328Urología Adulto


In [ ]:
df_estados[lista]

,CONCATENADO,RUT,FECHA_SOLIC_nuevo,ESPECIALIDAD_DESTINO,ESTADO_SOL_nuevo
0,6402728Urología Adulto,6402728,2024-04-11,Urología Adulto,Solicitada
1,7380981Urología Adulto,7380981,2023-12-18,Urología Adulto,Citada
2,9211740Urología Adulto,9211740,2024-02-15,Urología Adulto,Finalizada
3,5743029Cirugía Abdominal Adulto,5743029,2024-05-31,Cirugía Abdominal Adulto,Finalizada
4,7816012Endocrinología Adulto,7816012,2024-05-28,Endocrinología Adulto,Finalizada
...,...,...,...,...,...
256,8029404Urología Adulto,8029404,2024-06-17,Urología Adulto,Citada
257,9034836Urología Adulto,9034836,2024-06-17,Urología Adulto,Citada
258,10675861Endocrinología Adulto,10675861,2024-06-20,Endocrinología Adulto,Pendiente de Autorizar
259,18547328Urología Adulto,18547328,2024-06-17,Urología Adulto,Citada


In [ ]:
wb = Workbook()
ws = wb.active

for r in dataframe_to_rows(df_estados[lista], index= False, header = True):
    ws.append(r)

for cell in ws['C'][1:]:
    cell.number_format = 'DD-MM-YYYY'
nombre_archivo = f'{today.year}{month_formatted}{day_formatted}_estados_derivaciones.xlsx'
wb.save(os.path.join(dir_actual, '..','DERIVACIONES',f'{nombre_archivo}'))